Imports

In [167]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, date, timedelta
from faker import Faker
from faker.providers import BaseProvider
from math import ceil

AMR Spreadsheet

AMRID -> process

SupplierID -> extract  + process

Assignedby -> person + process

DateAssigned -> process

Destination -> process

DateSubmitted -> extract

AmountAvailable -> extract

Synthesis -> should be n/a for extracts

SourceId -> extract

SourceType -> extract

Species -> Get from genomics or collection

Permit -> collection

Pure -> Yes for pure compounds, no for extracts

MS -> just link if there to process

NMR -> just link if there to process

ResearchGroup -> extract

Submittedby -> person + extract

Solubility -> n/a for extracts

Stereo comments -> n/a for extracts

Solvent for extraction -> extract

SMILE -> n/a for extract

MW -> n/a for extract

AdditionalMetadata -> removed

ExistingPatent -> n/a for extract

ExistingLiterature -> extract

Comments -> removed

In [168]:
fake = Faker()
Faker.seed(1234)
random.seed(1234)

class Provider(BaseProvider):
    submitters = ['Ruby Johnson', 'Robert Hobson', 'Julia Reed']
    assigners = ['Ruby Johnson', 'Joshua Keener']
    destinations = ['Rhodes Bioassay', 'Rhodes MS', 'Rhodes Bioassay/Rhodes MS', 'Rhodes Bioassay, PK assay', 'Rhodes MS/NMR', 'Membrane Disruption assay', 'Rhodes LC/MS + NMR & Bioassay']
    start_date = datetime(2050, 1, 1)
    end_date = datetime(2055, 1, 1)
    binaryChoice = ['Y', 'N']
    existingLit = ['Y', 'N', 'https//doi.org/choice1', 'https//doi.org/choice2', 'https//doi.org/choice3', 'https//doi.org/choice4', 'https//doi.org/choice5', 'https//doi.org/choice6']
    researchgroups = ['Smith', 'Roodt', 'Wellington', 'Preston']
    solvents = ['DMSO', 'Methanol']

    def year(self):
        return random.randint(2041, 2049)

    def submittedDate(self):
        self.submittedDate = fake.date_between_dates(
            date_start=self.start_date, date_end=self.end_date).strftime('%d/%m/%Y')

        return self.submittedDate
    
    def submittedBy(self):
        return random.choice(self.submitters)

    def assignedBy(self):
        return random.choice(self.assigners)

    def destinationOfSample(self):
        return random.choice(self.destinations)

    def amountAvailable(self):
        return (round (random.uniform(0.0, 1.0)*100, 1))

    def synthesisPotential(self):
        return "N/A"

    def species(self):
        return fake.word()

    def pure(self):
        return "N"

    def ms(self):
        return random.choice(self.binaryChoice)
    
    def nmr(self):
        return random.choice(self.binaryChoice)

    def researchGroup(self):
        return random.choice(self.researchgroups)

    def solubility(self):
        return "N/A"

    def stereoComments(self):
        return "N/A"

    def smileStructure(self):
        return "N/A"

    def mw(self):
        return "N/A"

    def additionalMetadata(self):
        return random.choice(self.binaryChoice)

    def existingPatent(self):
        return "N"

    def existingLiterature(self):
        return random.choice(self.existingLit)
    
    def solvent(self):
        return random.choice(self.solvents)

# Add the Provider to our faker object
fake.add_provider(Provider)

# Go up by one for AMR ID
# Supplier ID - based on date of collection, so must be same or earlier year than submittedDate
#               some TIC, some SAF, Some strains (e.g. S 20 20), MIAP_155, HMF-1, MBA_F406_PDM
# Date assigned - do a week after submittedDate
# Source ID - TIC number, MIAP etc - based on Supplier ID (start with this, then make Supplier ID)
# Source type - based on source ID (microbial vs sponge vs stromatolite)
# Permit - RES and year before or same as source ID

def create_AMR_data(fake, no_of_rows):
    columns = ['year', 'assignedBy', 'destinationOfSample', 'submittedDate', 'amountAvailable', 'synthesisPotential', 'species', 'pure', 'ms', 'nmr',
                'researchGroup', 'submittedBy', 'solubility', 'stereoComments', 'smileStructure', 'mw', 'additionalMetadata', 'existingPatent', 'existingLiterature',
                'solvent']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

original_df = create_AMR_data(fake, 100000)
original_df

,year,assignedBy,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,species,pure,ms,nmr,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,2048,Ruby Johnson,Rhodes Bioassay/Rhodes MS,01/10/2053,30.2,N/A,suddenly,N,Y,Y,Preston,Ruby Johnson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
1,2042,Ruby Johnson,Membrane Disruption assay,30/12/2050,26.6,N/A,character,N,Y,Y,Smith,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,DMSO
2,2041,Joshua Keener,Rhodes MS/NMR,24/01/2050,9.3,N/A,near,N,N,N,Wellington,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
3,2042,Ruby Johnson,Membrane Disruption assay,09/10/2050,31.7,N/A,can,N,N,Y,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice1,DMSO
4,2041,Joshua Keener,Rhodes MS/NMR,14/12/2054,66.1,N/A,maintain,N,Y,Y,Wellington,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2042,Joshua Keener,"Rhodes Bioassay, PK assay",28/05/2053,31.0,N/A,foreign,N,N,N,Roodt,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
99996,2041,Joshua Keener,Rhodes MS/NMR,22/01/2052,58.4,N/A,arrive,N,N,N,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,Methanol
99997,2048,Ruby Johnson,Rhodes MS,31/07/2051,54.7,N/A,carry,N,Y,Y,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice5,DMSO
99998,2043,Joshua Keener,Rhodes Bioassay/Rhodes MS,31/01/2051,8.3,N/A,stuff,N,Y,N,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice1,DMSO


Adding in AMR ID

In [169]:
# Go up by one for AMR ID
original_df.insert(0, 'amrID', range(1, 1 + len(original_df)))
original_df
print(original_df.dtypes)

amrID                    int64
year                     int64
assignedBy              object
destinationOfSample     object
submittedDate           object
amountAvailable        float64
synthesisPotential      object
species                 object
pure                    object
ms                      object
nmr                     object
researchGroup           object
submittedBy             object
solubility              object
stereoComments          object
smileStructure          object
mw                      object
additionalMetadata      object
existingPatent          object
existingLiterature      object
solvent                 object
dtype: object


In [170]:
original_df['amrID'] = 'AMR' + original_df['amrID'].astype(str).str.zfill(5)
print(original_df.dtypes)

amrID                   object
year                     int64
assignedBy              object
destinationOfSample     object
submittedDate           object
amountAvailable        float64
synthesisPotential      object
species                 object
pure                    object
ms                      object
nmr                     object
researchGroup           object
submittedBy             object
solubility              object
stereoComments          object
smileStructure          object
mw                      object
additionalMetadata      object
existingPatent          object
existingLiterature      object
solvent                 object
dtype: object


In [171]:
original_df

,amrID,year,assignedBy,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,species,pure,ms,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,2048,Ruby Johnson,Rhodes Bioassay/Rhodes MS,01/10/2053,30.2,N/A,suddenly,N,Y,...,Preston,Ruby Johnson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
1,AMR00002,2042,Ruby Johnson,Membrane Disruption assay,30/12/2050,26.6,N/A,character,N,Y,...,Smith,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,DMSO
2,AMR00003,2041,Joshua Keener,Rhodes MS/NMR,24/01/2050,9.3,N/A,near,N,N,...,Wellington,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
3,AMR00004,2042,Ruby Johnson,Membrane Disruption assay,09/10/2050,31.7,N/A,can,N,N,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice1,DMSO
4,AMR00005,2041,Joshua Keener,Rhodes MS/NMR,14/12/2054,66.1,N/A,maintain,N,Y,...,Wellington,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,AMR99996,2042,Joshua Keener,"Rhodes Bioassay, PK assay",28/05/2053,31.0,N/A,foreign,N,N,...,Roodt,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
99996,AMR99997,2041,Joshua Keener,Rhodes MS/NMR,22/01/2052,58.4,N/A,arrive,N,N,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,Methanol
99997,AMR99998,2048,Ruby Johnson,Rhodes MS,31/07/2051,54.7,N/A,carry,N,Y,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice5,DMSO
99998,AMR99999,2043,Joshua Keener,Rhodes Bioassay/Rhodes MS,31/01/2051,8.3,N/A,stuff,N,Y,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice1,DMSO


Date Assigned column addition

In [172]:
# Date assigned - do a week after submittedDate
original_df.submittedDate = pd.to_datetime(original_df.submittedDate, format='%d/%m/%Y')
original_df.insert(3, 'assignedDate', original_df['submittedDate'])
original_df

,amrID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,species,pure,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,2048,Ruby Johnson,2053-10-01,Rhodes Bioassay/Rhodes MS,2053-10-01,30.2,N/A,suddenly,N,...,Preston,Ruby Johnson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
1,AMR00002,2042,Ruby Johnson,2050-12-30,Membrane Disruption assay,2050-12-30,26.6,N/A,character,N,...,Smith,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,DMSO
2,AMR00003,2041,Joshua Keener,2050-01-24,Rhodes MS/NMR,2050-01-24,9.3,N/A,near,N,...,Wellington,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
3,AMR00004,2042,Ruby Johnson,2050-10-09,Membrane Disruption assay,2050-10-09,31.7,N/A,can,N,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice1,DMSO
4,AMR00005,2041,Joshua Keener,2054-12-14,Rhodes MS/NMR,2054-12-14,66.1,N/A,maintain,N,...,Wellington,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,AMR99996,2042,Joshua Keener,2053-05-28,"Rhodes Bioassay, PK assay",2053-05-28,31.0,N/A,foreign,N,...,Roodt,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
99996,AMR99997,2041,Joshua Keener,2052-01-22,Rhodes MS/NMR,2052-01-22,58.4,N/A,arrive,N,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,Methanol
99997,AMR99998,2048,Ruby Johnson,2051-07-31,Rhodes MS,2051-07-31,54.7,N/A,carry,N,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice5,DMSO
99998,AMR99999,2043,Joshua Keener,2051-01-31,Rhodes Bioassay/Rhodes MS,2051-01-31,8.3,N/A,stuff,N,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice1,DMSO


In [173]:

original_df['assignedDate'] = original_df['assignedDate'] + timedelta(days=7)
original_df['assignedDate'] = original_df['assignedDate'].dt.strftime('%d/%m/%Y')
original_df['submittedDate'] = original_df['submittedDate'].dt.strftime('%d/%m/%Y')
original_df

,amrID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,species,pure,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,2048,Ruby Johnson,08/10/2053,Rhodes Bioassay/Rhodes MS,01/10/2053,30.2,N/A,suddenly,N,...,Preston,Ruby Johnson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
1,AMR00002,2042,Ruby Johnson,06/01/2051,Membrane Disruption assay,30/12/2050,26.6,N/A,character,N,...,Smith,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,DMSO
2,AMR00003,2041,Joshua Keener,31/01/2050,Rhodes MS/NMR,24/01/2050,9.3,N/A,near,N,...,Wellington,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
3,AMR00004,2042,Ruby Johnson,16/10/2050,Membrane Disruption assay,09/10/2050,31.7,N/A,can,N,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice1,DMSO
4,AMR00005,2041,Joshua Keener,21/12/2054,Rhodes MS/NMR,14/12/2054,66.1,N/A,maintain,N,...,Wellington,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,AMR99996,2042,Joshua Keener,04/06/2053,"Rhodes Bioassay, PK assay",28/05/2053,31.0,N/A,foreign,N,...,Roodt,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
99996,AMR99997,2041,Joshua Keener,29/01/2052,Rhodes MS/NMR,22/01/2052,58.4,N/A,arrive,N,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,Methanol
99997,AMR99998,2048,Ruby Johnson,07/08/2051,Rhodes MS,31/07/2051,54.7,N/A,carry,N,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice5,DMSO
99998,AMR99999,2043,Joshua Keener,07/02/2051,Rhodes Bioassay/Rhodes MS,31/01/2051,8.3,N/A,stuff,N,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice1,DMSO


Adding Permit column

In [174]:
# Permit - RES and year before or same as source ID
original_df.insert(9, 'permitNo', original_df['year'].astype(str))
original_df['permitNo'] = 'RES' + original_df['permitNo']
original_df

,amrID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,species,permitNo,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,2048,Ruby Johnson,08/10/2053,Rhodes Bioassay/Rhodes MS,01/10/2053,30.2,N/A,suddenly,RES2048,...,Preston,Ruby Johnson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
1,AMR00002,2042,Ruby Johnson,06/01/2051,Membrane Disruption assay,30/12/2050,26.6,N/A,character,RES2042,...,Smith,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,DMSO
2,AMR00003,2041,Joshua Keener,31/01/2050,Rhodes MS/NMR,24/01/2050,9.3,N/A,near,RES2041,...,Wellington,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
3,AMR00004,2042,Ruby Johnson,16/10/2050,Membrane Disruption assay,09/10/2050,31.7,N/A,can,RES2042,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice1,DMSO
4,AMR00005,2041,Joshua Keener,21/12/2054,Rhodes MS/NMR,14/12/2054,66.1,N/A,maintain,RES2041,...,Wellington,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,AMR99996,2042,Joshua Keener,04/06/2053,"Rhodes Bioassay, PK assay",28/05/2053,31.0,N/A,foreign,RES2042,...,Roodt,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
99996,AMR99997,2041,Joshua Keener,29/01/2052,Rhodes MS/NMR,22/01/2052,58.4,N/A,arrive,RES2041,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,Methanol
99997,AMR99998,2048,Ruby Johnson,07/08/2051,Rhodes MS,31/07/2051,54.7,N/A,carry,RES2048,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice5,DMSO
99998,AMR99999,2043,Joshua Keener,07/02/2051,Rhodes Bioassay/Rhodes MS,31/01/2051,8.3,N/A,stuff,RES2043,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice1,DMSO


Adding Source ID

In [175]:
# Source ID - TIC number, MIAP etc - start with this, then make Supplier ID
# Examples: TIC2040-001; MOZ42-001; SAF43-001; MIAP_207(B/AIA/ISP); CSF_1ABC_20; 05-008*#5; OB4-REP3_026; AAP_14; SU-Fungi-A16
original_df.insert(8, 'sourceID', original_df['year'].astype(str))
original_df

,amrID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,species,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,2048,Ruby Johnson,08/10/2053,Rhodes Bioassay/Rhodes MS,01/10/2053,30.2,N/A,2048,suddenly,...,Preston,Ruby Johnson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
1,AMR00002,2042,Ruby Johnson,06/01/2051,Membrane Disruption assay,30/12/2050,26.6,N/A,2042,character,...,Smith,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,DMSO
2,AMR00003,2041,Joshua Keener,31/01/2050,Rhodes MS/NMR,24/01/2050,9.3,N/A,2041,near,...,Wellington,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
3,AMR00004,2042,Ruby Johnson,16/10/2050,Membrane Disruption assay,09/10/2050,31.7,N/A,2042,can,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice1,DMSO
4,AMR00005,2041,Joshua Keener,21/12/2054,Rhodes MS/NMR,14/12/2054,66.1,N/A,2041,maintain,...,Wellington,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,AMR99996,2042,Joshua Keener,04/06/2053,"Rhodes Bioassay, PK assay",28/05/2053,31.0,N/A,2042,foreign,...,Roodt,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
99996,AMR99997,2041,Joshua Keener,29/01/2052,Rhodes MS/NMR,22/01/2052,58.4,N/A,2041,arrive,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,Methanol
99997,AMR99998,2048,Ruby Johnson,07/08/2051,Rhodes MS,31/07/2051,54.7,N/A,2048,carry,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice5,DMSO
99998,AMR99999,2043,Joshua Keener,07/02/2051,Rhodes Bioassay/Rhodes MS,31/01/2051,8.3,N/A,2043,stuff,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice1,DMSO


In [176]:
numTIC = len(original_df[original_df.year == 2041])
numMOZ = len(original_df[original_df.year == 2042])
numSAF = len(original_df[original_df.year == 2043])
numMIAP = len(original_df[original_df.year == 2044])
numCSF = len(original_df[original_df.year == 2045])
numStrain = len(original_df[original_df.year == 2046])
numOB = len(original_df[original_df.year == 2047])
numAAP = len(original_df[original_df.year == 2048])
numSU = len(original_df[original_df.year == 2049])

In [177]:
# Examples: TIC2040-001; AMRU; MOZ42-001; SAF43-001; MIAP_207(B/AIA/ISP); CSF_1ABC_20; 05-008*#5; OB4-REP3_026; AAP_14; SU-Fungi-A16
original_df.loc[original_df['year'] == 2041, 'sourceID'] = 'TIC' + original_df['sourceID']
original_df.loc[original_df['year'] == 2041, 'sourceID'] = original_df.loc[original_df['year'] == 2041, 'sourceID'].str.cat(map(str, range(1, 1 + numTIC)), sep='-')
original_df.loc[original_df['year'] == 2042, 'sourceID'] = 'MOZ42'
original_df.loc[original_df['year'] == 2042, 'sourceID'] = original_df.loc[original_df['year'] == 2042, 'sourceID'].str.cat(map(str, range(1, 1 + numMOZ)), sep='-')
original_df.loc[original_df['year'] == 2043, 'sourceID'] = 'SAF43'
original_df.loc[original_df['year'] == 2043, 'sourceID'] = original_df.loc[original_df['year'] == 2043, 'sourceID'].str.cat(map(str, range(1, 1 + numSAF)), sep='-')
original_df.loc[original_df['year'] == 2044, 'sourceID'] = 'MIAP'
original_df.loc[original_df['year'] == 2044, 'sourceID'] = original_df.loc[original_df['year'] == 2044, 'sourceID'].str.cat(map(str, range(1, 1 + numMIAP)), sep='_')
original_df.loc[original_df['year'] == 2045, 'sourceID'] = 'CSF_1ABC'
original_df.loc[original_df['year'] == 2045, 'sourceID'] = original_df.loc[original_df['year'] == 2045, 'sourceID'].str.cat(map(str, np.array(range(1, 1 + numCSF)) // 3 + 1), sep='_')
original_df.loc[original_df['year'] == 2046, 'sourceID'] = ''
original_df.loc[original_df['year'] == 2046, 'sourceID'] = original_df.loc[original_df['year'] == 2046, 'sourceID'].str.cat(map(str, np.array(range(1, 1 + numStrain)) // 900 + 1))
original_df.loc[original_df['year'] == 2046, 'sourceID'] = original_df.loc[original_df['year'] == 2046, 'sourceID'].str.cat(map(str, np.array(range(1, 1 + numStrain)) // 90 + 1), sep='-')
original_df.loc[original_df['year'] == 2046, 'sourceID'] = original_df.loc[original_df['year'] == 2046, 'sourceID'].str.cat(map(str, np.array(range(numStrain)) // 3 + 1), sep='*#')
original_df.loc[original_df['year'] == 2047, 'sourceID'] = 'OB4-REP'
original_df.loc[original_df['year'] == 2047, 'sourceID'] = original_df.loc[original_df['year'] == 2047, 'sourceID'].str.cat(map(str, np.array(range(1, 1 + numOB)) // 120 + 1))
original_df.loc[original_df['year'] == 2047, 'sourceID'] = original_df.loc[original_df['year'] == 2047, 'sourceID'].str.cat(map(str, np.array(range(1, 1 + numOB)) // 3 + 1), sep='_')
original_df.loc[original_df['year'] == 2048, 'sourceID'] = 'AAP'
original_df.loc[original_df['year'] == 2048, 'sourceID'] = original_df.loc[original_df['year'] == 2048, 'sourceID'].str.cat(map(str, range(1, 1 + numAAP)), sep='_')
original_df.loc[original_df['year'] == 2049, 'sourceID'] = 'SU-Fungi-A'
original_df.loc[original_df['year'] == 2049, 'sourceID'] = original_df.loc[original_df['year'] == 2049, 'sourceID'].str.cat(map(str, range(1, 1 + numSU)))
original_df

,amrID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,species,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,2048,Ruby Johnson,08/10/2053,Rhodes Bioassay/Rhodes MS,01/10/2053,30.2,N/A,AAP_1,suddenly,...,Preston,Ruby Johnson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
1,AMR00002,2042,Ruby Johnson,06/01/2051,Membrane Disruption assay,30/12/2050,26.6,N/A,MOZ42-1,character,...,Smith,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,DMSO
2,AMR00003,2041,Joshua Keener,31/01/2050,Rhodes MS/NMR,24/01/2050,9.3,N/A,TIC2041-1,near,...,Wellington,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
3,AMR00004,2042,Ruby Johnson,16/10/2050,Membrane Disruption assay,09/10/2050,31.7,N/A,MOZ42-2,can,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice1,DMSO
4,AMR00005,2041,Joshua Keener,21/12/2054,Rhodes MS/NMR,14/12/2054,66.1,N/A,TIC2041-2,maintain,...,Wellington,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,AMR99996,2042,Joshua Keener,04/06/2053,"Rhodes Bioassay, PK assay",28/05/2053,31.0,N/A,MOZ42-11029,foreign,...,Roodt,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
99996,AMR99997,2041,Joshua Keener,29/01/2052,Rhodes MS/NMR,22/01/2052,58.4,N/A,TIC2041-11086,arrive,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,Methanol
99997,AMR99998,2048,Ruby Johnson,07/08/2051,Rhodes MS,31/07/2051,54.7,N/A,AAP_11175,carry,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice5,DMSO
99998,AMR99999,2043,Joshua Keener,07/02/2051,Rhodes Bioassay/Rhodes MS,31/01/2051,8.3,N/A,SAF43-11251,stuff,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice1,DMSO


Inserting Source type

In [178]:
# Source type - based on source ID (microbial vs sponge vs stromatolite)
# Examples: TIC2041-001 -- MNP; MOZ42-001 -- MNP; SAF43-001 -- MNP; MIAP_207(B/AIA/ISP) -- Marion Islands; CSF_1ABC_20 -- Stromatolites; 05-008*#5 -- Microbial; OB4-REP3_026 -- Stromatolites; AAP_14 -- ; SU-Fungi-A16 -- Soil
original_df.insert(9, 'sourceType', 'MNP')
original_df

,amrID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,sourceType,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,2048,Ruby Johnson,08/10/2053,Rhodes Bioassay/Rhodes MS,01/10/2053,30.2,N/A,AAP_1,MNP,...,Preston,Ruby Johnson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
1,AMR00002,2042,Ruby Johnson,06/01/2051,Membrane Disruption assay,30/12/2050,26.6,N/A,MOZ42-1,MNP,...,Smith,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,DMSO
2,AMR00003,2041,Joshua Keener,31/01/2050,Rhodes MS/NMR,24/01/2050,9.3,N/A,TIC2041-1,MNP,...,Wellington,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
3,AMR00004,2042,Ruby Johnson,16/10/2050,Membrane Disruption assay,09/10/2050,31.7,N/A,MOZ42-2,MNP,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice1,DMSO
4,AMR00005,2041,Joshua Keener,21/12/2054,Rhodes MS/NMR,14/12/2054,66.1,N/A,TIC2041-2,MNP,...,Wellington,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,AMR99996,2042,Joshua Keener,04/06/2053,"Rhodes Bioassay, PK assay",28/05/2053,31.0,N/A,MOZ42-11029,MNP,...,Roodt,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
99996,AMR99997,2041,Joshua Keener,29/01/2052,Rhodes MS/NMR,22/01/2052,58.4,N/A,TIC2041-11086,MNP,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,Methanol
99997,AMR99998,2048,Ruby Johnson,07/08/2051,Rhodes MS,31/07/2051,54.7,N/A,AAP_11175,MNP,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice5,DMSO
99998,AMR99999,2043,Joshua Keener,07/02/2051,Rhodes Bioassay/Rhodes MS,31/01/2051,8.3,N/A,SAF43-11251,MNP,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice1,DMSO


In [179]:
# Examples: TIC2040-001 -- MNP; AMRU001 -- Microbial; MOZ42-001 -- MNP; SAF43-001 -- MNP; MIAP_207(B/AIA/ISP) -- Marion Islands; CSF_1ABC_20 -- Stromatolites; 05-008*#5 -- Microbial; OB4-REP3_026 -- Stromatolites; AAP_14 -- ; SU-Fungi-A16 -- Soil
original_df.loc[original_df['year'] == 2044, 'sourceType'] = 'Marion Islands'
original_df.loc[original_df['year'] == 2045, 'sourceType'] = 'Stromatolites'
original_df.loc[original_df['year'] == 2046, 'sourceType'] = 'Microbial'
original_df.loc[original_df['year'] == 2047, 'sourceType'] = 'Stromatolites'
original_df.loc[original_df['year'] == 2048, 'sourceType'] = 'Bacterial'
original_df.loc[original_df['year'] == 2049, 'sourceType'] = 'Soil'
original_df

,amrID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,sourceType,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,2048,Ruby Johnson,08/10/2053,Rhodes Bioassay/Rhodes MS,01/10/2053,30.2,N/A,AAP_1,Bacterial,...,Preston,Ruby Johnson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
1,AMR00002,2042,Ruby Johnson,06/01/2051,Membrane Disruption assay,30/12/2050,26.6,N/A,MOZ42-1,MNP,...,Smith,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,DMSO
2,AMR00003,2041,Joshua Keener,31/01/2050,Rhodes MS/NMR,24/01/2050,9.3,N/A,TIC2041-1,MNP,...,Wellington,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
3,AMR00004,2042,Ruby Johnson,16/10/2050,Membrane Disruption assay,09/10/2050,31.7,N/A,MOZ42-2,MNP,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice1,DMSO
4,AMR00005,2041,Joshua Keener,21/12/2054,Rhodes MS/NMR,14/12/2054,66.1,N/A,TIC2041-2,MNP,...,Wellington,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,AMR99996,2042,Joshua Keener,04/06/2053,"Rhodes Bioassay, PK assay",28/05/2053,31.0,N/A,MOZ42-11029,MNP,...,Roodt,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
99996,AMR99997,2041,Joshua Keener,29/01/2052,Rhodes MS/NMR,22/01/2052,58.4,N/A,TIC2041-11086,MNP,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,Methanol
99997,AMR99998,2048,Ruby Johnson,07/08/2051,Rhodes MS,31/07/2051,54.7,N/A,AAP_11175,Bacterial,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice5,DMSO
99998,AMR99999,2043,Joshua Keener,07/02/2051,Rhodes Bioassay/Rhodes MS,31/01/2051,8.3,N/A,SAF43-11251,MNP,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice1,DMSO


Inserting SupplierID

In [180]:
# Supplier ID - based on date of collection, so must be same or earlier year than submittedDate
original_df.insert(1, 'supplierID', original_df['sourceID'].astype(str))
original_df

,amrID,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,AAP_1,2048,Ruby Johnson,08/10/2053,Rhodes Bioassay/Rhodes MS,01/10/2053,30.2,N/A,AAP_1,...,Preston,Ruby Johnson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
1,AMR00002,MOZ42-1,2042,Ruby Johnson,06/01/2051,Membrane Disruption assay,30/12/2050,26.6,N/A,MOZ42-1,...,Smith,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,DMSO
2,AMR00003,TIC2041-1,2041,Joshua Keener,31/01/2050,Rhodes MS/NMR,24/01/2050,9.3,N/A,TIC2041-1,...,Wellington,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
3,AMR00004,MOZ42-2,2042,Ruby Johnson,16/10/2050,Membrane Disruption assay,09/10/2050,31.7,N/A,MOZ42-2,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice1,DMSO
4,AMR00005,TIC2041-2,2041,Joshua Keener,21/12/2054,Rhodes MS/NMR,14/12/2054,66.1,N/A,TIC2041-2,...,Wellington,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,AMR99996,MOZ42-11029,2042,Joshua Keener,04/06/2053,"Rhodes Bioassay, PK assay",28/05/2053,31.0,N/A,MOZ42-11029,...,Roodt,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
99996,AMR99997,TIC2041-11086,2041,Joshua Keener,29/01/2052,Rhodes MS/NMR,22/01/2052,58.4,N/A,TIC2041-11086,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,Methanol
99997,AMR99998,AAP_11175,2048,Ruby Johnson,07/08/2051,Rhodes MS,31/07/2051,54.7,N/A,AAP_11175,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice5,DMSO
99998,AMR99999,SAF43-11251,2043,Joshua Keener,07/02/2051,Rhodes Bioassay/Rhodes MS,31/01/2051,8.3,N/A,SAF43-11251,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice1,DMSO


In [181]:
# Examples: TIC2040-001; MOZ42-001; SAF43-001; MIAP_207(B/AIA/ISP); CSF_1ABC_20; 05-008*#5; OB4-REP3_026; AAP_14; SU-Fungi-A16
# To: TIC2040-001JK; MOZ42-001JK; SAF42-001JK ; MIAP_207(B/AIA/ISP) keep same; AMRU077_A; R1 - R3; AMRU026_A; AAP_14; SU-Fungi-A16

# original_df.loc[original_df['year'] == 2045, 'supplierID'] = original_df.loc[original_df['year'] == 2045, 'supplierID'].str.cat(map(chr, (np.array(range(1, 1 + numCSF)) % 3) + 65))
original_df.loc[original_df['year'] == 2045, 'supplierID'] = 'AMRU'
original_df.loc[original_df['year'] == 2045, 'supplierID'] = original_df.loc[original_df['year'] == 2045, 'supplierID'].str.cat(map(str, ((np.array(range(1, 1 + numCSF)) // 3 + 1) + (np.array(range(1, 1 + numCSF)) // 3))))
original_df.loc[original_df['year'] == 2045, 'supplierID'] = original_df.loc[original_df['year'] == 2045, 'supplierID'].str.cat(map(chr, (np.array(range(1, 1 + numCSF)) % 3) + 65))

original_df.loc[original_df['year'] == 2047, 'supplierID'] = 'AMRU'
original_df.loc[original_df['year'] == 2047, 'supplierID'] = original_df.loc[original_df['year'] == 2047, 'supplierID'].str.cat(map(str, ((np.array(range(1, 1 + numOB)) // 3 + 1) + (np.array(range(1, 1 + numOB)) // 3 + 1))))
original_df.loc[original_df['year'] == 2047, 'supplierID'] = original_df.loc[original_df['year'] == 2047, 'supplierID'].str.cat(map(chr, (np.array(range(1, 1 + numOB)) % 3) + 65))

original_df.loc[original_df['year'] == 2046, 'supplierID'] = original_df.loc[original_df['year'] == 2046, 'supplierID'] + 'R'
original_df.loc[original_df['year'] == 2046, 'supplierID'] = original_df.loc[original_df['year'] == 2046, 'supplierID'].str.cat(map(str, (np.array(range(numStrain)) % 3 + 1)))

original_df

,amrID,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,...,researchGroup,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent
0,AMR00001,AAP_1,2048,Ruby Johnson,08/10/2053,Rhodes Bioassay/Rhodes MS,01/10/2053,30.2,N/A,AAP_1,...,Preston,Ruby Johnson,N/A,N/A,N/A,N/A,N,N,Y,DMSO
1,AMR00002,MOZ42-1,2042,Ruby Johnson,06/01/2051,Membrane Disruption assay,30/12/2050,26.6,N/A,MOZ42-1,...,Smith,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,DMSO
2,AMR00003,TIC2041-1,2041,Joshua Keener,31/01/2050,Rhodes MS/NMR,24/01/2050,9.3,N/A,TIC2041-1,...,Wellington,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO
3,AMR00004,MOZ42-2,2042,Ruby Johnson,16/10/2050,Membrane Disruption assay,09/10/2050,31.7,N/A,MOZ42-2,...,Roodt,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice1,DMSO
4,AMR00005,TIC2041-2,2041,Joshua Keener,21/12/2054,Rhodes MS/NMR,14/12/2054,66.1,N/A,TIC2041-2,...,Wellington,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,DMSO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,AMR99996,MOZ42-11029,2042,Joshua Keener,04/06/2053,"Rhodes Bioassay, PK assay",28/05/2053,31.0,N/A,MOZ42-11029,...,Roodt,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol
99996,AMR99997,TIC2041-11086,2041,Joshua Keener,29/01/2052,Rhodes MS/NMR,22/01/2052,58.4,N/A,TIC2041-11086,...,Smith,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,Methanol
99997,AMR99998,AAP_11175,2048,Ruby Johnson,07/08/2051,Rhodes MS,31/07/2051,54.7,N/A,AAP_11175,...,Wellington,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice5,DMSO
99998,AMR99999,SAF43-11251,2043,Joshua Keener,07/02/2051,Rhodes Bioassay/Rhodes MS,31/01/2051,8.3,N/A,SAF43-11251,...,Roodt,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice1,DMSO


In [182]:
# Making column with initials of submitter
nameSeries = original_df['submittedBy']
initialList = []
for index, value in nameSeries.items():
    initial = ""
    allNames = value.split(" ")
    for name in allNames:
        initial = initial+name[0].upper()
    # print(initial)
    initialList.append(initial)

initialSeries = pd.Series(initialList)
print(initialSeries)


0        RJ
1        RJ
2        RJ
3        JR
4        JR
         ..
99995    RJ
99996    JR
99997    RH
99998    RH
99999    RJ
Length: 100000, dtype: object


In [183]:
# Add initials column and initials to correct supplier ID types
original_df.insert(26, 'submitterInitials', initialSeries)
original_df.loc[original_df['year'] == 2041, 'supplierID'] = original_df.loc[original_df['year'] == 2041, 'supplierID'].astype(str) + original_df.loc[original_df['year'] == 2041, 'submitterInitials'].astype(str)
original_df.loc[original_df['year'] == 2042, 'supplierID'] = original_df.loc[original_df['year'] == 2042, 'supplierID'].astype(str) + original_df.loc[original_df['year'] == 2042, 'submitterInitials'].astype(str)
original_df.loc[original_df['year'] == 2043, 'supplierID'] = original_df.loc[original_df['year'] == 2043, 'supplierID'].astype(str) + original_df.loc[original_df['year'] == 2043, 'submitterInitials'].astype(str)
original_df

,amrID,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,...,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent,submitterInitials
0,AMR00001,AAP_1,2048,Ruby Johnson,08/10/2053,Rhodes Bioassay/Rhodes MS,01/10/2053,30.2,N/A,AAP_1,...,Ruby Johnson,N/A,N/A,N/A,N/A,N,N,Y,DMSO,RJ
1,AMR00002,MOZ42-1RJ,2042,Ruby Johnson,06/01/2051,Membrane Disruption assay,30/12/2050,26.6,N/A,MOZ42-1,...,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,DMSO,RJ
2,AMR00003,TIC2041-1RJ,2041,Joshua Keener,31/01/2050,Rhodes MS/NMR,24/01/2050,9.3,N/A,TIC2041-1,...,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO,RJ
3,AMR00004,MOZ42-2JR,2042,Ruby Johnson,16/10/2050,Membrane Disruption assay,09/10/2050,31.7,N/A,MOZ42-2,...,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice1,DMSO,JR
4,AMR00005,TIC2041-2JR,2041,Joshua Keener,21/12/2054,Rhodes MS/NMR,14/12/2054,66.1,N/A,TIC2041-2,...,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,DMSO,JR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,AMR99996,MOZ42-11029RJ,2042,Joshua Keener,04/06/2053,"Rhodes Bioassay, PK assay",28/05/2053,31.0,N/A,MOZ42-11029,...,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol,RJ
99996,AMR99997,TIC2041-11086JR,2041,Joshua Keener,29/01/2052,Rhodes MS/NMR,22/01/2052,58.4,N/A,TIC2041-11086,...,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,Methanol,JR
99997,AMR99998,AAP_11175,2048,Ruby Johnson,07/08/2051,Rhodes MS,31/07/2051,54.7,N/A,AAP_11175,...,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice5,DMSO,RH
99998,AMR99999,SAF43-11251RH,2043,Joshua Keener,07/02/2051,Rhodes Bioassay/Rhodes MS,31/01/2051,8.3,N/A,SAF43-11251,...,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice1,DMSO,RH


Fractions: MIAP, SU-Fungi and AAP records are never fractionated. Therefore, we must remove the records with years 2044, 2048 and 2049

In [184]:
subset_df = original_df.copy(deep=True)
indexFraction = subset_df[(subset_df['year'] == 2044) | (subset_df['year'] == 2048) | (subset_df['year'] == 2049)].index
subset_df.drop(indexFraction, inplace=True)
subset_df.reset_index(drop=True, inplace=True)
subset_df

,amrID,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,...,submittedBy,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent,submitterInitials
0,AMR00002,MOZ42-1RJ,2042,Ruby Johnson,06/01/2051,Membrane Disruption assay,30/12/2050,26.6,N/A,MOZ42-1,...,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,DMSO,RJ
1,AMR00003,TIC2041-1RJ,2041,Joshua Keener,31/01/2050,Rhodes MS/NMR,24/01/2050,9.3,N/A,TIC2041-1,...,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO,RJ
2,AMR00004,MOZ42-2JR,2042,Ruby Johnson,16/10/2050,Membrane Disruption assay,09/10/2050,31.7,N/A,MOZ42-2,...,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice1,DMSO,JR
3,AMR00005,TIC2041-2JR,2041,Joshua Keener,21/12/2054,Rhodes MS/NMR,14/12/2054,66.1,N/A,TIC2041-2,...,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,DMSO,JR
4,AMR00006,MOZ42-3JR,2042,Joshua Keener,26/04/2050,Rhodes Bioassay/Rhodes MS,19/04/2050,58.8,N/A,MOZ42-3,...,Julia Reed,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice1,DMSO,JR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66647,AMR99994,AMRU7489B,2045,Joshua Keener,04/01/2055,Rhodes Bioassay,28/12/2054,31.5,N/A,CSF_1ABC_3745,...,Julia Reed,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,DMSO,JR
66648,AMR99995,MOZ42-11028RH,2042,Ruby Johnson,28/12/2051,Membrane Disruption assay,21/12/2051,1.2,N/A,MOZ42-11028,...,Robert Hobson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice4,Methanol,RH
66649,AMR99996,MOZ42-11029RJ,2042,Joshua Keener,04/06/2053,"Rhodes Bioassay, PK assay",28/05/2053,31.0,N/A,MOZ42-11029,...,Ruby Johnson,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,Methanol,RJ
66650,AMR99997,TIC2041-11086JR,2041,Joshua Keener,29/01/2052,Rhodes MS/NMR,22/01/2052,58.4,N/A,TIC2041-11086,...,Julia Reed,N/A,N/A,N/A,N/A,Y,N,N,Methanol,JR


MNP Spreadsheet

MNP ID -> process

Supplier ID -> fraction + process

Assigned by -> process

Date assigned -> process

Destination -> process

Date submitted -> fraction

Amount available -> fraction

Sample type -> fraction

Concentration -> fraction

Solvent -> fraction

Project -> fraction

Source of sample -> fraction (from extract/fraction table)

Species -> removed

Permit no -> removed

Research group -> fraction

Submitted by -> fraction

Comments -> removed

Level (1st, 2nd, 3rd level etc) -> fraction

In [185]:
print(list(subset_df.columns.values))

['amrID', 'supplierID', 'year', 'assignedBy', 'assignedDate', 'destinationOfSample', 'submittedDate', 'amountAvailable', 'synthesisPotential', 'sourceID', 'sourceType', 'species', 'permitNo', 'pure', 'ms', 'nmr', 'researchGroup', 'submittedBy', 'solubility', 'stereoComments', 'smileStructure', 'mw', 'additionalMetadata', 'existingPatent', 'existingLiterature', 'solvent', 'submitterInitials']


In [186]:
# Need to remove columns that only extracts had/ must be replaced
# List: AMR ID, Source Id replaced with Supplier ID, source type, Destination, supplied date, synthesis potential, pure, MS, NMR, researchGroup
# solubility, stereoComments, SMILE, mw, additional metadata, existing patent, existing literature
# Will add 21 days to assigned date to get supplied date, and then add another week to that to get assigned date
subset_df = subset_df[["supplierID", "year", "assignedDate", "species", "permitNo"]]
subset_df

,supplierID,year,assignedDate,species,permitNo
0,MOZ42-1RJ,2042,06/01/2051,character,RES2042
1,TIC2041-1RJ,2041,31/01/2050,near,RES2041
2,MOZ42-2JR,2042,16/10/2050,can,RES2042
3,TIC2041-2JR,2041,21/12/2054,maintain,RES2041
4,MOZ42-3JR,2042,26/04/2050,know,RES2042
...,...,...,...,...,...
66647,AMRU7489B,2045,04/01/2055,medical,RES2045
66648,MOZ42-11028RH,2042,28/12/2051,minute,RES2042
66649,MOZ42-11029RJ,2042,04/06/2053,foreign,RES2042
66650,TIC2041-11086JR,2041,29/01/2052,arrive,RES2041


In [187]:
# Date assigned - do a week after submittedDate
subset_df.assignedDate = pd.to_datetime(subset_df.assignedDate, format='%d/%m/%Y')
subset_df.insert(3, 'submittedDate', subset_df['assignedDate'])
subset_df

C:\Users\Emily\AppData\Local\Temp\ipykernel_4880\1737996625.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df.assignedDate = pd.to_datetime(subset_df.assignedDate, format='%d/%m/%Y')


,supplierID,year,assignedDate,submittedDate,species,permitNo
0,MOZ42-1RJ,2042,2051-01-06,2051-01-06,character,RES2042
1,TIC2041-1RJ,2041,2050-01-31,2050-01-31,near,RES2041
2,MOZ42-2JR,2042,2050-10-16,2050-10-16,can,RES2042
3,TIC2041-2JR,2041,2054-12-21,2054-12-21,maintain,RES2041
4,MOZ42-3JR,2042,2050-04-26,2050-04-26,know,RES2042
...,...,...,...,...,...,...
66647,AMRU7489B,2045,2055-01-04,2055-01-04,medical,RES2045
66648,MOZ42-11028RH,2042,2051-12-28,2051-12-28,minute,RES2042
66649,MOZ42-11029RJ,2042,2053-06-04,2053-06-04,foreign,RES2042
66650,TIC2041-11086JR,2041,2052-01-29,2052-01-29,arrive,RES2041


In [188]:
subset_df['submittedDate'] = subset_df['submittedDate'] + timedelta(days=21)
subset_df['submittedDate'] = subset_df['submittedDate'].dt.strftime('%d/%m/%Y')
subset_df

C:\Users\Emily\AppData\Local\Temp\ipykernel_4880\1821189252.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['submittedDate'] = subset_df['submittedDate'] + timedelta(days=21)
C:\Users\Emily\AppData\Local\Temp\ipykernel_4880\1821189252.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['submittedDate'] = subset_df['submittedDate'].dt.strftime('%d/%m/%Y')


,supplierID,year,assignedDate,submittedDate,species,permitNo
0,MOZ42-1RJ,2042,2051-01-06,27/01/2051,character,RES2042
1,TIC2041-1RJ,2041,2050-01-31,21/02/2050,near,RES2041
2,MOZ42-2JR,2042,2050-10-16,06/11/2050,can,RES2042
3,TIC2041-2JR,2041,2054-12-21,11/01/2055,maintain,RES2041
4,MOZ42-3JR,2042,2050-04-26,17/05/2050,know,RES2042
...,...,...,...,...,...,...
66647,AMRU7489B,2045,2055-01-04,25/01/2055,medical,RES2045
66648,MOZ42-11028RH,2042,2051-12-28,18/01/2052,minute,RES2042
66649,MOZ42-11029RJ,2042,2053-06-04,25/06/2053,foreign,RES2042
66650,TIC2041-11086JR,2041,2052-01-29,19/02/2052,arrive,RES2041


In [189]:
# SubmittedDate is three weeks after extract assignedDate

# AssignedDate is one week after submittedDate
subset_df['assignedDate'] = subset_df['assignedDate'] + timedelta(days=28)
subset_df['assignedDate'] = subset_df['assignedDate'].dt.strftime('%d/%m/%Y')
subset_df

C:\Users\Emily\AppData\Local\Temp\ipykernel_4880\2221413924.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['assignedDate'] = subset_df['assignedDate'] + timedelta(days=28)
C:\Users\Emily\AppData\Local\Temp\ipykernel_4880\2221413924.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df['assignedDate'] = subset_df['assignedDate'].dt.strftime('%d/%m/%Y')


,supplierID,year,assignedDate,submittedDate,species,permitNo
0,MOZ42-1RJ,2042,03/02/2051,27/01/2051,character,RES2042
1,TIC2041-1RJ,2041,28/02/2050,21/02/2050,near,RES2041
2,MOZ42-2JR,2042,13/11/2050,06/11/2050,can,RES2042
3,TIC2041-2JR,2041,18/01/2055,11/01/2055,maintain,RES2041
4,MOZ42-3JR,2042,24/05/2050,17/05/2050,know,RES2042
...,...,...,...,...,...,...
66647,AMRU7489B,2045,01/02/2055,25/01/2055,medical,RES2045
66648,MOZ42-11028RH,2042,25/01/2052,18/01/2052,minute,RES2042
66649,MOZ42-11029RJ,2042,02/07/2053,25/06/2053,foreign,RES2042
66650,TIC2041-11086JR,2041,26/02/2052,19/02/2052,arrive,RES2041


In [190]:
fake = Faker()
Faker.seed(1234)
random.seed(1234)

class Provider(BaseProvider):
    submitters = ['Ruby Johnson', 'Amanda Bowker', 'Priscilla Evans']
    assigners = ['Ruby Johnson', 'Joshua Keener']
    destinations = ['RU Bioassay', 'Rhodes MS', 'CD Experiment', 'RU LC-MS & 1H-NMR', 'RU LC/MS']
    binaryChoice = ['Y', 'N']
    researchgroups = ['Smith', 'Roodt']
    solvents = ['MeOH', 'DCM:MeOH 2:1', 'DCM/MeOH']
    sampleTypes = ['Fraction', 'SPE fraction', 'Fraction from flash', 'HPLC Fraction']
    concentrations = [0.25, 0.1, 0.5, 1.0, 20.0, 50.0]
    comment = [None, None, 'Used compound X in fractionation', 'Used compound Y in fractionation', None, None, '50:50 setting', '20:20 setting']
    
    def submittedBy(self):
        return random.choice(self.submitters)

    def assignedBy(self):
        return random.choice(self.assigners)

    def destinationOfSample(self):
        return random.choice(self.destinations)

    def researchGroup(self):
        return random.choice(self.researchgroups)
    
    def solvent(self):
        return random.choice(self.solvents)
    
    def sampleType(self):
        return random.choice(self.sampleTypes)
    
    def concentration(self):
        return random.choice(self.concentrations)
    
    def comments(self):
        return random.choice(self.comment)

# Add the Provider to our faker object
fake.add_provider(Provider)

# First change any columns that must be the same for all fractions from a particular source
# Submitted by, research group, project, concentration, solvent, species, sample type, assigned by, submitted date and assigned date (but will be same either way), destination

def create_MNR_data(fake, no_of_rows):
    columns = ['assignedBy', 'destinationOfSample', 'sampleType', 'researchGroup', 'submittedBy', 'solvent', 'concentration', 'comments']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

fractionSubset_df = create_MNR_data(fake, len(subset_df))
fractionSubset_df


,assignedBy,destinationOfSample,sampleType,researchGroup,submittedBy,solvent,concentration,comments
0,Joshua Keener,RU LC-MS & 1H-NMR,SPE fraction,Smith,Ruby Johnson,DCM/MeOH,0.50,20:20 setting
1,Ruby Johnson,RU Bioassay,Fraction,Smith,Priscilla Evans,DCM:MeOH 2:1,20.00,Used compound Y in fractionation
2,Ruby Johnson,RU LC/MS,SPE fraction,Smith,Ruby Johnson,DCM:MeOH 2:1,50.00,Used compound Y in fractionation
3,Ruby Johnson,RU LC-MS & 1H-NMR,SPE fraction,Roodt,Ruby Johnson,MeOH,0.25,Used compound X in fractionation
4,Ruby Johnson,RU LC/MS,SPE fraction,Smith,Priscilla Evans,DCM:MeOH 2:1,0.10,Used compound Y in fractionation
...,...,...,...,...,...,...,...,...
66647,Joshua Keener,RU Bioassay,SPE fraction,Smith,Priscilla Evans,MeOH,0.10,None
66648,Joshua Keener,CD Experiment,Fraction from flash,Smith,Amanda Bowker,DCM:MeOH 2:1,50.00,None
66649,Ruby Johnson,RU LC-MS & 1H-NMR,SPE fraction,Roodt,Priscilla Evans,DCM:MeOH 2:1,0.50,Used compound Y in fractionation
66650,Joshua Keener,Rhodes MS,Fraction from flash,Smith,Priscilla Evans,DCM:MeOH 2:1,1.00,Used compound Y in fractionation


In [191]:
# Join subsets together to get almost full MNP table
newFraction_df = pd.concat([subset_df, fractionSubset_df], axis=1)
newFraction_df.rename(columns={'supplierID':'sourceID'}, inplace=True)
# Get columns in same order as MNP spreadsheet
orderCols = ['year', 'assignedBy', 'assignedDate', 'destinationOfSample', 'submittedDate', 'sampleType', 'concentration', 'solvent', 'sourceID', 'species', 'permitNo', 'researchGroup', 'submittedBy', 'comments']
newFraction_df = newFraction_df[orderCols]
newFraction_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2042,Joshua Keener,03/02/2051,RU LC-MS & 1H-NMR,27/01/2051,SPE fraction,0.50,DCM/MeOH,MOZ42-1RJ,character,RES2042,Smith,Ruby Johnson,20:20 setting
1,2041,Ruby Johnson,28/02/2050,RU Bioassay,21/02/2050,Fraction,20.00,DCM:MeOH 2:1,TIC2041-1RJ,near,RES2041,Smith,Priscilla Evans,Used compound Y in fractionation
2,2042,Ruby Johnson,13/11/2050,RU LC/MS,06/11/2050,SPE fraction,50.00,DCM:MeOH 2:1,MOZ42-2JR,can,RES2042,Smith,Ruby Johnson,Used compound Y in fractionation
3,2041,Ruby Johnson,18/01/2055,RU LC-MS & 1H-NMR,11/01/2055,SPE fraction,0.25,MeOH,TIC2041-2JR,maintain,RES2041,Roodt,Ruby Johnson,Used compound X in fractionation
4,2042,Ruby Johnson,24/05/2050,RU LC/MS,17/05/2050,SPE fraction,0.10,DCM:MeOH 2:1,MOZ42-3JR,know,RES2042,Smith,Priscilla Evans,Used compound Y in fractionation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66647,2045,Joshua Keener,01/02/2055,RU Bioassay,25/01/2055,SPE fraction,0.10,MeOH,AMRU7489B,medical,RES2045,Smith,Priscilla Evans,None
66648,2042,Joshua Keener,25/01/2052,CD Experiment,18/01/2052,Fraction from flash,50.00,DCM:MeOH 2:1,MOZ42-11028RH,minute,RES2042,Smith,Amanda Bowker,None
66649,2042,Ruby Johnson,02/07/2053,RU LC-MS & 1H-NMR,25/06/2053,SPE fraction,0.50,DCM:MeOH 2:1,MOZ42-11029RJ,foreign,RES2042,Roodt,Priscilla Evans,Used compound Y in fractionation
66650,2041,Joshua Keener,26/02/2052,Rhodes MS,19/02/2052,Fraction from flash,1.00,DCM:MeOH 2:1,TIC2041-11086JR,arrive,RES2041,Smith,Priscilla Evans,Used compound Y in fractionation


In [192]:
# Get 20000 records from subset that is fractionated
numSampled1 = 20000
fraction1_df = newFraction_df.sample(n = numSampled1, random_state=1231, ignore_index=True)
fraction2_df = newFraction_df.sample(n = numSampled1, random_state=1232, ignore_index=True)
fraction3_df = newFraction_df.sample(n = numSampled1, random_state=1233, ignore_index=True)
fraction4_df = newFraction_df.sample(n = numSampled1, random_state=1234, ignore_index=True)
fraction1_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2042,Joshua Keener,08/11/2054,RU Bioassay,01/11/2054,HPLC Fraction,50.00,DCM/MeOH,MOZ42-9322RH,six,RES2042,Roodt,Priscilla Evans,Used compound Y in fractionation
1,2042,Joshua Keener,28/11/2052,CD Experiment,21/11/2052,Fraction,0.10,DCM:MeOH 2:1,MOZ42-4072RH,decide,RES2042,Roodt,Amanda Bowker,None
2,2043,Joshua Keener,13/11/2054,RU LC/MS,06/11/2054,HPLC Fraction,1.00,DCM:MeOH 2:1,SAF43-11100RH,economy,RES2043,Smith,Ruby Johnson,None
3,2046,Ruby Johnson,25/11/2052,CD Experiment,18/11/2052,SPE fraction,0.10,MeOH,6-56*#1671R3,statement,RES2046,Smith,Priscilla Evans,Used compound X in fractionation
4,2047,Joshua Keener,27/11/2050,CD Experiment,20/11/2050,HPLC Fraction,0.50,DCM:MeOH 2:1,AMRU542C,common,RES2047,Smith,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2043,Joshua Keener,01/09/2054,RU LC/MS,25/08/2054,Fraction,50.00,DCM/MeOH,SAF43-1206RH,plan,RES2043,Smith,Priscilla Evans,None
19996,2041,Ruby Johnson,13/08/2050,RU Bioassay,06/08/2050,SPE fraction,0.25,DCM/MeOH,TIC2041-8540JR,sister,RES2041,Roodt,Amanda Bowker,None
19997,2042,Joshua Keener,08/01/2052,CD Experiment,01/01/2052,HPLC Fraction,50.00,MeOH,MOZ42-9664JR,recent,RES2042,Smith,Priscilla Evans,Used compound X in fractionation
19998,2041,Joshua Keener,31/05/2051,Rhodes MS,24/05/2051,Fraction,0.50,MeOH,TIC2041-3049RJ,cover,RES2041,Roodt,Amanda Bowker,None


In [193]:
fractionFirst_df = pd.concat([fraction1_df, fraction2_df, fraction3_df, fraction4_df], ignore_index=True)
fractionFirst_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2042,Joshua Keener,08/11/2054,RU Bioassay,01/11/2054,HPLC Fraction,50.00,DCM/MeOH,MOZ42-9322RH,six,RES2042,Roodt,Priscilla Evans,Used compound Y in fractionation
1,2042,Joshua Keener,28/11/2052,CD Experiment,21/11/2052,Fraction,0.10,DCM:MeOH 2:1,MOZ42-4072RH,decide,RES2042,Roodt,Amanda Bowker,None
2,2043,Joshua Keener,13/11/2054,RU LC/MS,06/11/2054,HPLC Fraction,1.00,DCM:MeOH 2:1,SAF43-11100RH,economy,RES2043,Smith,Ruby Johnson,None
3,2046,Ruby Johnson,25/11/2052,CD Experiment,18/11/2052,SPE fraction,0.10,MeOH,6-56*#1671R3,statement,RES2046,Smith,Priscilla Evans,Used compound X in fractionation
4,2047,Joshua Keener,27/11/2050,CD Experiment,20/11/2050,HPLC Fraction,0.50,DCM:MeOH 2:1,AMRU542C,common,RES2047,Smith,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,2041,Ruby Johnson,04/02/2053,CD Experiment,28/01/2053,Fraction from flash,0.10,DCM:MeOH 2:1,TIC2041-1762RH,method,RES2041,Smith,Ruby Johnson,20:20 setting
79996,2046,Ruby Johnson,05/06/2054,Rhodes MS,29/05/2054,Fraction,0.10,MeOH,7-67*#1989R2,success,RES2046,Smith,Priscilla Evans,None
79997,2045,Joshua Keener,16/08/2054,Rhodes MS,09/08/2054,HPLC Fraction,0.25,DCM:MeOH 2:1,AMRU1191B,suffer,RES2045,Roodt,Priscilla Evans,Used compound Y in fractionation
79998,2043,Ruby Johnson,27/10/2054,CD Experiment,20/10/2054,Fraction from flash,0.50,DCM:MeOH 2:1,SAF43-1599RH,various,RES2043,Smith,Ruby Johnson,None


In [194]:
# Project - microbial for AMNRU, MNP/Marine for rest
fractionFirst_df.insert(8, 'project', 'Marine')
fractionFirst_df.loc[fractionFirst_df['year'] == 2045, 'project'] = 'Microbial'
fractionFirst_df.loc[fractionFirst_df['year'] == 2047, 'project'] = 'Microbial'
fractionFirst_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2042,Joshua Keener,08/11/2054,RU Bioassay,01/11/2054,HPLC Fraction,50.00,DCM/MeOH,Marine,MOZ42-9322RH,six,RES2042,Roodt,Priscilla Evans,Used compound Y in fractionation
1,2042,Joshua Keener,28/11/2052,CD Experiment,21/11/2052,Fraction,0.10,DCM:MeOH 2:1,Marine,MOZ42-4072RH,decide,RES2042,Roodt,Amanda Bowker,None
2,2043,Joshua Keener,13/11/2054,RU LC/MS,06/11/2054,HPLC Fraction,1.00,DCM:MeOH 2:1,Marine,SAF43-11100RH,economy,RES2043,Smith,Ruby Johnson,None
3,2046,Ruby Johnson,25/11/2052,CD Experiment,18/11/2052,SPE fraction,0.10,MeOH,Marine,6-56*#1671R3,statement,RES2046,Smith,Priscilla Evans,Used compound X in fractionation
4,2047,Joshua Keener,27/11/2050,CD Experiment,20/11/2050,HPLC Fraction,0.50,DCM:MeOH 2:1,Microbial,AMRU542C,common,RES2047,Smith,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,2041,Ruby Johnson,04/02/2053,CD Experiment,28/01/2053,Fraction from flash,0.10,DCM:MeOH 2:1,Marine,TIC2041-1762RH,method,RES2041,Smith,Ruby Johnson,20:20 setting
79996,2046,Ruby Johnson,05/06/2054,Rhodes MS,29/05/2054,Fraction,0.10,MeOH,Marine,7-67*#1989R2,success,RES2046,Smith,Priscilla Evans,None
79997,2045,Joshua Keener,16/08/2054,Rhodes MS,09/08/2054,HPLC Fraction,0.25,DCM:MeOH 2:1,Microbial,AMRU1191B,suffer,RES2045,Roodt,Priscilla Evans,Used compound Y in fractionation
79998,2043,Ruby Johnson,27/10/2054,CD Experiment,20/10/2054,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,SAF43-1599RH,various,RES2043,Smith,Ruby Johnson,None


In [195]:
# Parts that are different per fraction:
# Amount available
fake = Faker()
Faker.seed(1234)
random.seed(1234)

class Provider(BaseProvider):
    def amountAvailable(self):
        return (round (random.uniform(0.0, 1.0)*100, 1))
    
fake.add_provider(Provider)

def create_amount_data(fake, no_of_rows):
    columns = ['amountAvailable']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

amount_df = create_amount_data(fake, len(fractionFirst_df))

fractionFirst_df = pd.concat([amount_df, fractionFirst_df], axis=1)
fractionFirst_df

,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,96.6,2042,Joshua Keener,08/11/2054,RU Bioassay,01/11/2054,HPLC Fraction,50.00,DCM/MeOH,Marine,MOZ42-9322RH,six,RES2042,Roodt,Priscilla Evans,Used compound Y in fractionation
1,44.1,2042,Joshua Keener,28/11/2052,CD Experiment,21/11/2052,Fraction,0.10,DCM:MeOH 2:1,Marine,MOZ42-4072RH,decide,RES2042,Roodt,Amanda Bowker,None
2,0.7,2043,Joshua Keener,13/11/2054,RU LC/MS,06/11/2054,HPLC Fraction,1.00,DCM:MeOH 2:1,Marine,SAF43-11100RH,economy,RES2043,Smith,Ruby Johnson,None
3,91.1,2046,Ruby Johnson,25/11/2052,CD Experiment,18/11/2052,SPE fraction,0.10,MeOH,Marine,6-56*#1671R3,statement,RES2046,Smith,Priscilla Evans,Used compound X in fractionation
4,93.9,2047,Joshua Keener,27/11/2050,CD Experiment,20/11/2050,HPLC Fraction,0.50,DCM:MeOH 2:1,Microbial,AMRU542C,common,RES2047,Smith,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,45.0,2041,Ruby Johnson,04/02/2053,CD Experiment,28/01/2053,Fraction from flash,0.10,DCM:MeOH 2:1,Marine,TIC2041-1762RH,method,RES2041,Smith,Ruby Johnson,20:20 setting
79996,38.3,2046,Ruby Johnson,05/06/2054,Rhodes MS,29/05/2054,Fraction,0.10,MeOH,Marine,7-67*#1989R2,success,RES2046,Smith,Priscilla Evans,None
79997,41.4,2045,Joshua Keener,16/08/2054,Rhodes MS,09/08/2054,HPLC Fraction,0.25,DCM:MeOH 2:1,Microbial,AMRU1191B,suffer,RES2045,Roodt,Priscilla Evans,Used compound Y in fractionation
79998,79.5,2043,Ruby Johnson,27/10/2054,CD Experiment,20/10/2054,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,SAF43-1599RH,various,RES2043,Smith,Ruby Johnson,None


In [196]:
# Add MNP ID
fractionFirst_df.insert(0, 'mnpID', range(1, 1 + len(fractionFirst_df)))
fractionFirst_df['mnpID'] = 'MNP' + fractionFirst_df['mnpID'].astype(str).str.zfill(5)
fractionFirst_df

,mnpID,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,MNP00001,96.6,2042,Joshua Keener,08/11/2054,RU Bioassay,01/11/2054,HPLC Fraction,50.00,DCM/MeOH,Marine,MOZ42-9322RH,six,RES2042,Roodt,Priscilla Evans,Used compound Y in fractionation
1,MNP00002,44.1,2042,Joshua Keener,28/11/2052,CD Experiment,21/11/2052,Fraction,0.10,DCM:MeOH 2:1,Marine,MOZ42-4072RH,decide,RES2042,Roodt,Amanda Bowker,None
2,MNP00003,0.7,2043,Joshua Keener,13/11/2054,RU LC/MS,06/11/2054,HPLC Fraction,1.00,DCM:MeOH 2:1,Marine,SAF43-11100RH,economy,RES2043,Smith,Ruby Johnson,None
3,MNP00004,91.1,2046,Ruby Johnson,25/11/2052,CD Experiment,18/11/2052,SPE fraction,0.10,MeOH,Marine,6-56*#1671R3,statement,RES2046,Smith,Priscilla Evans,Used compound X in fractionation
4,MNP00005,93.9,2047,Joshua Keener,27/11/2050,CD Experiment,20/11/2050,HPLC Fraction,0.50,DCM:MeOH 2:1,Microbial,AMRU542C,common,RES2047,Smith,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,MNP79996,45.0,2041,Ruby Johnson,04/02/2053,CD Experiment,28/01/2053,Fraction from flash,0.10,DCM:MeOH 2:1,Marine,TIC2041-1762RH,method,RES2041,Smith,Ruby Johnson,20:20 setting
79996,MNP79997,38.3,2046,Ruby Johnson,05/06/2054,Rhodes MS,29/05/2054,Fraction,0.10,MeOH,Marine,7-67*#1989R2,success,RES2046,Smith,Priscilla Evans,None
79997,MNP79998,41.4,2045,Joshua Keener,16/08/2054,Rhodes MS,09/08/2054,HPLC Fraction,0.25,DCM:MeOH 2:1,Microbial,AMRU1191B,suffer,RES2045,Roodt,Priscilla Evans,Used compound Y in fractionation
79998,MNP79999,79.5,2043,Ruby Johnson,27/10/2054,CD Experiment,20/10/2054,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,SAF43-1599RH,various,RES2043,Smith,Ruby Johnson,None


In [197]:
# Make SupplierIDs - same as sourceID, except have -01, -02 etc for initial fractions (will be a,b,c for rest of subfractions)
# 
sourceSeries = fractionFirst_df['sourceID']
listSupplier = []
supplier_freq = {}

for index, value in sourceSeries.items():
    if (value in supplier_freq):
        supplier_freq[value] += 1
    else:
        supplier_freq[value] = 1
    listSupplier.append(value + "-" + str(supplier_freq[value]).zfill(2))

supplierSeries = pd.Series(listSupplier)
#print(supplierSeries)

fractionFirst_df.insert(1, 'supplierID', supplierSeries)
# Include level (currently all 1)
fractionFirst_df.insert(17, 'level', 1)
fractionFirst_df

,mnpID,supplierID,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,level,comments
0,MNP00001,MOZ42-9322RH-01,96.6,2042,Joshua Keener,08/11/2054,RU Bioassay,01/11/2054,HPLC Fraction,50.00,DCM/MeOH,Marine,MOZ42-9322RH,six,RES2042,Roodt,Priscilla Evans,1,Used compound Y in fractionation
1,MNP00002,MOZ42-4072RH-01,44.1,2042,Joshua Keener,28/11/2052,CD Experiment,21/11/2052,Fraction,0.10,DCM:MeOH 2:1,Marine,MOZ42-4072RH,decide,RES2042,Roodt,Amanda Bowker,1,None
2,MNP00003,SAF43-11100RH-01,0.7,2043,Joshua Keener,13/11/2054,RU LC/MS,06/11/2054,HPLC Fraction,1.00,DCM:MeOH 2:1,Marine,SAF43-11100RH,economy,RES2043,Smith,Ruby Johnson,1,None
3,MNP00004,6-56*#1671R3-01,91.1,2046,Ruby Johnson,25/11/2052,CD Experiment,18/11/2052,SPE fraction,0.10,MeOH,Marine,6-56*#1671R3,statement,RES2046,Smith,Priscilla Evans,1,Used compound X in fractionation
4,MNP00005,AMRU542C-01,93.9,2047,Joshua Keener,27/11/2050,CD Experiment,20/11/2050,HPLC Fraction,0.50,DCM:MeOH 2:1,Microbial,AMRU542C,common,RES2047,Smith,Priscilla Evans,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,MNP79996,TIC2041-1762RH-03,45.0,2041,Ruby Johnson,04/02/2053,CD Experiment,28/01/2053,Fraction from flash,0.10,DCM:MeOH 2:1,Marine,TIC2041-1762RH,method,RES2041,Smith,Ruby Johnson,1,20:20 setting
79996,MNP79997,7-67*#1989R2-02,38.3,2046,Ruby Johnson,05/06/2054,Rhodes MS,29/05/2054,Fraction,0.10,MeOH,Marine,7-67*#1989R2,success,RES2046,Smith,Priscilla Evans,1,None
79997,MNP79998,AMRU1191B-03,41.4,2045,Joshua Keener,16/08/2054,Rhodes MS,09/08/2054,HPLC Fraction,0.25,DCM:MeOH 2:1,Microbial,AMRU1191B,suffer,RES2045,Roodt,Priscilla Evans,1,Used compound Y in fractionation
79998,MNP79999,SAF43-1599RH-02,79.5,2043,Ruby Johnson,27/10/2054,CD Experiment,20/10/2054,Fraction from flash,0.50,DCM:MeOH 2:1,Marine,SAF43-1599RH,various,RES2043,Smith,Ruby Johnson,1,None


Subfractions

In [198]:
subfraction_df = fractionFirst_df.copy(deep=True)
subfraction_df = subfraction_df[["supplierID", "year", "assignedDate", "species", "permitNo", "project"]]
subfraction_df

,supplierID,year,assignedDate,species,permitNo,project
0,MOZ42-9322RH-01,2042,08/11/2054,six,RES2042,Marine
1,MOZ42-4072RH-01,2042,28/11/2052,decide,RES2042,Marine
2,SAF43-11100RH-01,2043,13/11/2054,economy,RES2043,Marine
3,6-56*#1671R3-01,2046,25/11/2052,statement,RES2046,Marine
4,AMRU542C-01,2047,27/11/2050,common,RES2047,Microbial
...,...,...,...,...,...,...
79995,TIC2041-1762RH-03,2041,04/02/2053,method,RES2041,Marine
79996,7-67*#1989R2-02,2046,05/06/2054,success,RES2046,Marine
79997,AMRU1191B-03,2045,16/08/2054,suffer,RES2045,Microbial
79998,SAF43-1599RH-02,2043,27/10/2054,various,RES2043,Marine


In [199]:
# Inserting assignedDate and submittedDate
subfraction_df.assignedDate = pd.to_datetime(subfraction_df.assignedDate, format='%d/%m/%Y')
subfraction_df.insert(3, 'submittedDate', subfraction_df['assignedDate'])
# Date submitted - do 3 weeks after previous assignedDate
subfraction_df['submittedDate'] = subfraction_df['submittedDate'] + timedelta(days=21)
subfraction_df['submittedDate'] = subfraction_df['submittedDate'].dt.strftime('%d/%m/%Y')
# Date assigned - do a week after submittedDate
subfraction_df['assignedDate'] = subfraction_df['assignedDate'] + timedelta(days=28)
subfraction_df['assignedDate'] = subfraction_df['assignedDate'].dt.strftime('%d/%m/%Y')
subfraction_df

,supplierID,year,assignedDate,submittedDate,species,permitNo,project
0,MOZ42-9322RH-01,2042,06/12/2054,29/11/2054,six,RES2042,Marine
1,MOZ42-4072RH-01,2042,26/12/2052,19/12/2052,decide,RES2042,Marine
2,SAF43-11100RH-01,2043,11/12/2054,04/12/2054,economy,RES2043,Marine
3,6-56*#1671R3-01,2046,23/12/2052,16/12/2052,statement,RES2046,Marine
4,AMRU542C-01,2047,25/12/2050,18/12/2050,common,RES2047,Microbial
...,...,...,...,...,...,...,...
79995,TIC2041-1762RH-03,2041,04/03/2053,25/02/2053,method,RES2041,Marine
79996,7-67*#1989R2-02,2046,03/07/2054,26/06/2054,success,RES2046,Marine
79997,AMRU1191B-03,2045,13/09/2054,06/09/2054,suffer,RES2045,Microbial
79998,SAF43-1599RH-02,2043,24/11/2054,17/11/2054,various,RES2043,Marine


In [200]:
fake = Faker()
Faker.seed(1234)
random.seed(1234)

class Provider(BaseProvider):
    submitters = ['Ruby Johnson', 'Amanda Bowker', 'Priscilla Evans']
    assigners = ['Ruby Johnson', 'Joshua Keener']
    destinations = ['RU Bioassay', 'Rhodes MS', 'CD Experiment', 'RU LC-MS & 1H-NMR', 'RU LC/MS']
    researchgroups = ['Smith', 'Roodt']
    solvents = ['MeOH', 'DCM:MeOH 2:1', 'DCM/MeOH']
    sampleTypes = ['Fraction', 'SPE fraction', 'Fraction from flash', 'HPLC Fraction']
    concentrations = [0.25, 0.1, 0.5, 1.0, 20.0, 50.0]
    comment = [None, None, 'Used compound A in fractionation', 'Used compound B in fractionation', None, None, '80:20 setting', '60:40 setting']
    
    def submittedBy(self):
        return random.choice(self.submitters)

    def assignedBy(self):
        return random.choice(self.assigners)

    def destinationOfSample(self):
        return random.choice(self.destinations)

    def researchGroup(self):
        return random.choice(self.researchgroups)
    
    def solvent(self):
        return random.choice(self.solvents)
    
    def sampleType(self):
        return random.choice(self.sampleTypes)
    
    def concentration(self):
        return random.choice(self.concentrations)
    
    def comments(self):
        return random.choice(self.comment)

# Add the Provider to our faker object
fake.add_provider(Provider)

# First change any columns that must be the same for all fractions from a particular source
# Submitted by, research group, project, concentration, solvent, species, sample type, assigned by, submitted date and assigned date (but will be same either way), destination

def create_subfraction_data(fake, no_of_rows):
    columns = ['assignedBy', 'destinationOfSample', 'sampleType', 'researchGroup', 'submittedBy', 'solvent', 'concentration', 'comments']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

subfractionSubset_df = create_subfraction_data(fake, len(subfraction_df))
subfractionSubset_df

,assignedBy,destinationOfSample,sampleType,researchGroup,submittedBy,solvent,concentration,comments
0,Joshua Keener,RU Bioassay,HPLC Fraction,Smith,Ruby Johnson,MeOH,20.00,Used compound B in fractionation
1,Ruby Johnson,CD Experiment,Fraction from flash,Roodt,Amanda Bowker,DCM/MeOH,0.25,Used compound B in fractionation
2,Ruby Johnson,Rhodes MS,SPE fraction,Smith,Amanda Bowker,MeOH,50.00,Used compound A in fractionation
3,Ruby Johnson,RU Bioassay,HPLC Fraction,Smith,Amanda Bowker,DCM/MeOH,0.50,80:20 setting
4,Ruby Johnson,CD Experiment,SPE fraction,Roodt,Priscilla Evans,DCM:MeOH 2:1,20.00,None
...,...,...,...,...,...,...,...,...
79995,Ruby Johnson,CD Experiment,HPLC Fraction,Smith,Ruby Johnson,DCM:MeOH 2:1,50.00,None
79996,Ruby Johnson,RU Bioassay,SPE fraction,Roodt,Amanda Bowker,DCM/MeOH,0.25,None
79997,Joshua Keener,Rhodes MS,SPE fraction,Roodt,Ruby Johnson,DCM/MeOH,50.00,Used compound A in fractionation
79998,Joshua Keener,CD Experiment,SPE fraction,Smith,Priscilla Evans,MeOH,0.10,None


In [201]:
# Join subsets together to get almost full MNP table
newSubFraction_df = pd.concat([subfraction_df, subfractionSubset_df], axis=1)
newSubFraction_df.rename(columns={'supplierID':'sourceID'}, inplace=True)
# Get columns in same order as MNP spreadsheet
orderCols = ['year', 'assignedBy', 'assignedDate', 'destinationOfSample', 'submittedDate', 'sampleType', 'concentration', 'solvent', 'project', 'sourceID', 'species', 'permitNo', 'researchGroup', 'submittedBy', 'comments']
newSubFraction_df = newSubFraction_df[orderCols]
newSubFraction_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2042,Joshua Keener,06/12/2054,RU Bioassay,29/11/2054,HPLC Fraction,20.00,MeOH,Marine,MOZ42-9322RH-01,six,RES2042,Smith,Ruby Johnson,Used compound B in fractionation
1,2042,Ruby Johnson,26/12/2052,CD Experiment,19/12/2052,Fraction from flash,0.25,DCM/MeOH,Marine,MOZ42-4072RH-01,decide,RES2042,Roodt,Amanda Bowker,Used compound B in fractionation
2,2043,Ruby Johnson,11/12/2054,Rhodes MS,04/12/2054,SPE fraction,50.00,MeOH,Marine,SAF43-11100RH-01,economy,RES2043,Smith,Amanda Bowker,Used compound A in fractionation
3,2046,Ruby Johnson,23/12/2052,RU Bioassay,16/12/2052,HPLC Fraction,0.50,DCM/MeOH,Marine,6-56*#1671R3-01,statement,RES2046,Smith,Amanda Bowker,80:20 setting
4,2047,Ruby Johnson,25/12/2050,CD Experiment,18/12/2050,SPE fraction,20.00,DCM:MeOH 2:1,Microbial,AMRU542C-01,common,RES2047,Roodt,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,2041,Ruby Johnson,04/03/2053,CD Experiment,25/02/2053,HPLC Fraction,50.00,DCM:MeOH 2:1,Marine,TIC2041-1762RH-03,method,RES2041,Smith,Ruby Johnson,None
79996,2046,Ruby Johnson,03/07/2054,RU Bioassay,26/06/2054,SPE fraction,0.25,DCM/MeOH,Marine,7-67*#1989R2-02,success,RES2046,Roodt,Amanda Bowker,None
79997,2045,Joshua Keener,13/09/2054,Rhodes MS,06/09/2054,SPE fraction,50.00,DCM/MeOH,Microbial,AMRU1191B-03,suffer,RES2045,Roodt,Ruby Johnson,Used compound A in fractionation
79998,2043,Joshua Keener,24/11/2054,CD Experiment,17/11/2054,SPE fraction,0.10,MeOH,Marine,SAF43-1599RH-02,various,RES2043,Smith,Priscilla Evans,None


In [202]:
# Get 20000 records from subset that is fractionated
numSampled2 = 20000
subfraction1_df = newSubFraction_df.sample(n = numSampled2, random_state=1231, ignore_index=True)
subfraction2_df = newSubFraction_df.sample(n = numSampled2, random_state=1232, ignore_index=True)
subfraction3_df = newSubFraction_df.sample(n = numSampled2, random_state=1233, ignore_index=True)
subfraction4_df = newSubFraction_df.sample(n = numSampled2, random_state=1234, ignore_index=True)
subfraction1_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2043,Joshua Keener,04/12/2053,RU LC/MS,27/11/2053,SPE fraction,0.25,DCM/MeOH,Marine,SAF43-10430RH-02,just,RES2043,Smith,Amanda Bowker,Used compound A in fractionation
1,2042,Joshua Keener,18/09/2054,RU LC/MS,11/09/2054,SPE fraction,20.00,DCM/MeOH,Marine,MOZ42-7440RH-01,offer,RES2042,Smith,Priscilla Evans,80:20 setting
2,2045,Ruby Johnson,14/10/2050,RU Bioassay,07/10/2050,Fraction,1.00,DCM:MeOH 2:1,Microbial,AMRU2287C-01,which,RES2045,Roodt,Priscilla Evans,80:20 setting
3,2045,Joshua Keener,22/06/2050,RU LC-MS & 1H-NMR,15/06/2050,Fraction,50.00,DCM/MeOH,Microbial,AMRU2229A-01,point,RES2045,Smith,Ruby Johnson,Used compound A in fractionation
4,2045,Joshua Keener,11/07/2051,RU Bioassay,04/07/2051,Fraction from flash,1.00,MeOH,Microbial,AMRU6373B-01,mention,RES2045,Roodt,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2041,Joshua Keener,17/06/2052,CD Experiment,10/06/2052,Fraction,0.10,MeOH,Marine,TIC2041-1843RH-03,type,RES2041,Smith,Priscilla Evans,None
19996,2045,Ruby Johnson,31/12/2050,Rhodes MS,24/12/2050,Fraction,0.50,DCM:MeOH 2:1,Microbial,AMRU3385C-04,next,RES2045,Roodt,Amanda Bowker,Used compound B in fractionation
19997,2042,Ruby Johnson,14/06/2051,RU LC-MS & 1H-NMR,07/06/2051,HPLC Fraction,20.00,DCM:MeOH 2:1,Marine,MOZ42-9537RJ-01,whether,RES2042,Smith,Priscilla Evans,None
19998,2045,Joshua Keener,16/08/2052,RU LC/MS,09/08/2052,HPLC Fraction,0.10,DCM:MeOH 2:1,Microbial,AMRU3437B-01,three,RES2045,Roodt,Priscilla Evans,None


In [203]:
fractionSecond_df = pd.concat([subfraction1_df, subfraction2_df, subfraction3_df, subfraction4_df], ignore_index=True)
fractionSecond_df
# Already have project

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2043,Joshua Keener,04/12/2053,RU LC/MS,27/11/2053,SPE fraction,0.25,DCM/MeOH,Marine,SAF43-10430RH-02,just,RES2043,Smith,Amanda Bowker,Used compound A in fractionation
1,2042,Joshua Keener,18/09/2054,RU LC/MS,11/09/2054,SPE fraction,20.00,DCM/MeOH,Marine,MOZ42-7440RH-01,offer,RES2042,Smith,Priscilla Evans,80:20 setting
2,2045,Ruby Johnson,14/10/2050,RU Bioassay,07/10/2050,Fraction,1.00,DCM:MeOH 2:1,Microbial,AMRU2287C-01,which,RES2045,Roodt,Priscilla Evans,80:20 setting
3,2045,Joshua Keener,22/06/2050,RU LC-MS & 1H-NMR,15/06/2050,Fraction,50.00,DCM/MeOH,Microbial,AMRU2229A-01,point,RES2045,Smith,Ruby Johnson,Used compound A in fractionation
4,2045,Joshua Keener,11/07/2051,RU Bioassay,04/07/2051,Fraction from flash,1.00,MeOH,Microbial,AMRU6373B-01,mention,RES2045,Roodt,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,2042,Ruby Johnson,06/12/2054,Rhodes MS,29/11/2054,HPLC Fraction,20.00,DCM:MeOH 2:1,Marine,MOZ42-3068RH-02,blue,RES2042,Smith,Ruby Johnson,None
79996,2043,Ruby Johnson,30/03/2050,Rhodes MS,23/03/2050,HPLC Fraction,50.00,DCM:MeOH 2:1,Marine,SAF43-751RJ-01,either,RES2043,Roodt,Amanda Bowker,None
79997,2045,Ruby Johnson,01/01/2052,RU LC/MS,25/12/2051,SPE fraction,0.50,DCM/MeOH,Microbial,AMRU3893A-03,institution,RES2045,Roodt,Ruby Johnson,60:40 setting
79998,2041,Joshua Keener,08/07/2052,RU Bioassay,01/07/2052,Fraction from flash,50.00,MeOH,Marine,TIC2041-9790RH-01,out,RES2041,Smith,Ruby Johnson,None


In [204]:
# Parts that are different per subfraction:
# Amount available
fake = Faker()
Faker.seed(1233)
random.seed(1233)

class Provider(BaseProvider):
    def amountAvailable(self):
        return (round (random.uniform(0.0, 1.0)*100, 1))
    
fake.add_provider(Provider)

def create_amountSub_data(fake, no_of_rows):
    columns = ['amountAvailable']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

amountSub_df = create_amountSub_data(fake, len(fractionSecond_df))

fractionSecond_df = pd.concat([amountSub_df, fractionSecond_df], axis=1)
fractionSecond_df

,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,31.4,2043,Joshua Keener,04/12/2053,RU LC/MS,27/11/2053,SPE fraction,0.25,DCM/MeOH,Marine,SAF43-10430RH-02,just,RES2043,Smith,Amanda Bowker,Used compound A in fractionation
1,50.9,2042,Joshua Keener,18/09/2054,RU LC/MS,11/09/2054,SPE fraction,20.00,DCM/MeOH,Marine,MOZ42-7440RH-01,offer,RES2042,Smith,Priscilla Evans,80:20 setting
2,18.5,2045,Ruby Johnson,14/10/2050,RU Bioassay,07/10/2050,Fraction,1.00,DCM:MeOH 2:1,Microbial,AMRU2287C-01,which,RES2045,Roodt,Priscilla Evans,80:20 setting
3,28.2,2045,Joshua Keener,22/06/2050,RU LC-MS & 1H-NMR,15/06/2050,Fraction,50.00,DCM/MeOH,Microbial,AMRU2229A-01,point,RES2045,Smith,Ruby Johnson,Used compound A in fractionation
4,67.0,2045,Joshua Keener,11/07/2051,RU Bioassay,04/07/2051,Fraction from flash,1.00,MeOH,Microbial,AMRU6373B-01,mention,RES2045,Roodt,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,68.1,2042,Ruby Johnson,06/12/2054,Rhodes MS,29/11/2054,HPLC Fraction,20.00,DCM:MeOH 2:1,Marine,MOZ42-3068RH-02,blue,RES2042,Smith,Ruby Johnson,None
79996,52.7,2043,Ruby Johnson,30/03/2050,Rhodes MS,23/03/2050,HPLC Fraction,50.00,DCM:MeOH 2:1,Marine,SAF43-751RJ-01,either,RES2043,Roodt,Amanda Bowker,None
79997,75.0,2045,Ruby Johnson,01/01/2052,RU LC/MS,25/12/2051,SPE fraction,0.50,DCM/MeOH,Microbial,AMRU3893A-03,institution,RES2045,Roodt,Ruby Johnson,60:40 setting
79998,73.7,2041,Joshua Keener,08/07/2052,RU Bioassay,01/07/2052,Fraction from flash,50.00,MeOH,Marine,TIC2041-9790RH-01,out,RES2041,Smith,Ruby Johnson,None


In [205]:
fractionSecond_df.insert(0, 'mnpID', range(len(fractionFirst_df) + 1, 1 + len(fractionFirst_df) + len(fractionSecond_df)))
fractionSecond_df['mnpID'] = 'MNP' + fractionSecond_df['mnpID'].astype(str).str.zfill(5)
fractionSecond_df

,mnpID,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,MNP80001,31.4,2043,Joshua Keener,04/12/2053,RU LC/MS,27/11/2053,SPE fraction,0.25,DCM/MeOH,Marine,SAF43-10430RH-02,just,RES2043,Smith,Amanda Bowker,Used compound A in fractionation
1,MNP80002,50.9,2042,Joshua Keener,18/09/2054,RU LC/MS,11/09/2054,SPE fraction,20.00,DCM/MeOH,Marine,MOZ42-7440RH-01,offer,RES2042,Smith,Priscilla Evans,80:20 setting
2,MNP80003,18.5,2045,Ruby Johnson,14/10/2050,RU Bioassay,07/10/2050,Fraction,1.00,DCM:MeOH 2:1,Microbial,AMRU2287C-01,which,RES2045,Roodt,Priscilla Evans,80:20 setting
3,MNP80004,28.2,2045,Joshua Keener,22/06/2050,RU LC-MS & 1H-NMR,15/06/2050,Fraction,50.00,DCM/MeOH,Microbial,AMRU2229A-01,point,RES2045,Smith,Ruby Johnson,Used compound A in fractionation
4,MNP80005,67.0,2045,Joshua Keener,11/07/2051,RU Bioassay,04/07/2051,Fraction from flash,1.00,MeOH,Microbial,AMRU6373B-01,mention,RES2045,Roodt,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,MNP159996,68.1,2042,Ruby Johnson,06/12/2054,Rhodes MS,29/11/2054,HPLC Fraction,20.00,DCM:MeOH 2:1,Marine,MOZ42-3068RH-02,blue,RES2042,Smith,Ruby Johnson,None
79996,MNP159997,52.7,2043,Ruby Johnson,30/03/2050,Rhodes MS,23/03/2050,HPLC Fraction,50.00,DCM:MeOH 2:1,Marine,SAF43-751RJ-01,either,RES2043,Roodt,Amanda Bowker,None
79997,MNP159998,75.0,2045,Ruby Johnson,01/01/2052,RU LC/MS,25/12/2051,SPE fraction,0.50,DCM/MeOH,Microbial,AMRU3893A-03,institution,RES2045,Roodt,Ruby Johnson,60:40 setting
79998,MNP159999,73.7,2041,Joshua Keener,08/07/2052,RU Bioassay,01/07/2052,Fraction from flash,50.00,MeOH,Marine,TIC2041-9790RH-01,out,RES2041,Smith,Ruby Johnson,None


In [206]:
# Make SupplierIDs - same as sourceID, except have -01, -02 etc for initial fractions (will be a,b,c for rest of subfractions)
sourceSubSeries = fractionSecond_df['sourceID']
listSubSupplier = []
subsupplier_freq = {}

for index, value in sourceSubSeries.items():
    if (value in subsupplier_freq):
        subsupplier_freq[value] += 1
    else:
        subsupplier_freq[value] = 1
    listSubSupplier.append(value + str(chr(subsupplier_freq[value] + 96)))

subsupplierSeries = pd.Series(listSubSupplier)
#print(supplierSeries)

fractionSecond_df.insert(1, 'supplierID', subsupplierSeries)
# Include level (2)
fractionSecond_df.insert(17, 'level', 2)
fractionSecond_df

,mnpID,supplierID,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,level,comments
0,MNP80001,SAF43-10430RH-02a,31.4,2043,Joshua Keener,04/12/2053,RU LC/MS,27/11/2053,SPE fraction,0.25,DCM/MeOH,Marine,SAF43-10430RH-02,just,RES2043,Smith,Amanda Bowker,2,Used compound A in fractionation
1,MNP80002,MOZ42-7440RH-01a,50.9,2042,Joshua Keener,18/09/2054,RU LC/MS,11/09/2054,SPE fraction,20.00,DCM/MeOH,Marine,MOZ42-7440RH-01,offer,RES2042,Smith,Priscilla Evans,2,80:20 setting
2,MNP80003,AMRU2287C-01a,18.5,2045,Ruby Johnson,14/10/2050,RU Bioassay,07/10/2050,Fraction,1.00,DCM:MeOH 2:1,Microbial,AMRU2287C-01,which,RES2045,Roodt,Priscilla Evans,2,80:20 setting
3,MNP80004,AMRU2229A-01a,28.2,2045,Joshua Keener,22/06/2050,RU LC-MS & 1H-NMR,15/06/2050,Fraction,50.00,DCM/MeOH,Microbial,AMRU2229A-01,point,RES2045,Smith,Ruby Johnson,2,Used compound A in fractionation
4,MNP80005,AMRU6373B-01a,67.0,2045,Joshua Keener,11/07/2051,RU Bioassay,04/07/2051,Fraction from flash,1.00,MeOH,Microbial,AMRU6373B-01,mention,RES2045,Roodt,Priscilla Evans,2,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,MNP159996,MOZ42-3068RH-02b,68.1,2042,Ruby Johnson,06/12/2054,Rhodes MS,29/11/2054,HPLC Fraction,20.00,DCM:MeOH 2:1,Marine,MOZ42-3068RH-02,blue,RES2042,Smith,Ruby Johnson,2,None
79996,MNP159997,SAF43-751RJ-01b,52.7,2043,Ruby Johnson,30/03/2050,Rhodes MS,23/03/2050,HPLC Fraction,50.00,DCM:MeOH 2:1,Marine,SAF43-751RJ-01,either,RES2043,Roodt,Amanda Bowker,2,None
79997,MNP159998,AMRU3893A-03c,75.0,2045,Ruby Johnson,01/01/2052,RU LC/MS,25/12/2051,SPE fraction,0.50,DCM/MeOH,Microbial,AMRU3893A-03,institution,RES2045,Roodt,Ruby Johnson,2,60:40 setting
79998,MNP159999,TIC2041-9790RH-01c,73.7,2041,Joshua Keener,08/07/2052,RU Bioassay,01/07/2052,Fraction from flash,50.00,MeOH,Marine,TIC2041-9790RH-01,out,RES2041,Smith,Ruby Johnson,2,None


Subsubfractions

In [207]:
subsubfraction_df = fractionSecond_df.copy(deep=True)
subsubfraction_df = subsubfraction_df[["supplierID", "year", "assignedDate", "species", "permitNo", "project"]]
subsubfraction_df

,supplierID,year,assignedDate,species,permitNo,project
0,SAF43-10430RH-02a,2043,04/12/2053,just,RES2043,Marine
1,MOZ42-7440RH-01a,2042,18/09/2054,offer,RES2042,Marine
2,AMRU2287C-01a,2045,14/10/2050,which,RES2045,Microbial
3,AMRU2229A-01a,2045,22/06/2050,point,RES2045,Microbial
4,AMRU6373B-01a,2045,11/07/2051,mention,RES2045,Microbial
...,...,...,...,...,...,...
79995,MOZ42-3068RH-02b,2042,06/12/2054,blue,RES2042,Marine
79996,SAF43-751RJ-01b,2043,30/03/2050,either,RES2043,Marine
79997,AMRU3893A-03c,2045,01/01/2052,institution,RES2045,Microbial
79998,TIC2041-9790RH-01c,2041,08/07/2052,out,RES2041,Marine


In [208]:
# Inserting assignedDate and submittedDate
subsubfraction_df.assignedDate = pd.to_datetime(subsubfraction_df.assignedDate, format='%d/%m/%Y')
subsubfraction_df.insert(3, 'submittedDate', subsubfraction_df['assignedDate'])
# Date submitted - do 3 weeks after previous assignedDate
subsubfraction_df['submittedDate'] = subsubfraction_df['submittedDate'] + timedelta(days=21)
subsubfraction_df['submittedDate'] = subsubfraction_df['submittedDate'].dt.strftime('%d/%m/%Y')
# Date assigned - do a week after submittedDate
subsubfraction_df['assignedDate'] = subsubfraction_df['assignedDate'] + timedelta(days=28)
subsubfraction_df['assignedDate'] = subsubfraction_df['assignedDate'].dt.strftime('%d/%m/%Y')
subsubfraction_df

,supplierID,year,assignedDate,submittedDate,species,permitNo,project
0,SAF43-10430RH-02a,2043,01/01/2054,25/12/2053,just,RES2043,Marine
1,MOZ42-7440RH-01a,2042,16/10/2054,09/10/2054,offer,RES2042,Marine
2,AMRU2287C-01a,2045,11/11/2050,04/11/2050,which,RES2045,Microbial
3,AMRU2229A-01a,2045,20/07/2050,13/07/2050,point,RES2045,Microbial
4,AMRU6373B-01a,2045,08/08/2051,01/08/2051,mention,RES2045,Microbial
...,...,...,...,...,...,...,...
79995,MOZ42-3068RH-02b,2042,03/01/2055,27/12/2054,blue,RES2042,Marine
79996,SAF43-751RJ-01b,2043,27/04/2050,20/04/2050,either,RES2043,Marine
79997,AMRU3893A-03c,2045,29/01/2052,22/01/2052,institution,RES2045,Microbial
79998,TIC2041-9790RH-01c,2041,05/08/2052,29/07/2052,out,RES2041,Marine


In [209]:
fake = Faker()
Faker.seed(1233)
random.seed(1233)

class Provider(BaseProvider):
    submitters = ['Ruby Johnson', 'Amanda Bowker', 'Priscilla Evans']
    assigners = ['Ruby Johnson', 'Joshua Keener']
    destinations = ['RU Bioassay', 'Rhodes MS', 'CD Experiment', 'RU LC-MS & 1H-NMR', 'RU LC/MS']
    researchgroups = ['Smith', 'Roodt']
    solvents = ['MeOH', 'DCM:MeOH 2:1', 'DCM/MeOH']
    sampleTypes = ['Fraction', 'SPE fraction', 'Fraction from flash', 'HPLC Fraction']
    concentrations = [0.25, 0.1, 0.5, 1.0, 20.0, 50.0]
    comment = [None, None, 'Used compound F in fractionation', 'Used compound G in fractionation', None, None, '70:30 setting', '20:80 setting']
    
    def submittedBy(self):
        return random.choice(self.submitters)

    def assignedBy(self):
        return random.choice(self.assigners)

    def destinationOfSample(self):
        return random.choice(self.destinations)

    def researchGroup(self):
        return random.choice(self.researchgroups)
    
    def solvent(self):
        return random.choice(self.solvents)
    
    def sampleType(self):
        return random.choice(self.sampleTypes)
    
    def concentration(self):
        return random.choice(self.concentrations)
    
    def comments(self):
        return random.choice(self.comment)

# Add the Provider to our faker object
fake.add_provider(Provider)

# First change any columns that must be the same for all fractions from a particular source
# Submitted by, research group, project, concentration, solvent, species, sample type, assigned by, submitted date and assigned date (but will be same either way), destination

def create_subsubfraction_data(fake, no_of_rows):
    columns = ['assignedBy', 'destinationOfSample', 'sampleType', 'researchGroup', 'submittedBy', 'solvent', 'concentration', 'comments']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

subsubfractionSubset_df = create_subsubfraction_data(fake, len(subsubfraction_df))
subsubfractionSubset_df

,assignedBy,destinationOfSample,sampleType,researchGroup,submittedBy,solvent,concentration,comments
0,Joshua Keener,RU LC-MS & 1H-NMR,HPLC Fraction,Smith,Ruby Johnson,DCM:MeOH 2:1,0.10,None
1,Joshua Keener,CD Experiment,Fraction from flash,Roodt,Priscilla Evans,DCM/MeOH,50.00,20:80 setting
2,Joshua Keener,CD Experiment,Fraction,Roodt,Priscilla Evans,MeOH,0.25,None
3,Ruby Johnson,CD Experiment,SPE fraction,Roodt,Priscilla Evans,DCM:MeOH 2:1,1.00,None
4,Joshua Keener,RU LC-MS & 1H-NMR,HPLC Fraction,Roodt,Amanda Bowker,DCM:MeOH 2:1,50.00,None
...,...,...,...,...,...,...,...,...
79995,Ruby Johnson,CD Experiment,Fraction from flash,Smith,Priscilla Evans,DCM/MeOH,1.00,None
79996,Joshua Keener,RU Bioassay,Fraction from flash,Smith,Priscilla Evans,DCM/MeOH,50.00,None
79997,Joshua Keener,Rhodes MS,SPE fraction,Roodt,Amanda Bowker,DCM:MeOH 2:1,20.00,70:30 setting
79998,Ruby Johnson,CD Experiment,SPE fraction,Smith,Ruby Johnson,DCM:MeOH 2:1,1.00,None


In [210]:
newSubSubFraction_df = pd.concat([subsubfraction_df, subsubfractionSubset_df], axis=1)
newSubSubFraction_df.rename(columns={'supplierID':'sourceID'}, inplace=True)
# Get columns in same order as MNP spreadsheet
orderCols = ['year', 'assignedBy', 'assignedDate', 'destinationOfSample', 'submittedDate', 'sampleType', 'concentration', 'solvent', 'project', 'sourceID', 'species', 'permitNo', 'researchGroup', 'submittedBy', 'comments']
newSubSubFraction_df = newSubSubFraction_df[orderCols]
newSubSubFraction_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2043,Joshua Keener,01/01/2054,RU LC-MS & 1H-NMR,25/12/2053,HPLC Fraction,0.10,DCM:MeOH 2:1,Marine,SAF43-10430RH-02a,just,RES2043,Smith,Ruby Johnson,None
1,2042,Joshua Keener,16/10/2054,CD Experiment,09/10/2054,Fraction from flash,50.00,DCM/MeOH,Marine,MOZ42-7440RH-01a,offer,RES2042,Roodt,Priscilla Evans,20:80 setting
2,2045,Joshua Keener,11/11/2050,CD Experiment,04/11/2050,Fraction,0.25,MeOH,Microbial,AMRU2287C-01a,which,RES2045,Roodt,Priscilla Evans,None
3,2045,Ruby Johnson,20/07/2050,CD Experiment,13/07/2050,SPE fraction,1.00,DCM:MeOH 2:1,Microbial,AMRU2229A-01a,point,RES2045,Roodt,Priscilla Evans,None
4,2045,Joshua Keener,08/08/2051,RU LC-MS & 1H-NMR,01/08/2051,HPLC Fraction,50.00,DCM:MeOH 2:1,Microbial,AMRU6373B-01a,mention,RES2045,Roodt,Amanda Bowker,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,2042,Ruby Johnson,03/01/2055,CD Experiment,27/12/2054,Fraction from flash,1.00,DCM/MeOH,Marine,MOZ42-3068RH-02b,blue,RES2042,Smith,Priscilla Evans,None
79996,2043,Joshua Keener,27/04/2050,RU Bioassay,20/04/2050,Fraction from flash,50.00,DCM/MeOH,Marine,SAF43-751RJ-01b,either,RES2043,Smith,Priscilla Evans,None
79997,2045,Joshua Keener,29/01/2052,Rhodes MS,22/01/2052,SPE fraction,20.00,DCM:MeOH 2:1,Microbial,AMRU3893A-03c,institution,RES2045,Roodt,Amanda Bowker,70:30 setting
79998,2041,Ruby Johnson,05/08/2052,CD Experiment,29/07/2052,SPE fraction,1.00,DCM:MeOH 2:1,Marine,TIC2041-9790RH-01c,out,RES2041,Smith,Ruby Johnson,None


In [211]:
numSampled3 = 20000
subsubfraction1_df = newSubSubFraction_df.sample(n = numSampled3, random_state=1231, ignore_index=True)
subsubfraction2_df = newSubSubFraction_df.sample(n = numSampled3, random_state=1232, ignore_index=True)
subsubfraction3_df = newSubSubFraction_df.sample(n = numSampled3, random_state=1233, ignore_index=True)
subsubfraction4_df = newSubSubFraction_df.sample(n = numSampled3, random_state=1234, ignore_index=True)
subsubfraction1_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2047,Joshua Keener,23/12/2051,Rhodes MS,16/12/2051,HPLC Fraction,1.00,MeOH,Microbial,AMRU3638B-01b,today,RES2047,Roodt,Amanda Bowker,Used compound G in fractionation
1,2043,Joshua Keener,11/04/2054,Rhodes MS,04/04/2054,Fraction,0.10,DCM:MeOH 2:1,Marine,SAF43-6933JR-01b,loss,RES2043,Roodt,Ruby Johnson,Used compound G in fractionation
2,2043,Ruby Johnson,30/12/2052,CD Experiment,23/12/2052,Fraction from flash,50.00,DCM:MeOH 2:1,Marine,SAF43-6676RH-02a,return,RES2043,Smith,Ruby Johnson,None
3,2045,Ruby Johnson,26/10/2054,RU Bioassay,19/10/2054,SPE fraction,0.50,DCM:MeOH 2:1,Microbial,AMRU2459B-01a,respond,RES2045,Roodt,Ruby Johnson,Used compound F in fractionation
4,2043,Ruby Johnson,05/06/2051,CD Experiment,29/05/2051,Fraction,0.25,MeOH,Marine,SAF43-9299JR-01a,international,RES2043,Roodt,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2042,Ruby Johnson,31/07/2054,RU Bioassay,24/07/2054,Fraction,50.00,DCM:MeOH 2:1,Marine,MOZ42-5879JR-01a,place,RES2042,Roodt,Ruby Johnson,None
19996,2041,Ruby Johnson,03/03/2051,RU Bioassay,24/02/2051,Fraction,1.00,DCM/MeOH,Marine,TIC2041-3702RJ-01b,under,RES2041,Smith,Amanda Bowker,None
19997,2041,Joshua Keener,30/08/2050,Rhodes MS,23/08/2050,HPLC Fraction,1.00,MeOH,Marine,TIC2041-7413RJ-01a,thousand,RES2041,Roodt,Priscilla Evans,None
19998,2041,Joshua Keener,31/01/2055,RU LC-MS & 1H-NMR,24/01/2055,HPLC Fraction,0.50,MeOH,Marine,TIC2041-6855RJ-02a,whatever,RES2041,Roodt,Ruby Johnson,None


In [212]:
fractionThird_df = pd.concat([subsubfraction1_df, subsubfraction2_df, subsubfraction3_df, subsubfraction4_df], ignore_index=True)
fractionThird_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2047,Joshua Keener,23/12/2051,Rhodes MS,16/12/2051,HPLC Fraction,1.00,MeOH,Microbial,AMRU3638B-01b,today,RES2047,Roodt,Amanda Bowker,Used compound G in fractionation
1,2043,Joshua Keener,11/04/2054,Rhodes MS,04/04/2054,Fraction,0.10,DCM:MeOH 2:1,Marine,SAF43-6933JR-01b,loss,RES2043,Roodt,Ruby Johnson,Used compound G in fractionation
2,2043,Ruby Johnson,30/12/2052,CD Experiment,23/12/2052,Fraction from flash,50.00,DCM:MeOH 2:1,Marine,SAF43-6676RH-02a,return,RES2043,Smith,Ruby Johnson,None
3,2045,Ruby Johnson,26/10/2054,RU Bioassay,19/10/2054,SPE fraction,0.50,DCM:MeOH 2:1,Microbial,AMRU2459B-01a,respond,RES2045,Roodt,Ruby Johnson,Used compound F in fractionation
4,2043,Ruby Johnson,05/06/2051,CD Experiment,29/05/2051,Fraction,0.25,MeOH,Marine,SAF43-9299JR-01a,international,RES2043,Roodt,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,2046,Ruby Johnson,31/07/2054,CD Experiment,24/07/2054,SPE fraction,50.00,DCM:MeOH 2:1,Marine,11-101*#3009R2-01a,might,RES2046,Smith,Priscilla Evans,20:80 setting
79996,2047,Ruby Johnson,24/06/2051,Rhodes MS,17/06/2051,HPLC Fraction,0.50,DCM/MeOH,Microbial,AMRU5838A-01a,break,RES2047,Roodt,Ruby Johnson,None
79997,2047,Joshua Keener,26/01/2055,RU LC/MS,19/01/2055,HPLC Fraction,50.00,DCM:MeOH 2:1,Microbial,AMRU3694C-02a,something,RES2047,Smith,Ruby Johnson,None
79998,2046,Ruby Johnson,07/03/2052,RU LC/MS,29/02/2052,SPE fraction,0.10,DCM/MeOH,Marine,1-7*#202R1-01a,single,RES2046,Smith,Ruby Johnson,None


In [213]:
# Parts that are different per subsubfraction:
# Amount available
fake = Faker()
Faker.seed(1232)
random.seed(1232)

class Provider(BaseProvider):
    def amountAvailable(self):
        return (round (random.uniform(0.0, 1.0)*100, 1))
    
fake.add_provider(Provider)

def create_amountSub_data(fake, no_of_rows):
    columns = ['amountAvailable']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

amountsubSub_df = create_amountSub_data(fake, len(fractionThird_df))

fractionThird_df = pd.concat([amountsubSub_df, fractionThird_df], axis=1)
fractionThird_df

,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,57.7,2047,Joshua Keener,23/12/2051,Rhodes MS,16/12/2051,HPLC Fraction,1.00,MeOH,Microbial,AMRU3638B-01b,today,RES2047,Roodt,Amanda Bowker,Used compound G in fractionation
1,70.1,2043,Joshua Keener,11/04/2054,Rhodes MS,04/04/2054,Fraction,0.10,DCM:MeOH 2:1,Marine,SAF43-6933JR-01b,loss,RES2043,Roodt,Ruby Johnson,Used compound G in fractionation
2,90.8,2043,Ruby Johnson,30/12/2052,CD Experiment,23/12/2052,Fraction from flash,50.00,DCM:MeOH 2:1,Marine,SAF43-6676RH-02a,return,RES2043,Smith,Ruby Johnson,None
3,14.2,2045,Ruby Johnson,26/10/2054,RU Bioassay,19/10/2054,SPE fraction,0.50,DCM:MeOH 2:1,Microbial,AMRU2459B-01a,respond,RES2045,Roodt,Ruby Johnson,Used compound F in fractionation
4,3.3,2043,Ruby Johnson,05/06/2051,CD Experiment,29/05/2051,Fraction,0.25,MeOH,Marine,SAF43-9299JR-01a,international,RES2043,Roodt,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,72.7,2046,Ruby Johnson,31/07/2054,CD Experiment,24/07/2054,SPE fraction,50.00,DCM:MeOH 2:1,Marine,11-101*#3009R2-01a,might,RES2046,Smith,Priscilla Evans,20:80 setting
79996,60.5,2047,Ruby Johnson,24/06/2051,Rhodes MS,17/06/2051,HPLC Fraction,0.50,DCM/MeOH,Microbial,AMRU5838A-01a,break,RES2047,Roodt,Ruby Johnson,None
79997,75.0,2047,Joshua Keener,26/01/2055,RU LC/MS,19/01/2055,HPLC Fraction,50.00,DCM:MeOH 2:1,Microbial,AMRU3694C-02a,something,RES2047,Smith,Ruby Johnson,None
79998,26.9,2046,Ruby Johnson,07/03/2052,RU LC/MS,29/02/2052,SPE fraction,0.10,DCM/MeOH,Marine,1-7*#202R1-01a,single,RES2046,Smith,Ruby Johnson,None


In [214]:
# Make SupplierIDs - same as sourceID, except have -01, -02 etc for initial fractions (will be a,b,c for rest of subfractions)
# 
sourceSubSeries = fractionThird_df['sourceID']
listSubSupplier = []
subsupplier_freq = {}

for index, value in sourceSubSeries.items():
    if (value in subsupplier_freq):
        subsupplier_freq[value] += 1
    else:
        subsupplier_freq[value] = 1
    listSubSupplier.append(value + str(chr(subsupplier_freq[value] + 96)))

subsupplierSeries = pd.Series(listSubSupplier)
#print(supplierSeries)

fractionThird_df.insert(1, 'supplierID', subsupplierSeries)
fractionThird_df

,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,57.7,AMRU3638B-01ba,2047,Joshua Keener,23/12/2051,Rhodes MS,16/12/2051,HPLC Fraction,1.00,MeOH,Microbial,AMRU3638B-01b,today,RES2047,Roodt,Amanda Bowker,Used compound G in fractionation
1,70.1,SAF43-6933JR-01ba,2043,Joshua Keener,11/04/2054,Rhodes MS,04/04/2054,Fraction,0.10,DCM:MeOH 2:1,Marine,SAF43-6933JR-01b,loss,RES2043,Roodt,Ruby Johnson,Used compound G in fractionation
2,90.8,SAF43-6676RH-02aa,2043,Ruby Johnson,30/12/2052,CD Experiment,23/12/2052,Fraction from flash,50.00,DCM:MeOH 2:1,Marine,SAF43-6676RH-02a,return,RES2043,Smith,Ruby Johnson,None
3,14.2,AMRU2459B-01aa,2045,Ruby Johnson,26/10/2054,RU Bioassay,19/10/2054,SPE fraction,0.50,DCM:MeOH 2:1,Microbial,AMRU2459B-01a,respond,RES2045,Roodt,Ruby Johnson,Used compound F in fractionation
4,3.3,SAF43-9299JR-01aa,2043,Ruby Johnson,05/06/2051,CD Experiment,29/05/2051,Fraction,0.25,MeOH,Marine,SAF43-9299JR-01a,international,RES2043,Roodt,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,72.7,11-101*#3009R2-01ab,2046,Ruby Johnson,31/07/2054,CD Experiment,24/07/2054,SPE fraction,50.00,DCM:MeOH 2:1,Marine,11-101*#3009R2-01a,might,RES2046,Smith,Priscilla Evans,20:80 setting
79996,60.5,AMRU5838A-01ab,2047,Ruby Johnson,24/06/2051,Rhodes MS,17/06/2051,HPLC Fraction,0.50,DCM/MeOH,Microbial,AMRU5838A-01a,break,RES2047,Roodt,Ruby Johnson,None
79997,75.0,AMRU3694C-02ac,2047,Joshua Keener,26/01/2055,RU LC/MS,19/01/2055,HPLC Fraction,50.00,DCM:MeOH 2:1,Microbial,AMRU3694C-02a,something,RES2047,Smith,Ruby Johnson,None
79998,26.9,1-7*#202R1-01ac,2046,Ruby Johnson,07/03/2052,RU LC/MS,29/02/2052,SPE fraction,0.10,DCM/MeOH,Marine,1-7*#202R1-01a,single,RES2046,Smith,Ruby Johnson,None


In [215]:
#Get 20000 records that will be pure, rather than just fractions
pureFirst_df = fractionThird_df.sample(frac = 0.25, random_state=1234)
pureFirst_df

,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
68821,98.2,TIC2041-1065RH-02aa,2041,Joshua Keener,10/08/2054,Rhodes MS,03/08/2054,Fraction from flash,1.00,DCM:MeOH 2:1,Marine,TIC2041-1065RH-02a,church,RES2041,Smith,Ruby Johnson,70:30 setting
14323,42.0,SAF43-258RJ-01aa,2043,Joshua Keener,16/08/2051,RU LC-MS & 1H-NMR,09/08/2051,SPE fraction,0.25,DCM:MeOH 2:1,Marine,SAF43-258RJ-01a,society,RES2043,Smith,Amanda Bowker,None
23036,92.2,SAF43-2536JR-02ba,2043,Joshua Keener,21/05/2050,Rhodes MS,14/05/2050,Fraction from flash,1.00,DCM/MeOH,Marine,SAF43-2536JR-02b,something,RES2043,Roodt,Priscilla Evans,Used compound F in fractionation
28532,30.0,TIC2041-4160JR-02ab,2041,Ruby Johnson,24/06/2053,RU LC-MS & 1H-NMR,17/06/2053,Fraction from flash,20.00,MeOH,Marine,TIC2041-4160JR-02a,behind,RES2041,Smith,Ruby Johnson,None
62048,12.6,AMRU3724C-01aa,2047,Ruby Johnson,20/10/2054,RU LC-MS & 1H-NMR,13/10/2054,Fraction,50.00,DCM:MeOH 2:1,Microbial,AMRU3724C-01a,main,RES2047,Roodt,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55679,12.3,MOZ42-4395RJ-01aa,2042,Joshua Keener,10/04/2053,Rhodes MS,03/04/2053,Fraction,0.10,DCM:MeOH 2:1,Marine,MOZ42-4395RJ-01a,tough,RES2042,Roodt,Ruby Johnson,None
18038,20.2,AMRU5509B-01aa,2045,Joshua Keener,28/05/2053,RU Bioassay,21/05/2053,HPLC Fraction,0.25,MeOH,Microbial,AMRU5509B-01a,security,RES2045,Smith,Priscilla Evans,70:30 setting
60004,67.3,AMRU440C-03aa,2047,Ruby Johnson,25/07/2052,Rhodes MS,18/07/2052,Fraction from flash,1.00,MeOH,Microbial,AMRU440C-03a,receive,RES2047,Smith,Priscilla Evans,70:30 setting
6028,28.3,MOZ42-2781RH-02aa,2042,Joshua Keener,30/06/2053,RU LC-MS & 1H-NMR,23/06/2053,Fraction from flash,0.25,DCM/MeOH,Marine,MOZ42-2781RH-02a,bring,RES2042,Roodt,Ruby Johnson,20:80 setting


In [216]:
fractionThird_df = fractionThird_df.drop(pureFirst_df.index)
fractionThird_df.reset_index(drop=True, inplace=True)
fractionThird_df

,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,57.7,AMRU3638B-01ba,2047,Joshua Keener,23/12/2051,Rhodes MS,16/12/2051,HPLC Fraction,1.00,MeOH,Microbial,AMRU3638B-01b,today,RES2047,Roodt,Amanda Bowker,Used compound G in fractionation
1,70.1,SAF43-6933JR-01ba,2043,Joshua Keener,11/04/2054,Rhodes MS,04/04/2054,Fraction,0.10,DCM:MeOH 2:1,Marine,SAF43-6933JR-01b,loss,RES2043,Roodt,Ruby Johnson,Used compound G in fractionation
2,90.8,SAF43-6676RH-02aa,2043,Ruby Johnson,30/12/2052,CD Experiment,23/12/2052,Fraction from flash,50.00,DCM:MeOH 2:1,Marine,SAF43-6676RH-02a,return,RES2043,Smith,Ruby Johnson,None
3,14.2,AMRU2459B-01aa,2045,Ruby Johnson,26/10/2054,RU Bioassay,19/10/2054,SPE fraction,0.50,DCM:MeOH 2:1,Microbial,AMRU2459B-01a,respond,RES2045,Roodt,Ruby Johnson,Used compound F in fractionation
4,3.3,SAF43-9299JR-01aa,2043,Ruby Johnson,05/06/2051,CD Experiment,29/05/2051,Fraction,0.25,MeOH,Marine,SAF43-9299JR-01a,international,RES2043,Roodt,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,70.6,SAF43-10975RH-02aa,2043,Ruby Johnson,19/06/2050,Rhodes MS,12/06/2050,Fraction,1.00,MeOH,Marine,SAF43-10975RH-02a,quickly,RES2043,Roodt,Amanda Bowker,Used compound F in fractionation
59996,44.2,6-56*#1675R1-01ab,2046,Joshua Keener,06/01/2052,RU LC-MS & 1H-NMR,30/12/2051,SPE fraction,1.00,DCM:MeOH 2:1,Marine,6-56*#1675R1-01a,walk,RES2046,Smith,Ruby Johnson,70:30 setting
59997,88.3,MOZ42-2112JR-01ac,2042,Ruby Johnson,30/11/2053,RU LC/MS,23/11/2053,Fraction from flash,50.00,MeOH,Marine,MOZ42-2112JR-01a,raise,RES2042,Roodt,Amanda Bowker,None
59998,72.7,11-101*#3009R2-01ab,2046,Ruby Johnson,31/07/2054,CD Experiment,24/07/2054,SPE fraction,50.00,DCM:MeOH 2:1,Marine,11-101*#3009R2-01a,might,RES2046,Smith,Priscilla Evans,20:80 setting


In [217]:
offset = len(fractionFirst_df) + len(fractionSecond_df)
fractionThird_df.insert(0, 'mnpID', range(offset + 1, 1 + offset + len(fractionThird_df)))
fractionThird_df['mnpID'] = 'MNP' + fractionThird_df['mnpID'].astype(str).str.zfill(5)
# Include level (3)
fractionThird_df.insert(17, 'level', 3)
fractionThird_df

,mnpID,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,level,comments
0,MNP160001,57.7,AMRU3638B-01ba,2047,Joshua Keener,23/12/2051,Rhodes MS,16/12/2051,HPLC Fraction,1.00,MeOH,Microbial,AMRU3638B-01b,today,RES2047,Roodt,Amanda Bowker,3,Used compound G in fractionation
1,MNP160002,70.1,SAF43-6933JR-01ba,2043,Joshua Keener,11/04/2054,Rhodes MS,04/04/2054,Fraction,0.10,DCM:MeOH 2:1,Marine,SAF43-6933JR-01b,loss,RES2043,Roodt,Ruby Johnson,3,Used compound G in fractionation
2,MNP160003,90.8,SAF43-6676RH-02aa,2043,Ruby Johnson,30/12/2052,CD Experiment,23/12/2052,Fraction from flash,50.00,DCM:MeOH 2:1,Marine,SAF43-6676RH-02a,return,RES2043,Smith,Ruby Johnson,3,None
3,MNP160004,14.2,AMRU2459B-01aa,2045,Ruby Johnson,26/10/2054,RU Bioassay,19/10/2054,SPE fraction,0.50,DCM:MeOH 2:1,Microbial,AMRU2459B-01a,respond,RES2045,Roodt,Ruby Johnson,3,Used compound F in fractionation
4,MNP160005,3.3,SAF43-9299JR-01aa,2043,Ruby Johnson,05/06/2051,CD Experiment,29/05/2051,Fraction,0.25,MeOH,Marine,SAF43-9299JR-01a,international,RES2043,Roodt,Priscilla Evans,3,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,MNP219996,70.6,SAF43-10975RH-02aa,2043,Ruby Johnson,19/06/2050,Rhodes MS,12/06/2050,Fraction,1.00,MeOH,Marine,SAF43-10975RH-02a,quickly,RES2043,Roodt,Amanda Bowker,3,Used compound F in fractionation
59996,MNP219997,44.2,6-56*#1675R1-01ab,2046,Joshua Keener,06/01/2052,RU LC-MS & 1H-NMR,30/12/2051,SPE fraction,1.00,DCM:MeOH 2:1,Marine,6-56*#1675R1-01a,walk,RES2046,Smith,Ruby Johnson,3,70:30 setting
59997,MNP219998,88.3,MOZ42-2112JR-01ac,2042,Ruby Johnson,30/11/2053,RU LC/MS,23/11/2053,Fraction from flash,50.00,MeOH,Marine,MOZ42-2112JR-01a,raise,RES2042,Roodt,Amanda Bowker,3,None
59998,MNP219999,72.7,11-101*#3009R2-01ab,2046,Ruby Johnson,31/07/2054,CD Experiment,24/07/2054,SPE fraction,50.00,DCM:MeOH 2:1,Marine,11-101*#3009R2-01a,might,RES2046,Smith,Priscilla Evans,3,20:80 setting


Subsubsubfraction

In [218]:
sub3fraction_df = fractionThird_df.copy(deep=True)
sub3fraction_df = sub3fraction_df[["supplierID", "year", "assignedDate", "species", "permitNo", "project"]]
sub3fraction_df

,supplierID,year,assignedDate,species,permitNo,project
0,AMRU3638B-01ba,2047,23/12/2051,today,RES2047,Microbial
1,SAF43-6933JR-01ba,2043,11/04/2054,loss,RES2043,Marine
2,SAF43-6676RH-02aa,2043,30/12/2052,return,RES2043,Marine
3,AMRU2459B-01aa,2045,26/10/2054,respond,RES2045,Microbial
4,SAF43-9299JR-01aa,2043,05/06/2051,international,RES2043,Marine
...,...,...,...,...,...,...
59995,SAF43-10975RH-02aa,2043,19/06/2050,quickly,RES2043,Marine
59996,6-56*#1675R1-01ab,2046,06/01/2052,walk,RES2046,Marine
59997,MOZ42-2112JR-01ac,2042,30/11/2053,raise,RES2042,Marine
59998,11-101*#3009R2-01ab,2046,31/07/2054,might,RES2046,Marine


In [219]:
# Inserting assignedDate and submittedDate
sub3fraction_df.assignedDate = pd.to_datetime(sub3fraction_df.assignedDate, format='%d/%m/%Y')
sub3fraction_df.insert(3, 'submittedDate', sub3fraction_df['assignedDate'])
# Date submitted - do 3 weeks after previous assignedDate
sub3fraction_df['submittedDate'] = sub3fraction_df['submittedDate'] + timedelta(days=21)
sub3fraction_df['submittedDate'] = sub3fraction_df['submittedDate'].dt.strftime('%d/%m/%Y')
# Date assigned - do a week after submittedDate
sub3fraction_df['assignedDate'] = sub3fraction_df['assignedDate'] + timedelta(days=28)
sub3fraction_df['assignedDate'] = sub3fraction_df['assignedDate'].dt.strftime('%d/%m/%Y')
sub3fraction_df

,supplierID,year,assignedDate,submittedDate,species,permitNo,project
0,AMRU3638B-01ba,2047,20/01/2052,13/01/2052,today,RES2047,Microbial
1,SAF43-6933JR-01ba,2043,09/05/2054,02/05/2054,loss,RES2043,Marine
2,SAF43-6676RH-02aa,2043,27/01/2053,20/01/2053,return,RES2043,Marine
3,AMRU2459B-01aa,2045,23/11/2054,16/11/2054,respond,RES2045,Microbial
4,SAF43-9299JR-01aa,2043,03/07/2051,26/06/2051,international,RES2043,Marine
...,...,...,...,...,...,...,...
59995,SAF43-10975RH-02aa,2043,17/07/2050,10/07/2050,quickly,RES2043,Marine
59996,6-56*#1675R1-01ab,2046,03/02/2052,27/01/2052,walk,RES2046,Marine
59997,MOZ42-2112JR-01ac,2042,28/12/2053,21/12/2053,raise,RES2042,Marine
59998,11-101*#3009R2-01ab,2046,28/08/2054,21/08/2054,might,RES2046,Marine


In [220]:
fake = Faker()
Faker.seed(1232)
random.seed(1232)

class Provider(BaseProvider):
    submitters = ['Ruby Johnson', 'Amanda Bowker', 'Priscilla Evans']
    assigners = ['Ruby Johnson', 'Joshua Keener']
    destinations = ['RU Bioassay', 'Rhodes MS', 'CD Experiment', 'RU LC-MS & 1H-NMR', 'RU LC/MS']
    researchgroups = ['Smith', 'Roodt']
    solvents = ['MeOH', 'DCM:MeOH 2:1', 'DCM/MeOH']
    sampleTypes = ['Fraction', 'SPE fraction', 'Fraction from flash', 'HPLC Fraction']
    concentrations = [0.25, 0.1, 0.5, 1.0, 20.0, 50.0]
    comment = [None, None, 'Used compound Q in fractionation', 'Used compound R in fractionation', None, None, '10:90 setting', 'Combination setting']
    
    def submittedBy(self):
        return random.choice(self.submitters)

    def assignedBy(self):
        return random.choice(self.assigners)

    def destinationOfSample(self):
        return random.choice(self.destinations)

    def researchGroup(self):
        return random.choice(self.researchgroups)
    
    def solvent(self):
        return random.choice(self.solvents)
    
    def sampleType(self):
        return random.choice(self.sampleTypes)
    
    def concentration(self):
        return random.choice(self.concentrations)
    
    def comments(self):
        return random.choice(self.comment)

# Add the Provider to our faker object
fake.add_provider(Provider)

# First change any columns that must be the same for all fractions from a particular source
# Submitted by, research group, project, concentration, solvent, species, sample type, assigned by, submitted date and assigned date (but will be same either way), destination

def create_sub3fraction_data(fake, no_of_rows):
    columns = ['assignedBy', 'destinationOfSample', 'sampleType', 'researchGroup', 'submittedBy', 'solvent', 'concentration', 'comments']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

sub3fractionSubset_df = create_sub3fraction_data(fake, len(sub3fraction_df))
sub3fractionSubset_df

,assignedBy,destinationOfSample,sampleType,researchGroup,submittedBy,solvent,concentration,comments
0,Ruby Johnson,CD Experiment,HPLC Fraction,Roodt,Ruby Johnson,MeOH,0.50,None
1,Joshua Keener,RU LC/MS,Fraction from flash,Smith,Amanda Bowker,MeOH,0.25,10:90 setting
2,Joshua Keener,Rhodes MS,Fraction from flash,Smith,Priscilla Evans,DCM/MeOH,0.10,Used compound Q in fractionation
3,Ruby Johnson,Rhodes MS,Fraction,Smith,Priscilla Evans,MeOH,50.00,None
4,Ruby Johnson,CD Experiment,Fraction,Smith,Amanda Bowker,DCM/MeOH,1.00,None
...,...,...,...,...,...,...,...,...
59995,Joshua Keener,RU LC-MS & 1H-NMR,HPLC Fraction,Roodt,Amanda Bowker,DCM:MeOH 2:1,0.25,None
59996,Ruby Johnson,Rhodes MS,SPE fraction,Roodt,Amanda Bowker,DCM:MeOH 2:1,0.25,Used compound Q in fractionation
59997,Ruby Johnson,CD Experiment,Fraction,Smith,Ruby Johnson,DCM/MeOH,0.25,None
59998,Ruby Johnson,CD Experiment,Fraction,Smith,Amanda Bowker,DCM/MeOH,1.00,None


In [221]:
newSub3Fraction_df = pd.concat([sub3fraction_df, sub3fractionSubset_df], axis=1)
newSub3Fraction_df.rename(columns={'supplierID':'sourceID'}, inplace=True)
# Get columns in same order as MNP spreadsheet
orderCols = ['year', 'assignedBy', 'assignedDate', 'destinationOfSample', 'submittedDate', 'sampleType', 'concentration', 'solvent', 'project', 'sourceID', 'species', 'permitNo', 'researchGroup', 'submittedBy', 'comments']
newSub3Fraction_df = newSub3Fraction_df[orderCols]
newSub3Fraction_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2047,Ruby Johnson,20/01/2052,CD Experiment,13/01/2052,HPLC Fraction,0.50,MeOH,Microbial,AMRU3638B-01ba,today,RES2047,Roodt,Ruby Johnson,None
1,2043,Joshua Keener,09/05/2054,RU LC/MS,02/05/2054,Fraction from flash,0.25,MeOH,Marine,SAF43-6933JR-01ba,loss,RES2043,Smith,Amanda Bowker,10:90 setting
2,2043,Joshua Keener,27/01/2053,Rhodes MS,20/01/2053,Fraction from flash,0.10,DCM/MeOH,Marine,SAF43-6676RH-02aa,return,RES2043,Smith,Priscilla Evans,Used compound Q in fractionation
3,2045,Ruby Johnson,23/11/2054,Rhodes MS,16/11/2054,Fraction,50.00,MeOH,Microbial,AMRU2459B-01aa,respond,RES2045,Smith,Priscilla Evans,None
4,2043,Ruby Johnson,03/07/2051,CD Experiment,26/06/2051,Fraction,1.00,DCM/MeOH,Marine,SAF43-9299JR-01aa,international,RES2043,Smith,Amanda Bowker,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,2043,Joshua Keener,17/07/2050,RU LC-MS & 1H-NMR,10/07/2050,HPLC Fraction,0.25,DCM:MeOH 2:1,Marine,SAF43-10975RH-02aa,quickly,RES2043,Roodt,Amanda Bowker,None
59996,2046,Ruby Johnson,03/02/2052,Rhodes MS,27/01/2052,SPE fraction,0.25,DCM:MeOH 2:1,Marine,6-56*#1675R1-01ab,walk,RES2046,Roodt,Amanda Bowker,Used compound Q in fractionation
59997,2042,Ruby Johnson,28/12/2053,CD Experiment,21/12/2053,Fraction,0.25,DCM/MeOH,Marine,MOZ42-2112JR-01ac,raise,RES2042,Smith,Ruby Johnson,None
59998,2046,Ruby Johnson,28/08/2054,CD Experiment,21/08/2054,Fraction,1.00,DCM/MeOH,Marine,11-101*#3009R2-01ab,might,RES2046,Smith,Amanda Bowker,None


In [222]:
numSampled4 = 20000
sub3fraction1_df = newSub3Fraction_df.sample(n = numSampled4, random_state=1231, ignore_index=True)
sub3fraction2_df = newSub3Fraction_df.sample(n = numSampled4, random_state=1232, ignore_index=True)
sub3fraction3_df = newSub3Fraction_df.sample(n = numSampled4, random_state=1233, ignore_index=True)
sub3fraction4_df = newSub3Fraction_df.sample(n = numSampled4, random_state=1234, ignore_index=True)
sub3fraction1_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2046,Ruby Johnson,14/08/2051,CD Experiment,07/08/2051,HPLC Fraction,0.5,MeOH,Marine,9-87*#2587R3-03ba,according,RES2046,Smith,Amanda Bowker,None
1,2042,Joshua Keener,23/10/2050,RU LC/MS,16/10/2050,SPE fraction,20.0,MeOH,Marine,MOZ42-6552RJ-02bb,several,RES2042,Roodt,Priscilla Evans,Used compound Q in fractionation
2,2046,Joshua Keener,05/10/2052,RU Bioassay,28/09/2052,SPE fraction,50.0,DCM:MeOH 2:1,Marine,10-96*#2861R2-01ab,network,RES2046,Smith,Priscilla Evans,None
3,2045,Joshua Keener,30/09/2053,Rhodes MS,23/09/2053,HPLC Fraction,50.0,DCM:MeOH 2:1,Microbial,AMRU6133C-01aa,miss,RES2045,Smith,Amanda Bowker,10:90 setting
4,2042,Ruby Johnson,24/11/2052,RU Bioassay,17/11/2052,Fraction from flash,20.0,DCM/MeOH,Marine,MOZ42-4483JR-01bb,writer,RES2042,Roodt,Priscilla Evans,Used compound R in fractionation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2046,Joshua Keener,28/01/2052,RU Bioassay,21/01/2052,Fraction from flash,50.0,MeOH,Marine,3-28*#826R1-02bb,game,RES2046,Roodt,Amanda Bowker,None
19996,2041,Ruby Johnson,17/03/2052,Rhodes MS,10/03/2052,SPE fraction,50.0,DCM:MeOH 2:1,Marine,TIC2041-5240RJ-02ba,practice,RES2041,Roodt,Amanda Bowker,None
19997,2042,Ruby Johnson,15/07/2050,Rhodes MS,08/07/2050,SPE fraction,1.0,DCM/MeOH,Marine,MOZ42-3789RH-01ab,market,RES2042,Roodt,Ruby Johnson,Combination setting
19998,2045,Joshua Keener,10/06/2053,RU LC-MS & 1H-NMR,03/06/2053,SPE fraction,50.0,DCM:MeOH 2:1,Microbial,AMRU5571C-02bb,management,RES2045,Smith,Ruby Johnson,None


In [223]:
fractionFourth_df = pd.concat([sub3fraction1_df, sub3fraction2_df, sub3fraction3_df, sub3fraction4_df], ignore_index=True)
fractionFourth_df

,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,2046,Ruby Johnson,14/08/2051,CD Experiment,07/08/2051,HPLC Fraction,0.50,MeOH,Marine,9-87*#2587R3-03ba,according,RES2046,Smith,Amanda Bowker,None
1,2042,Joshua Keener,23/10/2050,RU LC/MS,16/10/2050,SPE fraction,20.00,MeOH,Marine,MOZ42-6552RJ-02bb,several,RES2042,Roodt,Priscilla Evans,Used compound Q in fractionation
2,2046,Joshua Keener,05/10/2052,RU Bioassay,28/09/2052,SPE fraction,50.00,DCM:MeOH 2:1,Marine,10-96*#2861R2-01ab,network,RES2046,Smith,Priscilla Evans,None
3,2045,Joshua Keener,30/09/2053,Rhodes MS,23/09/2053,HPLC Fraction,50.00,DCM:MeOH 2:1,Microbial,AMRU6133C-01aa,miss,RES2045,Smith,Amanda Bowker,10:90 setting
4,2042,Ruby Johnson,24/11/2052,RU Bioassay,17/11/2052,Fraction from flash,20.00,DCM/MeOH,Marine,MOZ42-4483JR-01bb,writer,RES2042,Roodt,Priscilla Evans,Used compound R in fractionation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,2046,Joshua Keener,30/09/2050,RU LC-MS & 1H-NMR,23/09/2050,SPE fraction,50.00,DCM/MeOH,Marine,1-10*#288R3-01ba,election,RES2046,Smith,Priscilla Evans,None
79996,2041,Ruby Johnson,24/08/2052,Rhodes MS,17/08/2052,Fraction,50.00,DCM/MeOH,Marine,TIC2041-707RH-02ba,industry,RES2041,Smith,Ruby Johnson,None
79997,2043,Ruby Johnson,01/11/2052,CD Experiment,25/10/2052,HPLC Fraction,1.00,DCM:MeOH 2:1,Marine,SAF43-10779RH-01cb,score,RES2043,Roodt,Ruby Johnson,Combination setting
79998,2045,Joshua Keener,23/12/2051,RU LC/MS,16/12/2051,Fraction from flash,50.00,DCM:MeOH 2:1,Microbial,AMRU4767C-01ac,in,RES2045,Roodt,Priscilla Evans,10:90 setting


In [224]:
# Parts that are different per subsubsubfraction:
# Amount available
fake = Faker()
Faker.seed(1231)
random.seed(1231)

class Provider(BaseProvider):
    def amountAvailable(self):
        return (round (random.uniform(0.0, 1.0)*100, 1))
    
fake.add_provider(Provider)

def create_amountSub_data(fake, no_of_rows):
    columns = ['amountAvailable']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

amount3Sub_df = create_amountSub_data(fake, len(fractionFourth_df))

fractionFourth_df = pd.concat([amount3Sub_df, fractionFourth_df], axis=1)
fractionFourth_df

,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,37.9,2046,Ruby Johnson,14/08/2051,CD Experiment,07/08/2051,HPLC Fraction,0.50,MeOH,Marine,9-87*#2587R3-03ba,according,RES2046,Smith,Amanda Bowker,None
1,25.7,2042,Joshua Keener,23/10/2050,RU LC/MS,16/10/2050,SPE fraction,20.00,MeOH,Marine,MOZ42-6552RJ-02bb,several,RES2042,Roodt,Priscilla Evans,Used compound Q in fractionation
2,41.6,2046,Joshua Keener,05/10/2052,RU Bioassay,28/09/2052,SPE fraction,50.00,DCM:MeOH 2:1,Marine,10-96*#2861R2-01ab,network,RES2046,Smith,Priscilla Evans,None
3,39.4,2045,Joshua Keener,30/09/2053,Rhodes MS,23/09/2053,HPLC Fraction,50.00,DCM:MeOH 2:1,Microbial,AMRU6133C-01aa,miss,RES2045,Smith,Amanda Bowker,10:90 setting
4,51.5,2042,Ruby Johnson,24/11/2052,RU Bioassay,17/11/2052,Fraction from flash,20.00,DCM/MeOH,Marine,MOZ42-4483JR-01bb,writer,RES2042,Roodt,Priscilla Evans,Used compound R in fractionation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,85.1,2046,Joshua Keener,30/09/2050,RU LC-MS & 1H-NMR,23/09/2050,SPE fraction,50.00,DCM/MeOH,Marine,1-10*#288R3-01ba,election,RES2046,Smith,Priscilla Evans,None
79996,46.5,2041,Ruby Johnson,24/08/2052,Rhodes MS,17/08/2052,Fraction,50.00,DCM/MeOH,Marine,TIC2041-707RH-02ba,industry,RES2041,Smith,Ruby Johnson,None
79997,21.4,2043,Ruby Johnson,01/11/2052,CD Experiment,25/10/2052,HPLC Fraction,1.00,DCM:MeOH 2:1,Marine,SAF43-10779RH-01cb,score,RES2043,Roodt,Ruby Johnson,Combination setting
79998,22.7,2045,Joshua Keener,23/12/2051,RU LC/MS,16/12/2051,Fraction from flash,50.00,DCM:MeOH 2:1,Microbial,AMRU4767C-01ac,in,RES2045,Roodt,Priscilla Evans,10:90 setting


In [225]:
# Make SupplierIDs - same as sourceID, except have -01, -02 etc for initial fractions (will be a,b,c for rest of subfractions)
# 
sourceSubSeries = fractionFourth_df['sourceID']
listSubSupplier = []
subsupplier_freq = {}

for index, value in sourceSubSeries.items():
    if (value in subsupplier_freq):
        subsupplier_freq[value] += 1
    else:
        subsupplier_freq[value] = 1
    listSubSupplier.append(value + str(chr(subsupplier_freq[value] + 96)))

subsupplierSeries = pd.Series(listSubSupplier)
#print(supplierSeries)

fractionFourth_df.insert(1, 'supplierID', subsupplierSeries)
fractionFourth_df

,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,37.9,9-87*#2587R3-03baa,2046,Ruby Johnson,14/08/2051,CD Experiment,07/08/2051,HPLC Fraction,0.50,MeOH,Marine,9-87*#2587R3-03ba,according,RES2046,Smith,Amanda Bowker,None
1,25.7,MOZ42-6552RJ-02bba,2042,Joshua Keener,23/10/2050,RU LC/MS,16/10/2050,SPE fraction,20.00,MeOH,Marine,MOZ42-6552RJ-02bb,several,RES2042,Roodt,Priscilla Evans,Used compound Q in fractionation
2,41.6,10-96*#2861R2-01aba,2046,Joshua Keener,05/10/2052,RU Bioassay,28/09/2052,SPE fraction,50.00,DCM:MeOH 2:1,Marine,10-96*#2861R2-01ab,network,RES2046,Smith,Priscilla Evans,None
3,39.4,AMRU6133C-01aaa,2045,Joshua Keener,30/09/2053,Rhodes MS,23/09/2053,HPLC Fraction,50.00,DCM:MeOH 2:1,Microbial,AMRU6133C-01aa,miss,RES2045,Smith,Amanda Bowker,10:90 setting
4,51.5,MOZ42-4483JR-01bba,2042,Ruby Johnson,24/11/2052,RU Bioassay,17/11/2052,Fraction from flash,20.00,DCM/MeOH,Marine,MOZ42-4483JR-01bb,writer,RES2042,Roodt,Priscilla Evans,Used compound R in fractionation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,85.1,1-10*#288R3-01bac,2046,Joshua Keener,30/09/2050,RU LC-MS & 1H-NMR,23/09/2050,SPE fraction,50.00,DCM/MeOH,Marine,1-10*#288R3-01ba,election,RES2046,Smith,Priscilla Evans,None
79996,46.5,TIC2041-707RH-02baa,2041,Ruby Johnson,24/08/2052,Rhodes MS,17/08/2052,Fraction,50.00,DCM/MeOH,Marine,TIC2041-707RH-02ba,industry,RES2041,Smith,Ruby Johnson,None
79997,21.4,SAF43-10779RH-01cbc,2043,Ruby Johnson,01/11/2052,CD Experiment,25/10/2052,HPLC Fraction,1.00,DCM:MeOH 2:1,Marine,SAF43-10779RH-01cb,score,RES2043,Roodt,Ruby Johnson,Combination setting
79998,22.7,AMRU4767C-01aca,2045,Joshua Keener,23/12/2051,RU LC/MS,16/12/2051,Fraction from flash,50.00,DCM:MeOH 2:1,Microbial,AMRU4767C-01ac,in,RES2045,Roodt,Priscilla Evans,10:90 setting


In [226]:
#Get 40000 records that will be pure, rather than just fractions
pureSecond_df = fractionFourth_df.sample(frac = 0.5, random_state=1233)
pureSecond_df

,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
53200,69.1,AMRU4621B-01aaa,2045,Joshua Keener,16/12/2052,RU LC/MS,09/12/2052,HPLC Fraction,0.50,DCM:MeOH 2:1,Microbial,AMRU4621B-01aa,occur,RES2045,Roodt,Ruby Johnson,10:90 setting
63377,64.4,MOZ42-10083JR-01caa,2042,Ruby Johnson,08/03/2054,RU LC-MS & 1H-NMR,01/03/2054,HPLC Fraction,0.10,DCM:MeOH 2:1,Marine,MOZ42-10083JR-01ca,conference,RES2042,Roodt,Priscilla Evans,10:90 setting
22630,58.2,TIC2041-3836JR-01aca,2041,Ruby Johnson,19/07/2052,RU LC-MS & 1H-NMR,12/07/2052,Fraction from flash,0.25,MeOH,Marine,TIC2041-3836JR-01ac,democratic,RES2041,Roodt,Amanda Bowker,None
16510,52.4,TIC2041-9785RH-01aaa,2041,Joshua Keener,08/04/2055,RU LC-MS & 1H-NMR,01/04/2055,Fraction,0.10,MeOH,Marine,TIC2041-9785RH-01aa,until,RES2041,Smith,Ruby Johnson,10:90 setting
9850,26.6,SAF43-4812RH-01bda,2043,Ruby Johnson,24/01/2051,RU LC-MS & 1H-NMR,17/01/2051,Fraction from flash,20.00,DCM:MeOH 2:1,Marine,SAF43-4812RH-01bd,plant,RES2043,Roodt,Amanda Bowker,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11779,59.2,TIC2041-434RH-01aba,2041,Ruby Johnson,19/11/2052,CD Experiment,12/11/2052,HPLC Fraction,50.00,MeOH,Marine,TIC2041-434RH-01ab,provide,RES2041,Smith,Ruby Johnson,Used compound R in fractionation
6932,28.6,TIC2041-10200RH-01aba,2041,Joshua Keener,23/06/2054,RU Bioassay,16/06/2054,Fraction,0.25,DCM/MeOH,Marine,TIC2041-10200RH-01ab,evening,RES2041,Smith,Amanda Bowker,Used compound R in fractionation
23579,23.9,AMRU4631B-01aaa,2045,Ruby Johnson,13/01/2055,RU Bioassay,06/01/2055,HPLC Fraction,0.10,MeOH,Microbial,AMRU4631B-01aa,people,RES2045,Roodt,Ruby Johnson,10:90 setting
53448,22.9,AMRU2153C-01baa,2045,Joshua Keener,28/01/2053,CD Experiment,21/01/2053,SPE fraction,0.10,DCM:MeOH 2:1,Microbial,AMRU2153C-01ba,above,RES2045,Roodt,Priscilla Evans,None


In [227]:
fractionFourth_df = fractionFourth_df.drop(pureSecond_df.index)
fractionFourth_df.reset_index(drop=True, inplace=True)
fractionFourth_df

,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,37.9,9-87*#2587R3-03baa,2046,Ruby Johnson,14/08/2051,CD Experiment,07/08/2051,HPLC Fraction,0.5,MeOH,Marine,9-87*#2587R3-03ba,according,RES2046,Smith,Amanda Bowker,None
1,25.7,MOZ42-6552RJ-02bba,2042,Joshua Keener,23/10/2050,RU LC/MS,16/10/2050,SPE fraction,20.0,MeOH,Marine,MOZ42-6552RJ-02bb,several,RES2042,Roodt,Priscilla Evans,Used compound Q in fractionation
2,41.6,10-96*#2861R2-01aba,2046,Joshua Keener,05/10/2052,RU Bioassay,28/09/2052,SPE fraction,50.0,DCM:MeOH 2:1,Marine,10-96*#2861R2-01ab,network,RES2046,Smith,Priscilla Evans,None
3,39.4,AMRU6133C-01aaa,2045,Joshua Keener,30/09/2053,Rhodes MS,23/09/2053,HPLC Fraction,50.0,DCM:MeOH 2:1,Microbial,AMRU6133C-01aa,miss,RES2045,Smith,Amanda Bowker,10:90 setting
4,51.5,MOZ42-4483JR-01bba,2042,Ruby Johnson,24/11/2052,RU Bioassay,17/11/2052,Fraction from flash,20.0,DCM/MeOH,Marine,MOZ42-4483JR-01bb,writer,RES2042,Roodt,Priscilla Evans,Used compound R in fractionation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,71.4,AMRU4335B-01aaa,2045,Joshua Keener,09/06/2053,CD Experiment,02/06/2053,HPLC Fraction,50.0,MeOH,Microbial,AMRU4335B-01aa,perform,RES2045,Roodt,Priscilla Evans,Combination setting
39996,7.9,SAF43-4656JR-01aac,2043,Joshua Keener,08/07/2052,RU Bioassay,01/07/2052,Fraction from flash,0.1,DCM:MeOH 2:1,Marine,SAF43-4656JR-01aa,body,RES2043,Smith,Priscilla Evans,None
39997,72.9,MOZ42-1272RJ-01abc,2042,Ruby Johnson,20/10/2050,Rhodes MS,13/10/2050,SPE fraction,20.0,DCM/MeOH,Marine,MOZ42-1272RJ-01ab,east,RES2042,Smith,Ruby Johnson,Used compound R in fractionation
39998,89.0,MOZ42-6381RH-01baa,2042,Joshua Keener,03/09/2051,RU Bioassay,27/08/2051,HPLC Fraction,50.0,DCM:MeOH 2:1,Marine,MOZ42-6381RH-01ba,describe,RES2042,Roodt,Amanda Bowker,Used compound R in fractionation


In [228]:
offset = len(fractionFirst_df) + len(fractionSecond_df) + len(fractionThird_df)
fractionFourth_df.insert(0, 'mnpID', range(offset + 1, 1 + offset + len(fractionFourth_df)))
fractionFourth_df['mnpID'] = 'MNP' + fractionFourth_df['mnpID'].astype(str).str.zfill(5)
# Include level (4)
fractionFourth_df.insert(17, 'level', 4)
fractionFourth_df

,mnpID,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,level,comments
0,MNP220001,37.9,9-87*#2587R3-03baa,2046,Ruby Johnson,14/08/2051,CD Experiment,07/08/2051,HPLC Fraction,0.5,MeOH,Marine,9-87*#2587R3-03ba,according,RES2046,Smith,Amanda Bowker,4,None
1,MNP220002,25.7,MOZ42-6552RJ-02bba,2042,Joshua Keener,23/10/2050,RU LC/MS,16/10/2050,SPE fraction,20.0,MeOH,Marine,MOZ42-6552RJ-02bb,several,RES2042,Roodt,Priscilla Evans,4,Used compound Q in fractionation
2,MNP220003,41.6,10-96*#2861R2-01aba,2046,Joshua Keener,05/10/2052,RU Bioassay,28/09/2052,SPE fraction,50.0,DCM:MeOH 2:1,Marine,10-96*#2861R2-01ab,network,RES2046,Smith,Priscilla Evans,4,None
3,MNP220004,39.4,AMRU6133C-01aaa,2045,Joshua Keener,30/09/2053,Rhodes MS,23/09/2053,HPLC Fraction,50.0,DCM:MeOH 2:1,Microbial,AMRU6133C-01aa,miss,RES2045,Smith,Amanda Bowker,4,10:90 setting
4,MNP220005,51.5,MOZ42-4483JR-01bba,2042,Ruby Johnson,24/11/2052,RU Bioassay,17/11/2052,Fraction from flash,20.0,DCM/MeOH,Marine,MOZ42-4483JR-01bb,writer,RES2042,Roodt,Priscilla Evans,4,Used compound R in fractionation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,MNP259996,71.4,AMRU4335B-01aaa,2045,Joshua Keener,09/06/2053,CD Experiment,02/06/2053,HPLC Fraction,50.0,MeOH,Microbial,AMRU4335B-01aa,perform,RES2045,Roodt,Priscilla Evans,4,Combination setting
39996,MNP259997,7.9,SAF43-4656JR-01aac,2043,Joshua Keener,08/07/2052,RU Bioassay,01/07/2052,Fraction from flash,0.1,DCM:MeOH 2:1,Marine,SAF43-4656JR-01aa,body,RES2043,Smith,Priscilla Evans,4,None
39997,MNP259998,72.9,MOZ42-1272RJ-01abc,2042,Ruby Johnson,20/10/2050,Rhodes MS,13/10/2050,SPE fraction,20.0,DCM/MeOH,Marine,MOZ42-1272RJ-01ab,east,RES2042,Smith,Ruby Johnson,4,Used compound R in fractionation
39998,MNP259999,89.0,MOZ42-6381RH-01baa,2042,Joshua Keener,03/09/2051,RU Bioassay,27/08/2051,HPLC Fraction,50.0,DCM:MeOH 2:1,Marine,MOZ42-6381RH-01ba,describe,RES2042,Roodt,Amanda Bowker,4,Used compound R in fractionation


#### Pure compounds

In [229]:
# Get all pure fractions together
pureComp_df = pd.concat([pureFirst_df, pureSecond_df], ignore_index=True)
pureComp_df

,amountAvailable,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,comments
0,98.2,TIC2041-1065RH-02aa,2041,Joshua Keener,10/08/2054,Rhodes MS,03/08/2054,Fraction from flash,1.00,DCM:MeOH 2:1,Marine,TIC2041-1065RH-02a,church,RES2041,Smith,Ruby Johnson,70:30 setting
1,42.0,SAF43-258RJ-01aa,2043,Joshua Keener,16/08/2051,RU LC-MS & 1H-NMR,09/08/2051,SPE fraction,0.25,DCM:MeOH 2:1,Marine,SAF43-258RJ-01a,society,RES2043,Smith,Amanda Bowker,None
2,92.2,SAF43-2536JR-02ba,2043,Joshua Keener,21/05/2050,Rhodes MS,14/05/2050,Fraction from flash,1.00,DCM/MeOH,Marine,SAF43-2536JR-02b,something,RES2043,Roodt,Priscilla Evans,Used compound F in fractionation
3,30.0,TIC2041-4160JR-02ab,2041,Ruby Johnson,24/06/2053,RU LC-MS & 1H-NMR,17/06/2053,Fraction from flash,20.00,MeOH,Marine,TIC2041-4160JR-02a,behind,RES2041,Smith,Ruby Johnson,None
4,12.6,AMRU3724C-01aa,2047,Ruby Johnson,20/10/2054,RU LC-MS & 1H-NMR,13/10/2054,Fraction,50.00,DCM:MeOH 2:1,Microbial,AMRU3724C-01a,main,RES2047,Roodt,Priscilla Evans,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,59.2,TIC2041-434RH-01aba,2041,Ruby Johnson,19/11/2052,CD Experiment,12/11/2052,HPLC Fraction,50.00,MeOH,Marine,TIC2041-434RH-01ab,provide,RES2041,Smith,Ruby Johnson,Used compound R in fractionation
59996,28.6,TIC2041-10200RH-01aba,2041,Joshua Keener,23/06/2054,RU Bioassay,16/06/2054,Fraction,0.25,DCM/MeOH,Marine,TIC2041-10200RH-01ab,evening,RES2041,Smith,Amanda Bowker,Used compound R in fractionation
59997,23.9,AMRU4631B-01aaa,2045,Ruby Johnson,13/01/2055,RU Bioassay,06/01/2055,HPLC Fraction,0.10,MeOH,Microbial,AMRU4631B-01aa,people,RES2045,Roodt,Ruby Johnson,10:90 setting
59998,22.9,AMRU2153C-01baa,2045,Joshua Keener,28/01/2053,CD Experiment,21/01/2053,SPE fraction,0.10,DCM:MeOH 2:1,Microbial,AMRU2153C-01ba,above,RES2045,Roodt,Priscilla Evans,None


In [230]:
# Keep from fractions:
# Assignedby, dateAssigned, dateSubmitted, amountAvailable, sourceID, year, species, permit, submittedby, solvent, sampleType

pureComp_df = pureComp_df[["supplierID", "year", "assignedBy", "assignedDate", "submittedDate", "amountAvailable", "sourceID", "species", "permitNo", "submittedBy", "solvent", "sampleType", "comments"]]
pureComp_df

,supplierID,year,assignedBy,assignedDate,submittedDate,amountAvailable,sourceID,species,permitNo,submittedBy,solvent,sampleType,comments
0,TIC2041-1065RH-02aa,2041,Joshua Keener,10/08/2054,03/08/2054,98.2,TIC2041-1065RH-02a,church,RES2041,Ruby Johnson,DCM:MeOH 2:1,Fraction from flash,70:30 setting
1,SAF43-258RJ-01aa,2043,Joshua Keener,16/08/2051,09/08/2051,42.0,SAF43-258RJ-01a,society,RES2043,Amanda Bowker,DCM:MeOH 2:1,SPE fraction,None
2,SAF43-2536JR-02ba,2043,Joshua Keener,21/05/2050,14/05/2050,92.2,SAF43-2536JR-02b,something,RES2043,Priscilla Evans,DCM/MeOH,Fraction from flash,Used compound F in fractionation
3,TIC2041-4160JR-02ab,2041,Ruby Johnson,24/06/2053,17/06/2053,30.0,TIC2041-4160JR-02a,behind,RES2041,Ruby Johnson,MeOH,Fraction from flash,None
4,AMRU3724C-01aa,2047,Ruby Johnson,20/10/2054,13/10/2054,12.6,AMRU3724C-01a,main,RES2047,Priscilla Evans,DCM:MeOH 2:1,Fraction,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,TIC2041-434RH-01aba,2041,Ruby Johnson,19/11/2052,12/11/2052,59.2,TIC2041-434RH-01ab,provide,RES2041,Ruby Johnson,MeOH,HPLC Fraction,Used compound R in fractionation
59996,TIC2041-10200RH-01aba,2041,Joshua Keener,23/06/2054,16/06/2054,28.6,TIC2041-10200RH-01ab,evening,RES2041,Amanda Bowker,DCM/MeOH,Fraction,Used compound R in fractionation
59997,AMRU4631B-01aaa,2045,Ruby Johnson,13/01/2055,06/01/2055,23.9,AMRU4631B-01aa,people,RES2045,Ruby Johnson,MeOH,HPLC Fraction,10:90 setting
59998,AMRU2153C-01baa,2045,Joshua Keener,28/01/2053,21/01/2053,22.9,AMRU2153C-01ba,above,RES2045,Priscilla Evans,DCM:MeOH 2:1,SPE fraction,None


In [231]:
fake = Faker()
Faker.seed(1233)
random.seed(1233)

class Provider(BaseProvider):
    destinations = ['Rhodes Bioassay', 'Rhodes MS', 'Rhodes Bioassay/Rhodes MS', 'Rhodes Bioassay, PK assay', 'Rhodes MS/NMR', 'Membrane Disruption assay', 'Rhodes LC/MS + NMR & Bioassay']
    binaryChoice = ['Y', 'N']
    researchgroups = ['Smith', 'Howard', 'Ackerman', 'Leer']
    solvents = ['DMSO', 'aq.MeOH, MeOH', 'DCM: MeOH 2:1']
    stereos = ['InterestPC1', 'InterestPC2', 'InterestPC3', 'InterestPC4', 'InterestPC5', 
               'InterestPC6', 'InterestPC7', 'InterestPC8', 'InterestPC9', 'InterestPC10','N/A']
    existingLit = ['Y', 'https://doi.org/comp1', 'https://doi.org/comp2', 'https://doi.org/comp3', 'https://doi.org/comp4', 'https://doi.org/comp5', 'https://doi.org/comp6', 'https://doi.org/comp7']

    def destinationOfSample(self):
        return random.choice(self.destinations)

    def synthesisPotential(self):
        return random.choice(self.binaryChoice)

    def pure(self):
        return "Y"

    def ms(self):
        return random.choice(self.binaryChoice)
    
    def nmr(self):
        return random.choice(self.binaryChoice)
    
    def sourceType(self):
        return "MNP"

    def researchGroup(self):
        return random.choice(self.researchgroups)

    def solubility(self):
        return random.choice(self.solvents)

    def stereoComments(self):
        return random.choice(self.stereos)

    def smileStructure(self):
        return fake.word()

    def mw(self):
        return (round (random.uniform(0.0, 1.0)*400, 1))

    def additionalMetadata(self):
        return random.choice(self.binaryChoice)

    def existingPatent(self):
        return "Y"

    def existingLiterature(self):
        return random.choice(self.existingLit)

# Add the Provider to our faker object
fake.add_provider(Provider)

# Change:
# AMR ID, supplierID, destination, synthesis, sourceType, pure, ms, nmr, researchGroup, solubility, 
# stereoComments, SMILE, mw, additionalMetadata, existingPatent, existingLiterature

def create_pure_data(fake, no_of_rows):
    columns = ['destinationOfSample', 'synthesisPotential', 'pure', 'ms', 'nmr', 'sourceType',
                'researchGroup', 'solubility', 'stereoComments', 'smileStructure', 'mw', 'additionalMetadata', 'existingPatent', 'existingLiterature']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

pureRest_df = create_pure_data(fake, len(pureComp_df))
pureRest_df

,destinationOfSample,synthesisPotential,pure,ms,nmr,sourceType,researchGroup,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature
0,Rhodes Bioassay/Rhodes MS,Y,Y,Y,N,MNP,Smith,DCM: MeOH 2:1,InterestPC6,former,78.1,N,Y,https://doi.org/comp7
1,Rhodes Bioassay/Rhodes MS,N,Y,N,N,MNP,Ackerman,"aq.MeOH, MeOH",InterestPC2,fish,284.5,Y,Y,Y
2,Rhodes MS/NMR,Y,Y,Y,Y,MNP,Smith,"aq.MeOH, MeOH",InterestPC1,Mr,190.5,N,Y,https://doi.org/comp2
3,Rhodes Bioassay/Rhodes MS,N,Y,N,Y,MNP,Ackerman,DMSO,InterestPC8,five,308.4,Y,Y,https://doi.org/comp7
4,Rhodes MS,N,Y,N,N,MNP,Leer,DCM: MeOH 2:1,InterestPC3,crime,346.5,N,Y,https://doi.org/comp6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,Rhodes Bioassay/Rhodes MS,Y,Y,N,Y,MNP,Howard,DMSO,InterestPC7,both,6.7,Y,Y,https://doi.org/comp7
59996,Rhodes MS/NMR,N,Y,Y,N,MNP,Ackerman,"aq.MeOH, MeOH",InterestPC2,lose,163.2,N,Y,https://doi.org/comp4
59997,Rhodes LC/MS + NMR & Bioassay,Y,Y,N,Y,MNP,Leer,DCM: MeOH 2:1,InterestPC8,gas,60.2,Y,Y,https://doi.org/comp1
59998,Rhodes MS,Y,Y,N,Y,MNP,Howard,"aq.MeOH, MeOH",InterestPC8,sound,310.4,N,Y,https://doi.org/comp5


In [232]:
# Concatenating the pure compound columns
pure_df = pd.concat([pureComp_df, pureRest_df], axis=1)
orderPure = ['supplierID', 'year', 'assignedBy', 'assignedDate', 'destinationOfSample', 'submittedDate', 'amountAvailable', 'synthesisPotential', 'sourceID', 'sourceType', 'species', 'permitNo', 'pure', 'ms', 'nmr', 'researchGroup', 'submittedBy', 'solubility', 'stereoComments', 'smileStructure', 'mw', 'additionalMetadata', 'existingPatent', 'existingLiterature', 'solvent', 'sampleType', 'comments']
pure_df = pure_df[orderPure]
pure_df

,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,sourceType,...,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent,sampleType,comments
0,TIC2041-1065RH-02aa,2041,Joshua Keener,10/08/2054,Rhodes Bioassay/Rhodes MS,03/08/2054,98.2,Y,TIC2041-1065RH-02a,MNP,...,DCM: MeOH 2:1,InterestPC6,former,78.1,N,Y,https://doi.org/comp7,DCM:MeOH 2:1,Fraction from flash,70:30 setting
1,SAF43-258RJ-01aa,2043,Joshua Keener,16/08/2051,Rhodes Bioassay/Rhodes MS,09/08/2051,42.0,N,SAF43-258RJ-01a,MNP,...,"aq.MeOH, MeOH",InterestPC2,fish,284.5,Y,Y,Y,DCM:MeOH 2:1,SPE fraction,None
2,SAF43-2536JR-02ba,2043,Joshua Keener,21/05/2050,Rhodes MS/NMR,14/05/2050,92.2,Y,SAF43-2536JR-02b,MNP,...,"aq.MeOH, MeOH",InterestPC1,Mr,190.5,N,Y,https://doi.org/comp2,DCM/MeOH,Fraction from flash,Used compound F in fractionation
3,TIC2041-4160JR-02ab,2041,Ruby Johnson,24/06/2053,Rhodes Bioassay/Rhodes MS,17/06/2053,30.0,N,TIC2041-4160JR-02a,MNP,...,DMSO,InterestPC8,five,308.4,Y,Y,https://doi.org/comp7,MeOH,Fraction from flash,None
4,AMRU3724C-01aa,2047,Ruby Johnson,20/10/2054,Rhodes MS,13/10/2054,12.6,N,AMRU3724C-01a,MNP,...,DCM: MeOH 2:1,InterestPC3,crime,346.5,N,Y,https://doi.org/comp6,DCM:MeOH 2:1,Fraction,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,TIC2041-434RH-01aba,2041,Ruby Johnson,19/11/2052,Rhodes Bioassay/Rhodes MS,12/11/2052,59.2,Y,TIC2041-434RH-01ab,MNP,...,DMSO,InterestPC7,both,6.7,Y,Y,https://doi.org/comp7,MeOH,HPLC Fraction,Used compound R in fractionation
59996,TIC2041-10200RH-01aba,2041,Joshua Keener,23/06/2054,Rhodes MS/NMR,16/06/2054,28.6,N,TIC2041-10200RH-01ab,MNP,...,"aq.MeOH, MeOH",InterestPC2,lose,163.2,N,Y,https://doi.org/comp4,DCM/MeOH,Fraction,Used compound R in fractionation
59997,AMRU4631B-01aaa,2045,Ruby Johnson,13/01/2055,Rhodes LC/MS + NMR & Bioassay,06/01/2055,23.9,Y,AMRU4631B-01aa,MNP,...,DCM: MeOH 2:1,InterestPC8,gas,60.2,Y,Y,https://doi.org/comp1,MeOH,HPLC Fraction,10:90 setting
59998,AMRU2153C-01baa,2045,Joshua Keener,28/01/2053,Rhodes MS,21/01/2053,22.9,Y,AMRU2153C-01ba,MNP,...,"aq.MeOH, MeOH",InterestPC8,sound,310.4,N,Y,https://doi.org/comp5,DCM:MeOH 2:1,SPE fraction,None


In [233]:
# Still to do: AMR ID, supplierID, existingPatent, existingLiterature
# Make AMR IDs
pureOffset = len(original_df)
pure_df.insert(0, 'amrID', range(pureOffset + 1, 1 + pureOffset + len(pure_df)))
pure_df['amrID'] = 'AMR' + pure_df['amrID'].astype(str).str.zfill(5)
pure_df

,amrID,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,...,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent,sampleType,comments
0,AMR100001,TIC2041-1065RH-02aa,2041,Joshua Keener,10/08/2054,Rhodes Bioassay/Rhodes MS,03/08/2054,98.2,Y,TIC2041-1065RH-02a,...,DCM: MeOH 2:1,InterestPC6,former,78.1,N,Y,https://doi.org/comp7,DCM:MeOH 2:1,Fraction from flash,70:30 setting
1,AMR100002,SAF43-258RJ-01aa,2043,Joshua Keener,16/08/2051,Rhodes Bioassay/Rhodes MS,09/08/2051,42.0,N,SAF43-258RJ-01a,...,"aq.MeOH, MeOH",InterestPC2,fish,284.5,Y,Y,Y,DCM:MeOH 2:1,SPE fraction,None
2,AMR100003,SAF43-2536JR-02ba,2043,Joshua Keener,21/05/2050,Rhodes MS/NMR,14/05/2050,92.2,Y,SAF43-2536JR-02b,...,"aq.MeOH, MeOH",InterestPC1,Mr,190.5,N,Y,https://doi.org/comp2,DCM/MeOH,Fraction from flash,Used compound F in fractionation
3,AMR100004,TIC2041-4160JR-02ab,2041,Ruby Johnson,24/06/2053,Rhodes Bioassay/Rhodes MS,17/06/2053,30.0,N,TIC2041-4160JR-02a,...,DMSO,InterestPC8,five,308.4,Y,Y,https://doi.org/comp7,MeOH,Fraction from flash,None
4,AMR100005,AMRU3724C-01aa,2047,Ruby Johnson,20/10/2054,Rhodes MS,13/10/2054,12.6,N,AMRU3724C-01a,...,DCM: MeOH 2:1,InterestPC3,crime,346.5,N,Y,https://doi.org/comp6,DCM:MeOH 2:1,Fraction,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,AMR159996,TIC2041-434RH-01aba,2041,Ruby Johnson,19/11/2052,Rhodes Bioassay/Rhodes MS,12/11/2052,59.2,Y,TIC2041-434RH-01ab,...,DMSO,InterestPC7,both,6.7,Y,Y,https://doi.org/comp7,MeOH,HPLC Fraction,Used compound R in fractionation
59996,AMR159997,TIC2041-10200RH-01aba,2041,Joshua Keener,23/06/2054,Rhodes MS/NMR,16/06/2054,28.6,N,TIC2041-10200RH-01ab,...,"aq.MeOH, MeOH",InterestPC2,lose,163.2,N,Y,https://doi.org/comp4,DCM/MeOH,Fraction,Used compound R in fractionation
59997,AMR159998,AMRU4631B-01aaa,2045,Ruby Johnson,13/01/2055,Rhodes LC/MS + NMR & Bioassay,06/01/2055,23.9,Y,AMRU4631B-01aa,...,DCM: MeOH 2:1,InterestPC8,gas,60.2,Y,Y,https://doi.org/comp1,MeOH,HPLC Fraction,10:90 setting
59998,AMR159999,AMRU2153C-01baa,2045,Joshua Keener,28/01/2053,Rhodes MS,21/01/2053,22.9,Y,AMRU2153C-01ba,...,"aq.MeOH, MeOH",InterestPC8,sound,310.4,N,Y,https://doi.org/comp5,DCM:MeOH 2:1,SPE fraction,None


In [234]:
# Add PC to the start of all supplierIDs
pure_df['supplierID'] = 'PC-' + pure_df['supplierID']
pure_df

,amrID,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,...,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent,sampleType,comments
0,AMR100001,PC-TIC2041-1065RH-02aa,2041,Joshua Keener,10/08/2054,Rhodes Bioassay/Rhodes MS,03/08/2054,98.2,Y,TIC2041-1065RH-02a,...,DCM: MeOH 2:1,InterestPC6,former,78.1,N,Y,https://doi.org/comp7,DCM:MeOH 2:1,Fraction from flash,70:30 setting
1,AMR100002,PC-SAF43-258RJ-01aa,2043,Joshua Keener,16/08/2051,Rhodes Bioassay/Rhodes MS,09/08/2051,42.0,N,SAF43-258RJ-01a,...,"aq.MeOH, MeOH",InterestPC2,fish,284.5,Y,Y,Y,DCM:MeOH 2:1,SPE fraction,None
2,AMR100003,PC-SAF43-2536JR-02ba,2043,Joshua Keener,21/05/2050,Rhodes MS/NMR,14/05/2050,92.2,Y,SAF43-2536JR-02b,...,"aq.MeOH, MeOH",InterestPC1,Mr,190.5,N,Y,https://doi.org/comp2,DCM/MeOH,Fraction from flash,Used compound F in fractionation
3,AMR100004,PC-TIC2041-4160JR-02ab,2041,Ruby Johnson,24/06/2053,Rhodes Bioassay/Rhodes MS,17/06/2053,30.0,N,TIC2041-4160JR-02a,...,DMSO,InterestPC8,five,308.4,Y,Y,https://doi.org/comp7,MeOH,Fraction from flash,None
4,AMR100005,PC-AMRU3724C-01aa,2047,Ruby Johnson,20/10/2054,Rhodes MS,13/10/2054,12.6,N,AMRU3724C-01a,...,DCM: MeOH 2:1,InterestPC3,crime,346.5,N,Y,https://doi.org/comp6,DCM:MeOH 2:1,Fraction,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,AMR159996,PC-TIC2041-434RH-01aba,2041,Ruby Johnson,19/11/2052,Rhodes Bioassay/Rhodes MS,12/11/2052,59.2,Y,TIC2041-434RH-01ab,...,DMSO,InterestPC7,both,6.7,Y,Y,https://doi.org/comp7,MeOH,HPLC Fraction,Used compound R in fractionation
59996,AMR159997,PC-TIC2041-10200RH-01aba,2041,Joshua Keener,23/06/2054,Rhodes MS/NMR,16/06/2054,28.6,N,TIC2041-10200RH-01ab,...,"aq.MeOH, MeOH",InterestPC2,lose,163.2,N,Y,https://doi.org/comp4,DCM/MeOH,Fraction,Used compound R in fractionation
59997,AMR159998,PC-AMRU4631B-01aaa,2045,Ruby Johnson,13/01/2055,Rhodes LC/MS + NMR & Bioassay,06/01/2055,23.9,Y,AMRU4631B-01aa,...,DCM: MeOH 2:1,InterestPC8,gas,60.2,Y,Y,https://doi.org/comp1,MeOH,HPLC Fraction,10:90 setting
59998,AMR159999,PC-AMRU2153C-01baa,2045,Joshua Keener,28/01/2053,Rhodes MS,21/01/2053,22.9,Y,AMRU2153C-01ba,...,"aq.MeOH, MeOH",InterestPC8,sound,310.4,N,Y,https://doi.org/comp5,DCM:MeOH 2:1,SPE fraction,None


In [235]:
# Make InterestPC4 and N/A have no existing literature and no existing patent
pure_df.loc[pure_df['stereoComments'] == 'InterestPC4', 'existingPatent'] = 'N'
pure_df.loc[pure_df['stereoComments'] == 'InterestPC4', 'existingLiterature'] = 'N'
pure_df.loc[pure_df['stereoComments'] == 'N/A', 'existingPatent'] = 'N'
pure_df.loc[pure_df['stereoComments'] == 'N/A', 'existingLiterature'] = 'N'
pure_df

,amrID,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,...,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent,sampleType,comments
0,AMR100001,PC-TIC2041-1065RH-02aa,2041,Joshua Keener,10/08/2054,Rhodes Bioassay/Rhodes MS,03/08/2054,98.2,Y,TIC2041-1065RH-02a,...,DCM: MeOH 2:1,InterestPC6,former,78.1,N,Y,https://doi.org/comp7,DCM:MeOH 2:1,Fraction from flash,70:30 setting
1,AMR100002,PC-SAF43-258RJ-01aa,2043,Joshua Keener,16/08/2051,Rhodes Bioassay/Rhodes MS,09/08/2051,42.0,N,SAF43-258RJ-01a,...,"aq.MeOH, MeOH",InterestPC2,fish,284.5,Y,Y,Y,DCM:MeOH 2:1,SPE fraction,None
2,AMR100003,PC-SAF43-2536JR-02ba,2043,Joshua Keener,21/05/2050,Rhodes MS/NMR,14/05/2050,92.2,Y,SAF43-2536JR-02b,...,"aq.MeOH, MeOH",InterestPC1,Mr,190.5,N,Y,https://doi.org/comp2,DCM/MeOH,Fraction from flash,Used compound F in fractionation
3,AMR100004,PC-TIC2041-4160JR-02ab,2041,Ruby Johnson,24/06/2053,Rhodes Bioassay/Rhodes MS,17/06/2053,30.0,N,TIC2041-4160JR-02a,...,DMSO,InterestPC8,five,308.4,Y,Y,https://doi.org/comp7,MeOH,Fraction from flash,None
4,AMR100005,PC-AMRU3724C-01aa,2047,Ruby Johnson,20/10/2054,Rhodes MS,13/10/2054,12.6,N,AMRU3724C-01a,...,DCM: MeOH 2:1,InterestPC3,crime,346.5,N,Y,https://doi.org/comp6,DCM:MeOH 2:1,Fraction,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,AMR159996,PC-TIC2041-434RH-01aba,2041,Ruby Johnson,19/11/2052,Rhodes Bioassay/Rhodes MS,12/11/2052,59.2,Y,TIC2041-434RH-01ab,...,DMSO,InterestPC7,both,6.7,Y,Y,https://doi.org/comp7,MeOH,HPLC Fraction,Used compound R in fractionation
59996,AMR159997,PC-TIC2041-10200RH-01aba,2041,Joshua Keener,23/06/2054,Rhodes MS/NMR,16/06/2054,28.6,N,TIC2041-10200RH-01ab,...,"aq.MeOH, MeOH",InterestPC2,lose,163.2,N,Y,https://doi.org/comp4,DCM/MeOH,Fraction,Used compound R in fractionation
59997,AMR159998,PC-AMRU4631B-01aaa,2045,Ruby Johnson,13/01/2055,Rhodes LC/MS + NMR & Bioassay,06/01/2055,23.9,Y,AMRU4631B-01aa,...,DCM: MeOH 2:1,InterestPC8,gas,60.2,Y,Y,https://doi.org/comp1,MeOH,HPLC Fraction,10:90 setting
59998,AMR159999,PC-AMRU2153C-01baa,2045,Joshua Keener,28/01/2053,Rhodes MS,21/01/2053,22.9,Y,AMRU2153C-01ba,...,"aq.MeOH, MeOH",InterestPC8,sound,310.4,N,Y,https://doi.org/comp5,DCM:MeOH 2:1,SPE fraction,None


### Make MNP and AMR dataframes

In [236]:
# MNP spreadsheet: fractionFirst, fractionSecond, records not pure from fractionThird, and records not pure from fractionFourth
mnp_df = fractionFirst_df.append(fractionSecond_df, ignore_index=True)
mnp_df = mnp_df.append(fractionThird_df, ignore_index=True)
mnp_df = mnp_df.append(fractionFourth_df, ignore_index=True)
mnp_df

C:\Users\Emily\AppData\Local\Temp\ipykernel_4880\2236908492.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mnp_df = fractionFirst_df.append(fractionSecond_df, ignore_index=True)
C:\Users\Emily\AppData\Local\Temp\ipykernel_4880\2236908492.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mnp_df = mnp_df.append(fractionThird_df, ignore_index=True)
C:\Users\Emily\AppData\Local\Temp\ipykernel_4880\2236908492.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mnp_df = mnp_df.append(fractionFourth_df, ignore_index=True)


,mnpID,supplierID,amountAvailable,year,assignedBy,assignedDate,destinationOfSample,submittedDate,sampleType,concentration,solvent,project,sourceID,species,permitNo,researchGroup,submittedBy,level,comments
0,MNP00001,MOZ42-9322RH-01,96.6,2042,Joshua Keener,08/11/2054,RU Bioassay,01/11/2054,HPLC Fraction,50.0,DCM/MeOH,Marine,MOZ42-9322RH,six,RES2042,Roodt,Priscilla Evans,1,Used compound Y in fractionation
1,MNP00002,MOZ42-4072RH-01,44.1,2042,Joshua Keener,28/11/2052,CD Experiment,21/11/2052,Fraction,0.1,DCM:MeOH 2:1,Marine,MOZ42-4072RH,decide,RES2042,Roodt,Amanda Bowker,1,None
2,MNP00003,SAF43-11100RH-01,0.7,2043,Joshua Keener,13/11/2054,RU LC/MS,06/11/2054,HPLC Fraction,1.0,DCM:MeOH 2:1,Marine,SAF43-11100RH,economy,RES2043,Smith,Ruby Johnson,1,None
3,MNP00004,6-56*#1671R3-01,91.1,2046,Ruby Johnson,25/11/2052,CD Experiment,18/11/2052,SPE fraction,0.1,MeOH,Marine,6-56*#1671R3,statement,RES2046,Smith,Priscilla Evans,1,Used compound X in fractionation
4,MNP00005,AMRU542C-01,93.9,2047,Joshua Keener,27/11/2050,CD Experiment,20/11/2050,HPLC Fraction,0.5,DCM:MeOH 2:1,Microbial,AMRU542C,common,RES2047,Smith,Priscilla Evans,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259995,MNP259996,AMRU4335B-01aaa,71.4,2045,Joshua Keener,09/06/2053,CD Experiment,02/06/2053,HPLC Fraction,50.0,MeOH,Microbial,AMRU4335B-01aa,perform,RES2045,Roodt,Priscilla Evans,4,Combination setting
259996,MNP259997,SAF43-4656JR-01aac,7.9,2043,Joshua Keener,08/07/2052,RU Bioassay,01/07/2052,Fraction from flash,0.1,DCM:MeOH 2:1,Marine,SAF43-4656JR-01aa,body,RES2043,Smith,Priscilla Evans,4,None
259997,MNP259998,MOZ42-1272RJ-01abc,72.9,2042,Ruby Johnson,20/10/2050,Rhodes MS,13/10/2050,SPE fraction,20.0,DCM/MeOH,Marine,MOZ42-1272RJ-01ab,east,RES2042,Smith,Ruby Johnson,4,Used compound R in fractionation
259998,MNP259999,MOZ42-6381RH-01baa,89.0,2042,Joshua Keener,03/09/2051,RU Bioassay,27/08/2051,HPLC Fraction,50.0,DCM:MeOH 2:1,Marine,MOZ42-6381RH-01ba,describe,RES2042,Roodt,Amanda Bowker,4,Used compound R in fractionation


In [237]:
original_df.drop(['submitterInitials'], axis=1, inplace=True)
amr_df = pd.concat([original_df, pure_df], ignore_index=True)
amr_df

,amrID,supplierID,year,assignedBy,assignedDate,destinationOfSample,submittedDate,amountAvailable,synthesisPotential,sourceID,...,solubility,stereoComments,smileStructure,mw,additionalMetadata,existingPatent,existingLiterature,solvent,sampleType,comments
0,AMR00001,AAP_1,2048,Ruby Johnson,08/10/2053,Rhodes Bioassay/Rhodes MS,01/10/2053,30.2,N/A,AAP_1,...,N/A,N/A,N/A,N/A,N,N,Y,DMSO,NaN,NaN
1,AMR00002,MOZ42-1RJ,2042,Ruby Johnson,06/01/2051,Membrane Disruption assay,30/12/2050,26.6,N/A,MOZ42-1,...,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice3,DMSO,NaN,NaN
2,AMR00003,TIC2041-1RJ,2041,Joshua Keener,31/01/2050,Rhodes MS/NMR,24/01/2050,9.3,N/A,TIC2041-1,...,N/A,N/A,N/A,N/A,Y,N,https//doi.org/choice6,DMSO,NaN,NaN
3,AMR00004,MOZ42-2JR,2042,Ruby Johnson,16/10/2050,Membrane Disruption assay,09/10/2050,31.7,N/A,MOZ42-2,...,N/A,N/A,N/A,N/A,N,N,https//doi.org/choice1,DMSO,NaN,NaN
4,AMR00005,TIC2041-2JR,2041,Joshua Keener,21/12/2054,Rhodes MS/NMR,14/12/2054,66.1,N/A,TIC2041-2,...,N/A,N/A,N/A,N/A,Y,N,N,DMSO,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159995,AMR159996,PC-TIC2041-434RH-01aba,2041,Ruby Johnson,19/11/2052,Rhodes Bioassay/Rhodes MS,12/11/2052,59.2,Y,TIC2041-434RH-01ab,...,DMSO,InterestPC7,both,6.7,Y,Y,https://doi.org/comp7,MeOH,HPLC Fraction,Used compound R in fractionation
159996,AMR159997,PC-TIC2041-10200RH-01aba,2041,Joshua Keener,23/06/2054,Rhodes MS/NMR,16/06/2054,28.6,N,TIC2041-10200RH-01ab,...,"aq.MeOH, MeOH",InterestPC2,lose,163.2,N,Y,https://doi.org/comp4,DCM/MeOH,Fraction,Used compound R in fractionation
159997,AMR159998,PC-AMRU4631B-01aaa,2045,Ruby Johnson,13/01/2055,Rhodes LC/MS + NMR & Bioassay,06/01/2055,23.9,Y,AMRU4631B-01aa,...,DCM: MeOH 2:1,InterestPC8,gas,60.2,Y,Y,https://doi.org/comp1,MeOH,HPLC Fraction,10:90 setting
159998,AMR159999,PC-AMRU2153C-01baa,2045,Joshua Keener,28/01/2053,Rhodes MS,21/01/2053,22.9,Y,AMRU2153C-01ba,...,"aq.MeOH, MeOH",InterestPC8,sound,310.4,N,Y,https://doi.org/comp5,DCM:MeOH 2:1,SPE fraction,None


### Strain Spreadsheet

In [238]:
strain_df = amr_df.copy(deep=True)
new_strain = strain_df.loc[strain_df['sourceType'] == "Bacterial"]
new_strain = new_strain[['supplierID']].reset_index(drop=True)
new_strain

,supplierID
0,AAP_1
1,AAP_2
2,AAP_3
3,AAP_4
4,AAP_5
...,...
11170,AAP_11171
11171,AAP_11172
11172,AAP_11173
11173,AAP_11174


In [239]:
fake = Faker()
Faker.seed(1233)
random.seed(1233)

class Provider(BaseProvider):
    cultivations = ['MEM', 'WYM', 'PDM', 'PDM+I/O']

    def cultivationMedia(self):
        return random.choice(self.cultivations)

# Add the Provider to our faker object
fake.add_provider(Provider)

# Change:
# AMR ID, supplierID, destination, synthesis, sourceType, pure, ms, nmr, researchGroup, solubility, 
# stereoComments, SMILE, mw, additionalMetadata, existingPatent, existingLiterature

def create_strain_data(fake, no_of_rows):
    columns = ['cultivationMedia']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

strain_df = create_strain_data(fake, len(new_strain))
strain_df = pd.concat([new_strain, strain_df], axis=1)
strain_df

,supplierID,cultivationMedia
0,AAP_1,PDM
1,AAP_2,PDM
2,AAP_3,PDM
3,AAP_4,WYM
4,AAP_5,PDM
...,...,...
11170,AAP_11171,MEM
11171,AAP_11172,WYM
11172,AAP_11173,MEM
11173,AAP_11174,WYM


### Bioassay spreadsheet

processID -> bioassay

responsiblePi -> bioassay

researcherId -> bioassay + person

dateReceived -> bioassay

sampleID -> associated with extract/fraction (repeated)

sourceSpecimen -> associated with extract/fraction (repeated)

molecularID -> bioassay

cultivationMedia -> strain

diluent -> bioassay

concentration -> bioassay

volume -> removed

amount -> bioassay

all other values (ecoli_viability to hppk_activity) -> bioassay

some have been removed

In [240]:

# Need amr/mnpID, dateAssigned, species
amr_proc = amr_df[["amrID", "researchGroup", "submittedBy", "assignedDate", "species", "destinationOfSample"]]
amr_proc.rename(columns={'amrID':'processID'}, inplace=True)
mnp_proc = mnp_df[["mnpID", "researchGroup", "submittedBy", "assignedDate", "species", "destinationOfSample"]]
mnp_proc.rename(columns={'mnpID':'processID'}, inplace=True)
proc_df = pd.concat([amr_proc, mnp_proc], ignore_index=True)
# Get both bioassay and pk
df_all = proc_df[proc_df.destinationOfSample.str.contains("Bioassay", regex=False)].append(proc_df[proc_df.destinationOfSample.str.contains("PK", regex=False)])
df_all.drop_duplicates(keep='first', inplace=True)
# Get only pk
pks = proc_df[proc_df.destinationOfSample.str.contains("PK", regex=False)]
# Find values where indexes are same, and put True in pk_requested
bioassay_df = df_all.merge(pks.drop_duplicates(), on=["processID", "researchGroup", "submittedBy", "assignedDate", "species", "destinationOfSample"], how='left', indicator=True)
bioassay_df['_merge'] = (bioassay_df['_merge'] == 'both')
bioassay_df.rename(columns={'_merge':'pk_requested'}, inplace=True)

# bioFirst_df = amr_df.sample(n=60, random_state=1234, ignore_index=True)
# bioFirst_df = bioFirst_df[["amrID", "researchGroup", "submittedBy", "assignedDate", "species"]]
# bioFirst_df.rename(columns={'amrID':'processID'}, inplace=True)
# bioSecond_df = mnp_df.sample(n=40, random_state=1234, ignore_index=True)
# bioSecond_df = bioSecond_df[["mnpID", "researchGroup", "submittedBy", "assignedDate", "species"]]
# bioSecond_df.rename(columns={'mnpID':'processID'}, inplace=True)
# bioassay_df = pd.concat([bioFirst_df, bioSecond_df], ignore_index=True)
bioassay_df

C:\Users\Emily\AppData\Local\Temp\ipykernel_4880\2488688655.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  amr_proc.rename(columns={'amrID':'processID'}, inplace=True)
C:\Users\Emily\AppData\Local\Temp\ipykernel_4880\2488688655.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mnp_proc.rename(columns={'mnpID':'processID'}, inplace=True)
C:\Users\Emily\AppData\Local\Temp\ipykernel_4880\2488688655.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = proc_df[proc_df.destinationOfSample.str.contains("Bioassay", regex=Fal

,processID,researchGroup,submittedBy,assignedDate,species,destinationOfSample,pk_requested
0,AMR00001,Preston,Ruby Johnson,08/10/2053,suddenly,Rhodes Bioassay/Rhodes MS,False
1,AMR00006,Smith,Julia Reed,26/04/2050,know,Rhodes Bioassay/Rhodes MS,False
2,AMR00007,Wellington,Robert Hobson,25/09/2050,top,Rhodes Bioassay,False
3,AMR00008,Smith,Julia Reed,09/11/2050,offer,"Rhodes Bioassay, PK assay",True
4,AMR00009,Wellington,Robert Hobson,13/01/2053,though,Rhodes Bioassay/Rhodes MS,False
...,...,...,...,...,...,...,...
144073,MNP259981,Roodt,Ruby Johnson,12/12/2050,style,RU Bioassay,False
144074,MNP259987,Roodt,Ruby Johnson,08/10/2054,summer,RU Bioassay,False
144075,MNP259991,Smith,Amanda Bowker,05/07/2052,score,RU Bioassay,False
144076,MNP259997,Smith,Priscilla Evans,08/07/2052,body,RU Bioassay,False


In [241]:
class Provider(BaseProvider):
    solvents = ['MeOH', 'Dried down']
    concentrations = [0.25, 0.1, 0.5, 1.0, 20.0, 50.0]

    def diluent(self):
        return random.choice(self.solvents)
    
    def concentration(self):
        return random.choice(self.concentrations)

    def amount(self):
        return (round (random.uniform(0.0, 1.0)*10, 1))

    def ecoliV(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def ecoliSD(self):
        return (round (random.uniform(0.0, 1.0)*10, 1))
    
    def saureusV(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def saureusSD(self):
        return (round (random.uniform(0.0, 1.0)*10, 1))
    
    def pareuV(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def pareuSD(self):
        return (round (random.uniform(0.0, 1.0)*10, 1))
    
    def saureusBioV(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def saureusBioSD(self):
        return (round (random.uniform(0.0, 1.0)*10, 1))
    
    def pareuBioV(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def pareuBioSD(self):
        return (round (random.uniform(0.0, 1.0)*10, 1))
    
    def cytotoxV(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def cytotoxSD(self):
        return (round (random.uniform(0.0, 1.0)*10, 1))
    
    def pkActivity(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def dxrActivity(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def confirm_dxrActivity(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    
    def hppkActivity(self):
        return (round (random.uniform(0.0, 1.0)*110, 1))
    

# Add the Provider to our faker object
fake.add_provider(Provider)

# Need: responsiblePI, researcher, dateReceived, diluent, concentration, amount, ecoliV, ecoliSD, saureusV, saureusSD, pareuV, pareuSD, saureusBioV, saureusBioSD,
# paureuBioV, paureuBioSD, cytotoxV, cytotoxSD, pkActivity, dxrActivity, confirm_dxrActivity, hppkActivity


def create_bioassay_data(fake, no_of_rows):
    columns = ['diluent', 'concentration', 'amount', 'ecoliV', 'ecoliSD', 'saureusV', 'saureusSD', 'pareuV', 'pareuSD', 'saureusBioV', 'saureusBioSD',
                'pareuBioV', 'pareuBioSD', 'cytotoxV', 'cytotoxSD', 'pkActivity', 'dxrActivity', 'confirm_dxrActivity', 'hppkActivity']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

rest_bio_df = create_bioassay_data(fake, len(bioassay_df))
rest_bio_df

,diluent,concentration,amount,ecoliV,ecoliSD,saureusV,saureusSD,pareuV,pareuSD,saureusBioV,saureusBioSD,pareuBioV,pareuBioSD,cytotoxV,cytotoxSD,pkActivity,dxrActivity,confirm_dxrActivity,hppkActivity
0,Dried down,0.10,8.8,90.9,7.8,64.1,1.0,80.8,4.6,48.1,7.1,17.8,2.9,68.2,1.3,75.0,103.4,5.3,71.1
1,Dried down,50.00,6.7,7.7,1.4,91.2,7.7,86.5,0.1,37.7,7.4,66.8,4.8,87.6,4.4,68.4,33.9,54.7,96.7
2,Dried down,0.10,8.7,76.1,2.3,69.9,2.2,78.3,4.9,44.7,8.9,48.6,7.8,39.6,3.9,33.7,47.3,100.3,83.4
3,MeOH,1.00,4.3,13.2,7.8,91.3,7.9,80.9,0.4,52.8,2.3,35.9,8.4,14.1,5.0,1.7,55.1,73.7,0.8
4,MeOH,0.50,6.0,5.3,1.9,29.3,3.8,3.4,5.6,83.9,8.6,82.3,8.6,2.7,1.1,79.0,77.8,65.7,39.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144073,Dried down,0.10,9.9,4.8,7.0,43.1,1.1,5.0,3.5,75.8,2.0,20.5,0.9,7.2,6.8,71.9,47.4,97.8,103.8
144074,MeOH,0.25,3.0,108.6,6.7,67.3,5.5,72.2,6.5,36.2,4.5,42.4,2.9,32.4,6.8,94.1,33.1,22.5,13.8
144075,MeOH,0.50,10.0,4.8,9.3,60.0,2.7,91.3,8.2,38.2,0.1,30.2,4.7,1.1,0.7,32.1,64.9,52.0,17.2
144076,Dried down,0.25,2.5,22.2,1.5,62.8,5.9,93.8,2.1,3.8,5.1,21.8,3.1,108.0,8.8,104.9,88.1,14.1,30.0


In [242]:
bio_df = pd.concat([bioassay_df, rest_bio_df], axis=1)
bio_df.rename(columns={'submittedBy':'researcher', 'species':'molecularID', 'assignedDate':'dateReceived', 'researchGroup':'responsiblePI'}, inplace=True)
bio_df

,processID,responsiblePI,researcher,dateReceived,molecularID,destinationOfSample,pk_requested,diluent,concentration,amount,...,saureusBioV,saureusBioSD,pareuBioV,pareuBioSD,cytotoxV,cytotoxSD,pkActivity,dxrActivity,confirm_dxrActivity,hppkActivity
0,AMR00001,Preston,Ruby Johnson,08/10/2053,suddenly,Rhodes Bioassay/Rhodes MS,False,Dried down,0.10,8.8,...,48.1,7.1,17.8,2.9,68.2,1.3,75.0,103.4,5.3,71.1
1,AMR00006,Smith,Julia Reed,26/04/2050,know,Rhodes Bioassay/Rhodes MS,False,Dried down,50.00,6.7,...,37.7,7.4,66.8,4.8,87.6,4.4,68.4,33.9,54.7,96.7
2,AMR00007,Wellington,Robert Hobson,25/09/2050,top,Rhodes Bioassay,False,Dried down,0.10,8.7,...,44.7,8.9,48.6,7.8,39.6,3.9,33.7,47.3,100.3,83.4
3,AMR00008,Smith,Julia Reed,09/11/2050,offer,"Rhodes Bioassay, PK assay",True,MeOH,1.00,4.3,...,52.8,2.3,35.9,8.4,14.1,5.0,1.7,55.1,73.7,0.8
4,AMR00009,Wellington,Robert Hobson,13/01/2053,though,Rhodes Bioassay/Rhodes MS,False,MeOH,0.50,6.0,...,83.9,8.6,82.3,8.6,2.7,1.1,79.0,77.8,65.7,39.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144073,MNP259981,Roodt,Ruby Johnson,12/12/2050,style,RU Bioassay,False,Dried down,0.10,9.9,...,75.8,2.0,20.5,0.9,7.2,6.8,71.9,47.4,97.8,103.8
144074,MNP259987,Roodt,Ruby Johnson,08/10/2054,summer,RU Bioassay,False,MeOH,0.25,3.0,...,36.2,4.5,42.4,2.9,32.4,6.8,94.1,33.1,22.5,13.8
144075,MNP259991,Smith,Amanda Bowker,05/07/2052,score,RU Bioassay,False,MeOH,0.50,10.0,...,38.2,0.1,30.2,4.7,1.1,0.7,32.1,64.9,52.0,17.2
144076,MNP259997,Smith,Priscilla Evans,08/07/2052,body,RU Bioassay,False,Dried down,0.25,2.5,...,3.8,5.1,21.8,3.1,108.0,8.8,104.9,88.1,14.1,30.0


In [243]:
# Turn off pk activity for those not requested
bio_df.loc[bio_df['pk_requested'] == False, 'pkActivity'] = None
# Remove destinationOfSample, pk_requested
bio_df.drop(['destinationOfSample', 'pk_requested'], axis=1, inplace=True)
bio_df

,processID,responsiblePI,researcher,dateReceived,molecularID,diluent,concentration,amount,ecoliV,ecoliSD,...,saureusBioV,saureusBioSD,pareuBioV,pareuBioSD,cytotoxV,cytotoxSD,pkActivity,dxrActivity,confirm_dxrActivity,hppkActivity
0,AMR00001,Preston,Ruby Johnson,08/10/2053,suddenly,Dried down,0.10,8.8,90.9,7.8,...,48.1,7.1,17.8,2.9,68.2,1.3,NaN,103.4,5.3,71.1
1,AMR00006,Smith,Julia Reed,26/04/2050,know,Dried down,50.00,6.7,7.7,1.4,...,37.7,7.4,66.8,4.8,87.6,4.4,NaN,33.9,54.7,96.7
2,AMR00007,Wellington,Robert Hobson,25/09/2050,top,Dried down,0.10,8.7,76.1,2.3,...,44.7,8.9,48.6,7.8,39.6,3.9,NaN,47.3,100.3,83.4
3,AMR00008,Smith,Julia Reed,09/11/2050,offer,MeOH,1.00,4.3,13.2,7.8,...,52.8,2.3,35.9,8.4,14.1,5.0,1.7,55.1,73.7,0.8
4,AMR00009,Wellington,Robert Hobson,13/01/2053,though,MeOH,0.50,6.0,5.3,1.9,...,83.9,8.6,82.3,8.6,2.7,1.1,NaN,77.8,65.7,39.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144073,MNP259981,Roodt,Ruby Johnson,12/12/2050,style,Dried down,0.10,9.9,4.8,7.0,...,75.8,2.0,20.5,0.9,7.2,6.8,NaN,47.4,97.8,103.8
144074,MNP259987,Roodt,Ruby Johnson,08/10/2054,summer,MeOH,0.25,3.0,108.6,6.7,...,36.2,4.5,42.4,2.9,32.4,6.8,NaN,33.1,22.5,13.8
144075,MNP259991,Smith,Amanda Bowker,05/07/2052,score,MeOH,0.50,10.0,4.8,9.3,...,38.2,0.1,30.2,4.7,1.1,0.7,NaN,64.9,52.0,17.2
144076,MNP259997,Smith,Priscilla Evans,08/07/2052,body,Dried down,0.25,2.5,22.2,1.5,...,3.8,5.1,21.8,3.1,108.0,8.8,NaN,88.1,14.1,30.0


#### Collection spreadsheets

Sample ID -> collection

Common Name -> collection - comes with taxonomic data

Other IDs -> collection

Country -> country

Province -> province

City -> city

Local Area -> localarea

Station -> station

Date -> collection

Latitude -> site

Longitude -> site

Depth min -> site

Depth max -> site

Collected by -> person_collected

Identified by -> collection/person

Classification -> collection

Other description -> collection

Kingdom to Species - use actual data

In [244]:
# Years with collection information - 2041, 2042, 2043
collectTIC = original_df.loc[original_df['year'] == 2041]
collectMOZ = original_df.loc[original_df['year'] == 2042]
collectSAF = original_df.loc[original_df['year'] == 2043]
collection_df = pd.concat([collectTIC, collectMOZ, collectSAF], ignore_index=True)
collection_df = collection_df[["sourceID", "permitNo"]]
collection_df

,sourceID,permitNo
0,TIC2041-1,RES2041
1,TIC2041-2,RES2041
2,TIC2041-3,RES2041
3,TIC2041-4,RES2041
4,TIC2041-5,RES2041
...,...,...
33361,SAF43-11247,RES2043
33362,SAF43-11248,RES2043
33363,SAF43-11249,RES2043
33364,SAF43-11250,RES2043


In [245]:
# Faker for Country, collected by, identified by, classification
fake = Faker()
Faker.seed(1235)
random.seed(1235)

class Provider(BaseProvider):
    collectors = ['Mandy Peterson', 'Clive Morrison', 'Trevor Andrews']
    collectorsTwo = ['Mandy Peterson', 'Clive Morrison', 'Trevor Andrews']
    identifiers = ['Clive Morrison', 'Harry Warner']
    classifications = ['Preliminary' ,'Complete', 'Questionable', 'Tentative']
    
    def collectedByOne(self):
        return random.choice(self.collectors)

    def collectedByTwo(self):
        return random.choice(self.collectorsTwo)

    def identifiedBy(self):
        return random.choice(self.identifiers)
    
    def classification(self):
        return random.choice(self.classifications)
    
# Add the Provider to our faker object
fake.add_provider(Provider)

# First change any columns that must be the same for all fractions from a particular source
# Submitted by, research group, project, concentration, solvent, species, sample type, assigned by, submitted date and assigned date (but will be same either way), destination

def create_collection_data(fake, no_of_rows):
    columns = ['collectedByOne', 'collectedByTwo', 'identifiedBy', 'classification']
    data =  {column: [getattr(fake, column)() for _ in range(no_of_rows)] for column in columns}
    df = pd.DataFrame(data=data)
    df = df[columns]

    return df

fakerCollection_df = create_collection_data(fake, len(collection_df))
# Make collectedBy and collectedByTwo one column

collectedOneList = fakerCollection_df['collectedByOne'].tolist()
collectedTwoList = fakerCollection_df['collectedByTwo'].tolist()
collectedTogether = []

for one, two in zip(collectedOneList, collectedTwoList):
    if one == two:
        newStr = one
    elif one == "Clive Morrison" or two == "Clive Morrison":
        newStr = one + " and " + two
    else:
        newStr = one + "; " + two
    collectedTogether.append(newStr)

togetherSeries = pd.Series(collectedTogether)
fakerCollection_df.insert(1, 'collectedBy', togetherSeries)
fakerCollection_df.drop(["collectedByOne", "collectedByTwo"], axis=1, inplace=True)
fakerCollection_df

,collectedBy,identifiedBy,classification
0,Trevor Andrews,Clive Morrison,Complete
1,Clive Morrison,Clive Morrison,Complete
2,Trevor Andrews and Clive Morrison,Clive Morrison,Questionable
3,Trevor Andrews and Clive Morrison,Harry Warner,Questionable
4,Clive Morrison and Trevor Andrews,Clive Morrison,Tentative
...,...,...,...
33361,Clive Morrison,Harry Warner,Complete
33362,Clive Morrison and Trevor Andrews,Harry Warner,Complete
33363,Trevor Andrews; Mandy Peterson,Clive Morrison,Complete
33364,Clive Morrison,Clive Morrison,Questionable


In [246]:
# Get the full location data for each country
# South Africa - TIC, Mozambique - MOZ, Seychelles (for SAF, even though not in real life)
# Make lists for strings to use in locations

def makeCountryLocation(numLocals, givenSeed):
    random.seed(givenSeed)
    local = []
    latitude = []
    longitude = []
    for i in range(numLocals):
        local.append(fake.word().capitalize() + " " + fake.word())
        latitude.append(round(random.uniform( -90,  90), 4))
        longitude.append(round(random.uniform(-180, 180), 5))

    station = []
    for i in range(numLocals * 2):
        station.append(fake.word().capitalize() + " " + fake.word())
        if i % 2 == 0:
            lat = latitude[i//2] - round(random.uniform(0, 1), 4)
            long = longitude[i//2] - round(random.uniform(0, 1), 4)
            latitude.append(round(lat, 4))
            longitude.append(round(long, 4))

    min = []
    max = []
    for i in range(numLocals * 4):
        min.append(random.randint(0, 15))
        max.append(min[i] + random.randint(4, 15))
        if i % 2 == 0:
            lat = latitude[i//2] + round(random.uniform(0, 0.1), 4)
            long = longitude[i//2] + round(random.uniform(0, 0.1), 4)
            latitude.append(round(lat, 4))
            longitude.append(round(long, 4))

    return local, station, latitude, longitude, min, max

# Uses strings to make up full locations
def makeLocations(numRecords, localList, stationList, latitudeList, longitudeList, minList, maxList):
    local = []
    station = []
    latitude = []
    longitude = []
    min = []
    max = []
    for i in range(numRecords):
        local.append(localList[i % len(localList)])
        station.append(stationList[i % len(stationList)])
        latitude.append(latitudeList[i % len(latitudeList)])
        longitude.append(longitudeList[i % len(longitudeList)])
        min.append(minList[i % len(minList)])
        max.append(maxList[i % len(maxList)])

    return (pd.DataFrame({'localArea':local, 'station':station, 'latitude':latitude, 'longitude':longitude, 'depthMin':min, 'depthMax':max}))
        
local, station, latitude, longitude, min, max = makeCountryLocation(2, 1234)
SAlocation_df = makeLocations(len(collectTIC), local, station, latitude, longitude, min, max)
SAlocation_df.insert(0, 'country', 'South Africa')
SAlocation_df.insert(1, 'region', 'Eastern Cape')
SAlocation_df.insert(2, 'city', fake.word().capitalize())
local, station, latitude, longitude, min, max = makeCountryLocation(2, 1233)
MOZlocation_df = makeLocations(len(collectMOZ), local, station, latitude, longitude, min, max)
MOZlocation_df.insert(0, 'country', 'Mozambique')
MOZlocation_df.insert(1, 'region', 'Gaza')
MOZlocation_df.insert(2, 'city', fake.word().capitalize())
local, station, latitude, longitude, min, max = makeCountryLocation(2, 1232)
SAFlocation_df = makeLocations(len(collectSAF), local, station, latitude, longitude, min, max)
SAFlocation_df.insert(0, 'country', 'Seychelles')
SAFlocation_df.insert(1, 'region', 'Bel Air')
SAFlocation_df.insert(2, 'city', fake.word().capitalize())
location_df = pd.concat([SAlocation_df, MOZlocation_df, SAFlocation_df], ignore_index=True)
location_df

,country,region,city,localArea,station,latitude,longitude,depthMin,depthMax
0,South Africa,Eastern Cape,Race,When risk,Thousand person,83.9616,-21.33626,11,18
1,South Africa,Eastern Cape,Race,Language north,Many include,-88.6515,147.95135,0,9
2,South Africa,Eastern Cape,Race,When risk,Church second,83.0223,-21.91850,15,28
3,South Africa,Eastern Cape,Race,Language north,Happy set,-89.3231,147.86740,3,7
4,South Africa,Eastern Cape,Race,When risk,Thousand person,83.9633,-21.23760,15,22
...,...,...,...,...,...,...,...,...,...
33361,Seychelles,Bel Air,Include,Hot sound,Develop public,13.9136,71.58080,10,15
33362,Seychelles,Bel Air,Include,American trade,Do rule,72.7755,-129.35330,10,25
33363,Seychelles,Bel Air,Include,Hot sound,Girl study,13.9299,72.47976,3,18
33364,Seychelles,Bel Air,Include,American trade,Shake son,73.4614,-128.86145,0,13


In [247]:
tax_df = pd.read_excel("taxonomy.xlsx")
tax_df = tax_df.head(len(location_df))
tax_df = pd.concat([tax_df] * 477).reset_index(drop=True)
tax_df.drop(tax_df.index[33366:], inplace=True)
tax_df

,Kingdom,Phylum,Subphylum,Class,Subclass,Superorder,Order,Suborder,Family,Subfamily,Genus,Subgenus,Species
0,Animalia,Chordata,Urochordata -Tunicata,Ascidiacea,NaN,Enterogona,Aplousobranchia,NaN,Pseudodistomidae,NaN,Pseudodistoma,NaN,sp 015 RSA SPN roseum
1,Animalia,Porifera,NaN,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Cyclacanthia,NaN,bellae
2,Animalia,Porifera,NaN,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Tsitsikamma,NaN,sp 004 RSA nguni
3,Animalia,Porifera,NaN,Demospongiae,NaN,NaN,Dictyoceratida,NaN,Ircinidae,NaN,Psammocinia,NaN,sp 001 RSA MH NMMU
4,Animalia,Porifera,NaN,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Tsitsikamma,NaN,sp 004 RSA nguni
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33361,Animalia,Porifera,NaN,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Tsitsikamma,NaN,pedunculata
33362,Animalia,Porifera,NaN,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Tsitsikamma,NaN,pedunculata
33363,Animalia,Porifera,NaN,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Tsitsikamma,NaN,pedunculata
33364,Animalia,Porifera,NaN,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Tsitsikamma,NaN,pedunculata


In [248]:
Faker.seed(1234)
# Combining what we have together:
collection = pd.concat([collection_df, location_df, fakerCollection_df, tax_df], axis=1)
collection.insert(1, 'otherIDs', 'NaN')
collection.insert(15, 'otherDescription', 'NaN')
date = []
for i in range(len(collectTIC)):
    date.append(fake.date_between_dates(date_start = datetime(2041, 1, 1), date_end = datetime(2041, 12, 31)).strftime('%d/%m/%Y'))
for i in range(len(collectMOZ)):
    date.append(fake.date_between_dates(date_start = datetime(2042, 1, 1), date_end = datetime(2042, 12, 31)).strftime('%d/%m/%Y'))
for i in range(len(collectSAF)):
    date.append(fake.date_between_dates(date_start = datetime(2043, 1, 1), date_end = datetime(2043, 12, 31)).strftime('%d/%m/%Y'))
collection.insert(2, 'collectedDate', date)
collection

,sourceID,otherIDs,collectedDate,permitNo,country,region,city,localArea,station,latitude,...,Class,Subclass,Superorder,Order,Suborder,Family,Subfamily,Genus,Subgenus,Species
0,TIC2041-1,NaN,30/10/2041,RES2041,South Africa,Eastern Cape,Race,When risk,Thousand person,83.9616,...,Ascidiacea,NaN,Enterogona,Aplousobranchia,NaN,Pseudodistomidae,NaN,Pseudodistoma,NaN,sp 015 RSA SPN roseum
1,TIC2041-2,NaN,21/06/2041,RES2041,South Africa,Eastern Cape,Race,Language north,Many include,-88.6515,...,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Cyclacanthia,NaN,bellae
2,TIC2041-3,NaN,15/02/2041,RES2041,South Africa,Eastern Cape,Race,When risk,Church second,83.0223,...,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Tsitsikamma,NaN,sp 004 RSA nguni
3,TIC2041-4,NaN,03/01/2041,RES2041,South Africa,Eastern Cape,Race,Language north,Happy set,-89.3231,...,Demospongiae,NaN,NaN,Dictyoceratida,NaN,Ircinidae,NaN,Psammocinia,NaN,sp 001 RSA MH NMMU
4,TIC2041-5,NaN,05/02/2041,RES2041,South Africa,Eastern Cape,Race,When risk,Thousand person,83.9633,...,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Tsitsikamma,NaN,sp 004 RSA nguni
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33361,SAF43-11247,NaN,05/11/2043,RES2043,Seychelles,Bel Air,Include,Hot sound,Develop public,13.9136,...,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Tsitsikamma,NaN,pedunculata
33362,SAF43-11248,NaN,17/04/2043,RES2043,Seychelles,Bel Air,Include,American trade,Do rule,72.7755,...,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Tsitsikamma,NaN,pedunculata
33363,SAF43-11249,NaN,13/12/2043,RES2043,Seychelles,Bel Air,Include,Hot sound,Girl study,13.9299,...,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Tsitsikamma,NaN,pedunculata
33364,SAF43-11250,NaN,03/06/2043,RES2043,Seychelles,Bel Air,Include,American trade,Shake son,73.4614,...,Demospongiae,NaN,NaN,Poecilosclerida,Latrunculina,Latrunculiidae,NaN,Tsitsikamma,NaN,pedunculata


### Converting to Excel spreadsheet

In [249]:
datatoexcelAMR = pd.ExcelWriter('TrialDummy1.xlsx')
datatoexcelMNP = pd.ExcelWriter('TrialDummy2.xlsx')
datatoexcelBioassay = pd.ExcelWriter('bioassayDummy.xlsx')
datatoexcelLocation = pd.ExcelWriter('location.xlsx')
datatoexcelCollection = pd.ExcelWriter('collection.xlsx')
datatoexcelStrain = pd.ExcelWriter('strain.xlsx')

amr_df.to_excel(datatoexcelAMR, index=False)
mnp_df.to_excel(datatoexcelMNP, index=False)
bio_df.to_excel(datatoexcelBioassay, index=False)
location_df.to_excel(datatoexcelLocation, index=False)
collection.to_excel(datatoexcelCollection, index=False)
strain_df.to_excel(datatoexcelStrain, index=False)

datatoexcelAMR.save()
datatoexcelMNP.save()
datatoexcelBioassay.save()
datatoexcelLocation.save()
datatoexcelCollection.save()
datatoexcelStrain.save()

C:\Users\Emily\AppData\Local\Temp\ipykernel_4880\12166985.py:15: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  datatoexcelAMR.save()
C:\Users\Emily\AppData\Local\Temp\ipykernel_4880\12166985.py:16: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  datatoexcelMNP.save()
C:\Users\Emily\AppData\Local\Temp\ipykernel_4880\12166985.py:17: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  datatoexcelBioassay.save()
C:\Users\Emily\AppData\Local\Temp\ipykernel_4880\12166985.py:18: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  datatoexcelLocation.save()
C:\Users\Emily\AppData\Local\Temp\ipykernel_4880\12166985.py:19: FutureWarning: save is not part of the public API, usage can give une